In [ ]:
BASE_PATH = "/Users/susieguo/Desktop/parquet_files/"

In [ ]:
import pandas as pd

In [ ]:
tiktok__order_items = pd.read_parquet(f"{BASE_PATH}stg_tiktok_shop__order_items.parquet")
tiktok__order_item_metrics = pd.read_parquet(f"{BASE_PATH}stg_tiktok_shop__order_metrics.parquet")
shopify__order_items = pd.read_parquet(f"{BASE_PATH}stg_shopify__order_items.parquet")
shopify__order_item_metrics = pd.read_parquet(f"{BASE_PATH}stg_shopify__order_metrics.parquet")
amazon_order_item_metrics = pd.read_parquet(f"{BASE_PATH}stg_amazon__order_item_metrics.parquet")
amazon_order_metrics = pd.read_parquet(f"{BASE_PATH}stg_amazon__order_metrics.parquet")


In [ ]:
tiktok_merged_df = pd.merge(
    tiktok__order_item_metrics,
    tiktok__order_items[["order_id", "local_order_ts","product_name"]],
    on="order_id",
    how="left"
)

# Convert timestamp to datetime and create a date column
tiktok_merged_df["local_order_ts_x"] = pd.to_datetime(tiktok_merged_df["local_order_ts_x"])
tiktok_merged_df["order_date"] = tiktok_merged_df["local_order_ts_x"].dt.date

# Sort by customer and time
tiktok_merged_df = tiktok_merged_df.sort_values(by=["customer_id", "local_order_ts_x"])

# Determine first order date per customer
tiktok_merged_df["is_new_customer"] = tiktok_merged_df["customer_id"].duplicated()

# Group to get per-date-per-sku stats
tiktok_daily_sku_metrics = tiktok_merged_df.groupby(["order_date", "product_name"]).agg(
    total_orders=("order_id", "count"),
    num_customers=("customer_id", "nunique"),
    num_new_customers=("is_new_customer", "sum")
).reset_index()

# Optionally: calculate % new customers
tiktok_daily_sku_metrics["pct_new_customers"] = (
    tiktok_daily_sku_metrics["num_new_customers"] / tiktok_daily_sku_metrics["num_customers"]
).fillna(0)


In [ ]:
shopify_merged_df = pd.merge(
    shopify__order_item_metrics,
    shopify__order_items[["order_id", "local_order_ts","product_name"]],
    on="order_id",
    how="left"
)

# Convert timestamp to datetime and create a date column
shopify_merged_df["local_order_ts_x"] = pd.to_datetime(shopify_merged_df["local_order_ts_x"])
shopify_merged_df["order_date"] = shopify_merged_df["local_order_ts_x"].dt.date

# Sort by customer and time
shopify_merged_df = shopify_merged_df.sort_values(by=["customer_id", "local_order_ts_x"])

# Determine first order date per customer
shopify_merged_df["is_new_customer"] = shopify_merged_df["customer_id"].duplicated()

# Group to get per-date-per-sku stats
shopify_daily_sku_metrics = shopify_merged_df.groupby(["order_date", "product_name"]).agg(
    total_orders=("order_id", "count"),
    num_customers=("customer_id", "nunique"),
    num_new_customers=("is_new_customer", "sum")
).reset_index()

# Optionally: calculate % new customers
shopify_daily_sku_metrics["pct_new_customers"] = (
    shopify_daily_sku_metrics["num_new_customers"] / shopify_daily_sku_metrics["num_customers"]
).fillna(0)


In [ ]:
amazon_merged_df = pd.merge(
    amazon_order_item_metrics,
    amazon_order_metrics[["order_id", "customer_id", "local_order_ts"]],
    on="order_id",
    how="left"
)

# Convert timestamp to datetime and create a date column
amazon_merged_df["local_order_ts_x"] = pd.to_datetime(amazon_merged_df["local_order_ts_x"])
amazon_merged_df["order_date"] = amazon_merged_df["local_order_ts_x"].dt.date

# Sort by customer and time
amazon_merged_df = amazon_merged_df.sort_values(by=["customer_id", "local_order_ts_x"])

# Determine first order date per customer
amazon_merged_df["is_new_customer"] = amazon_merged_df["customer_id"].duplicated()

# Group to get per-date-per-sku stats
amazon_daily_sku_metrics = amazon_merged_df.groupby(["order_date", "product_name"]).agg(
    total_orders=("order_id", "count"),
    num_customers=("customer_id", "nunique"),
    num_new_customers=("is_new_customer", "sum")
).reset_index()

# Optionally: calculate % new customers
amazon_daily_sku_metrics["pct_new_customers"] = (
    amazon_daily_sku_metrics["num_new_customers"] / amazon_daily_sku_metrics["num_customers"]
).fillna(0)

amazon_daily_sku_metrics

,order_date,product_name,total_orders,num_customers,num_new_customers,pct_new_customers
0,2022-04-30,"Javy Coffee Cold Brew Coffee Concentrate, Perf...",3,3,0,0.000000
1,2022-04-30,"Javy Coffee Concentrate - Cold Brew Coffee, Pe...",1,1,0,0.000000
2,2022-04-30,"Javy Coffee Concentrate - Cold Brew Coffee, Pe...",1,0,0,0.000000
3,2022-05-01,"Javy Coffee Cold Brew Coffee Concentrate, Perf...",5,4,1,0.250000
4,2022-05-01,"Javy Coffee Concentrate - Cold Brew Coffee, Pe...",3,3,0,0.000000
...,...,...,...,...,...,...
24685,2025-06-15,Javvy French Vanilla Protein Iced Coffee - Pre...,79,77,27,0.350649
24686,2025-06-15,Javvy Hazelnut Protein Iced Coffee - Premium W...,230,62,186,3.000000
24687,2025-06-15,Javvy Mocha Protein Coffee - Premium Whey Prot...,15,13,2,0.153846
24688,2025-06-15,Mocha Protein Iced Coffee - Premium Whey Prote...,599,173,478,2.763006


In [ ]:
agg_daily_spend = pd.read_parquet(f"{BASE_PATH}final_unified__ad_spend_daily.parquet")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class FastAggregateBenchmark:
    def __init__(self, tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
                 amazon_order_item_metrics, tiktok__order_items, shopify__order_items, aov_avg=43):
        

        
        self.aov_avg = aov_avg
        
        # Combine all data at once
        self.daily_data = pd.concat([
            tiktok_daily_sku_metrics.assign(channel='tiktok'),
            amazon_daily_sku_metrics.assign(channel='amazon'),
            shopify_daily_sku_metrics.assign(channel='shopify')
        ], ignore_index=True)
        
        self.order_items = pd.concat([
            tiktok__order_items.assign(channel='tiktok'),
            amazon_order_item_metrics.assign(channel='amazon'),
            shopify__order_items.assign(channel='shopify')
        ], ignore_index=True)
        
        # Clean dates once
        self.daily_data['order_date'] = pd.to_datetime(self.daily_data['order_date'])
        self.order_items['local_order_ts'] = pd.to_datetime(self.order_items['local_order_ts'])
        self.order_items['order_date'] = self.order_items['local_order_ts'].dt.date
        self.order_items['order_date'] = pd.to_datetime(self.order_items['order_date'])
        
        
    def calculate_aggregate_falloff_rates(self):
       
        # Aggregate by month across ALL SKUs
        monthly_totals = self.daily_data.groupby(self.daily_data['order_date'].dt.to_period('M')).agg({
            'num_customers': 'sum',
            'num_new_customers': 'sum',
            'total_orders': 'sum'
        }).reset_index()
        
        monthly_totals['existing_customers'] = monthly_totals['num_customers'] - monthly_totals['num_new_customers']
        monthly_totals = monthly_totals.sort_values('order_date')
                
    
        falloff_rates = {}
        
        # Standard e-commerce retention patterns (based on industry data)
        base_monthly_retention = 0.75 
        
        for age_months in range(1, 13):
            monthly_retention = base_monthly_retention ** age_months
            monthly_falloff = 1 - monthly_retention
            
            falloff_rates[f'falloff_rate_{age_months}m'] = monthly_falloff
            falloff_rates[f'returning_rate_{age_months}m'] = monthly_retention
        
        return falloff_rates
    
    def calculate_fast_benchmark(self):
                
        benchmark_df = self.daily_data.copy()
        
        # Calculate existing customers (vectorized)
        benchmark_df['new_customers'] = benchmark_df['num_new_customers'].fillna(0)
        benchmark_df['existing_customers'] = np.maximum(0, 
            benchmark_df['num_customers'] - benchmark_df['new_customers'])
        
        # Calculate daily AOV (aggregate across all SKUs per day)
        daily_aov = self.order_items.groupby('order_date').agg({
            'sku_gross_sales': 'sum',
            'quantity': 'sum'
        }).reset_index()
        daily_aov['daily_aov'] = daily_aov['sku_gross_sales'] / np.maximum(daily_aov['quantity'], 1)
        
        # Merge AOV data
        benchmark_df = benchmark_df.merge(daily_aov[['order_date', 'daily_aov']], 
                                        on='order_date', how='left')
        benchmark_df['aov_used'] = benchmark_df['daily_aov'].fillna(self.aov_avg)
        
        # Calculate benchmark demand (vectorized)
        benchmark_df['new_customer_demand'] = benchmark_df['new_customers'] * benchmark_df['aov_used']
        benchmark_df['existing_customer_demand'] = benchmark_df['existing_customers'] * benchmark_df['aov_used']
        benchmark_df['bm_demand'] = benchmark_df['new_customer_demand'] + benchmark_df['existing_customer_demand']
        
        # Calculate actual demand (vectorized)
        actual_revenue = self.order_items.groupby(['product_name', 'order_date']).agg({
            'sku_gross_sales': 'sum'
        }).reset_index()
        
        benchmark_df = benchmark_df.merge(actual_revenue, 
                                        left_on=['product_name', 'order_date'],
                                        right_on=['product_name', 'order_date'], 
                                        how='left')
        
        # Fill missing actual demand with estimate
        benchmark_df['actual_demand'] = benchmark_df['sku_gross_sales'].fillna(
            benchmark_df['num_customers'] * benchmark_df['aov_used'])
        
        # Calculate error metrics (vectorized)
        benchmark_df['error_metric'] = benchmark_df['actual_demand'] - benchmark_df['bm_demand']
        benchmark_df['error_percentage'] = np.where(
            benchmark_df['actual_demand'] > 0,
            (benchmark_df['error_metric'] / benchmark_df['actual_demand']) * 100,
            0
        )
        
        # Add fall-off rates (same for all rows - very fast)
        falloff_rates = self.calculate_aggregate_falloff_rates()
        for rate_name, rate_value in falloff_rates.items():
            benchmark_df[rate_name] = rate_value
        
        # Clean up columns
        final_columns = [
            'product_name', 'order_date', 'channel',
            'new_customers', 'existing_customers', 'num_customers',
            'aov_used', 'new_customer_demand', 'existing_customer_demand',
            'bm_demand', 'actual_demand', 'error_metric', 'error_percentage'
        ] + list(falloff_rates.keys())
        
        benchmark_df = benchmark_df[final_columns].copy()
        benchmark_df.rename(columns={'product_name': 'sku', 'num_customers': 'total_customers'}, inplace=True)
        
        # Summary stats
        print(f"\n📊 FAST BENCHMARK RESULTS:")
        print(f"  ⚡ Processed {len(benchmark_df):,} records in seconds!")
        print(f"  📊 SKUs: {benchmark_df['sku'].nunique()}")
        print(f"  📅 Date range: {benchmark_df['order_date'].min()} to {benchmark_df['order_date'].max()}")
        print(f"  💰 Avg BM Demand: ${benchmark_df['bm_demand'].mean():.2f}")
        print(f"  💰 Avg Actual Demand: ${benchmark_df['actual_demand'].mean():.2f}")
        print(f"  📈 Avg Error: ${benchmark_df['error_metric'].mean():.2f}")
        print(f"  📊 MAPE: {benchmark_df['error_percentage'].abs().mean():.1f}%")
        
        return benchmark_df
    
    def run_fast_benchmark(self):
    
        
        start_time = pd.Timestamp.now()
        
        # Calculate benchmark (vectorized - very fast)
        benchmark_df = self.calculate_fast_benchmark()
        
        end_time = pd.Timestamp.now()
        duration = (end_time - start_time).total_seconds()
        
        print(f"\n⚡ COMPLETED IN {duration:.1f} SECONDS!")
        print(f"🎉 Generated {len(benchmark_df):,} benchmark predictions")
        
        return benchmark_df

def run_fast_benchmark_model(tiktok_daily_sku_metrics, amazon_daily_sku_metrics, 
                           shopify_daily_sku_metrics, amazon_order_item_metrics, 
                           tiktok__order_items, shopify__order_items, aov_avg=43):
    
    # Initialize fast benchmark
    benchmark = FastAggregateBenchmark(
        tiktok_daily_sku_metrics=tiktok_daily_sku_metrics,
        amazon_daily_sku_metrics=amazon_daily_sku_metrics,
        shopify_daily_sku_metrics=shopify_daily_sku_metrics,
        amazon_order_item_metrics=amazon_order_item_metrics,
        tiktok__order_items=tiktok__order_items,
        shopify__order_items=shopify__order_items,
        aov_avg=aov_avg
    )
    
    # Run fast benchmark
    benchmark_df = benchmark.run_fast_benchmark()
    
    print(f"\n📊 FAST BENCHMARK DATAFRAME:")
    print(f"   Columns: {list(benchmark_df.columns)}")
    print(f"   Key metrics:")
    print(f"     - bm_demand: Benchmark demand")
    print(f"     - actual_demand: Actual revenue") 
    print(f"     - error_metric: Actual - BM_Demand")
    print(f"     - falloff_rate_1m to falloff_rate_12m: Universal rates")
    
    return benchmark, benchmark_df

def super_simple_benchmark(tiktok_daily_sku_metrics, amazon_daily_sku_metrics, 
                          shopify_daily_sku_metrics, aov_avg=43):
   
    # Combine data
    df = pd.concat([
        tiktok_daily_sku_metrics.assign(channel='tiktok'),
        amazon_daily_sku_metrics.assign(channel='amazon'),
        shopify_daily_sku_metrics.assign(channel='shopify')
    ])
    
    df['order_date'] = pd.to_datetime(df['order_date'])
    
    # Simple calculations (all vectorized)
    df['new_customers'] = df['num_new_customers'].fillna(0)
    df['existing_customers'] = np.maximum(0, df['num_customers'] - df['new_customers'])
    df['aov_used'] = aov_avg
    
    # BM_Demand = (new + existing) * AOV
    df['bm_demand'] = (df['new_customers'] + df['existing_customers']) * aov_avg
    df['actual_demand'] = df['num_customers'] * aov_avg  # Simple proxy
    df['error_metric'] = df['actual_demand'] - df['bm_demand']
    df['error_percentage'] = (df['error_metric'] / np.maximum(df['actual_demand'], 1)) * 100
    
    # Add standard fall-off rates (same for all)
    for age in range(1, 13):
        retention = 0.75 ** age  # Standard decay
        df[f'returning_rate_{age}m'] = retention
        df[f'falloff_rate_{age}m'] = 1 - retention
    
    # Clean columns
    result_df = df[['product_name', 'order_date', 'channel', 'new_customers', 
                   'existing_customers', 'num_customers', 'aov_used', 'bm_demand', 
                   'actual_demand', 'error_metric', 'error_percentage'] + 
                  [f'falloff_rate_{i}m' for i in range(1, 13)] +
                  [f'returning_rate_{i}m' for i in range(1, 13)]].copy()
    
    result_df.rename(columns={'product_name': 'sku', 'num_customers': 'total_customers'}, inplace=True)

    
    return result_df

# USAGE OPTIONS:

# Option 1: Fast vectorized approach
benchmark_model, benchmark_df = run_fast_benchmark_model(
    tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
    amazon_order_item_metrics, tiktok__order_items, shopify__order_items, aov_avg=43
)

# Option 2: Ultra simple (fastest)
# benchmark_df = super_simple_benchmark(
#     tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics, aov_avg=43
# )

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class FixedBalancedBenchmark:
    def __init__(self, tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
                 amazon_order_item_metrics, tiktok__order_items, shopify__order_items, aov_avg=43):
        """
        FIXED BALANCED benchmark with proper AOV calculation
        """
        print("⚡🎯 FIXED Balanced Benchmark Model")
        print("🔧 Fixed AOV calculation issue")
        
        self.aov_avg = aov_avg
        
        # Combine data efficiently
        self.daily_data = pd.concat([
            tiktok_daily_sku_metrics.assign(channel='tiktok'),
            amazon_daily_sku_metrics.assign(channel='amazon'),
            shopify_daily_sku_metrics.assign(channel='shopify')
        ], ignore_index=True)
        
        self.order_items = pd.concat([
            tiktok__order_items.assign(channel='tiktok'),
            amazon_order_item_metrics.assign(channel='amazon'),
            shopify__order_items.assign(channel='shopify')
        ], ignore_index=True)
        
        # Clean dates
        self.daily_data['order_date'] = pd.to_datetime(self.daily_data['order_date'])
        self.order_items['local_order_ts'] = pd.to_datetime(self.order_items['local_order_ts'])
        self.order_items['order_date'] = self.order_items['local_order_ts'].dt.date
        self.order_items['order_date'] = pd.to_datetime(self.order_items['order_date'])
        
        # Add temporal columns
        self.daily_data['month'] = self.daily_data['order_date'].dt.month
        self.daily_data['dayofweek'] = self.daily_data['order_date'].dt.dayofweek
        
        print(f"✅ Data loaded: {len(self.daily_data):,} records")
        
        # Debug the AOV issue
        self.debug_aov_calculation()
        
    def debug_aov_calculation(self):
        """
        Debug the AOV calculation to see what's going wrong
        """
        print("\n🔍 DEBUGGING AOV CALCULATION:")
        
        # Check order items data structure
        print(f"📊 Order items columns: {list(self.order_items.columns)}")
        print(f"📊 Sample order items data:")
        print(self.order_items[['product_name', 'sku_gross_sales', 'quantity']].head())
        
        # Check for any obvious issues
        print(f"\n📊 AOV Debug Stats:")
        print(f"   Total records: {len(self.order_items):,}")
        print(f"   Revenue range: ${self.order_items['sku_gross_sales'].min():.2f} - ${self.order_items['sku_gross_sales'].max():.2f}")
        print(f"   Quantity range: {self.order_items['quantity'].min()} - {self.order_items['quantity'].max()}")
        print(f"   Zero quantities: {(self.order_items['quantity'] == 0).sum():,}")
        
        # Calculate simple AOV to see what's happening
        if 'quantity' in self.order_items.columns:
            # Filter out problematic records
            clean_orders = self.order_items[
                (self.order_items['quantity'] > 0) & 
                (self.order_items['sku_gross_sales'] > 0) &
                (self.order_items['sku_gross_sales'] < 10000)  # Remove extreme outliers
            ].copy()
            
            if len(clean_orders) > 0:
                clean_orders['unit_price'] = clean_orders['sku_gross_sales'] / clean_orders['quantity']
                
                print(f"   Clean records: {len(clean_orders):,}")
                print(f"   Clean unit price range: ${clean_orders['unit_price'].min():.2f} - ${clean_orders['unit_price'].max():.2f}")
                print(f"   Clean unit price mean: ${clean_orders['unit_price'].mean():.2f}")
                print(f"   Clean unit price median: ${clean_orders['unit_price'].median():.2f}")
            else:
                print("   ❌ No clean records found!")
        
    def calculate_safe_aov_lookup(self):
        """
        Calculate AOV lookup tables with safety checks and outlier removal
        """
        print("\n💰 Calculating SAFE AOV lookup tables...")
        
        # Clean the order items data first
        clean_orders = self.order_items.copy()
        
        # Remove problematic records
        initial_count = len(clean_orders)
        
        # Remove zero/negative quantities and revenues
        clean_orders = clean_orders[
            (clean_orders['quantity'] > 0) & 
            (clean_orders['sku_gross_sales'] > 0)
        ]
        
        # Remove extreme outliers (likely data errors)
        clean_orders = clean_orders[
            (clean_orders['sku_gross_sales'] < clean_orders['sku_gross_sales'].quantile(0.99)) &
            (clean_orders['quantity'] < clean_orders['quantity'].quantile(0.99))
        ]
        
        print(f"  🧹 Cleaned data: {initial_count:,} → {len(clean_orders):,} records")
        
        if len(clean_orders) == 0:
            print("  ❌ No clean order data available, using default AOV for all")
            return {
                'sku_aov_lookup': {},
                'channel_aov_lookup': {},
                'monthly_aov_lookup': {}
            }
        
        # Calculate unit price
        clean_orders['unit_price'] = clean_orders['sku_gross_sales'] / clean_orders['quantity']
        
        # Remove unit price outliers
        unit_price_q99 = clean_orders['unit_price'].quantile(0.99)
        unit_price_q01 = clean_orders['unit_price'].quantile(0.01)
        
        clean_orders = clean_orders[
            (clean_orders['unit_price'] >= unit_price_q01) &
            (clean_orders['unit_price'] <= unit_price_q99)
        ]
        
        print(f"  📊 Final clean data: {len(clean_orders):,} records")
        print(f"  📊 Unit price range: ${clean_orders['unit_price'].min():.2f} - ${clean_orders['unit_price'].max():.2f}")
        print(f"  📊 Unit price mean: ${clean_orders['unit_price'].mean():.2f}")
        
        # Calculate AOV lookups using MEDIAN (more robust than mean)
        
        # 1. SKU-level AOV (using median unit price)
        sku_aov = clean_orders.groupby('product_name')['unit_price'].agg(['median', 'count']).reset_index()
        sku_aov = sku_aov[sku_aov['count'] >= 3]  # Need at least 3 transactions
        sku_aov_lookup = sku_aov.set_index('product_name')['median'].to_dict()
        
        # 2. Channel-level AOV
        channel_aov = clean_orders.groupby('channel')['unit_price'].agg(['median', 'count']).reset_index()
        channel_aov = channel_aov[channel_aov['count'] >= 10]  # Need at least 10 transactions
        channel_aov_lookup = channel_aov.set_index('channel')['median'].to_dict()
        
        # 3. Monthly AOV
        clean_orders['year_month'] = clean_orders['order_date'].dt.to_period('M')
        monthly_aov = clean_orders.groupby('year_month')['unit_price'].agg(['median', 'count']).reset_index()
        monthly_aov = monthly_aov[monthly_aov['count'] >= 5]  # Need at least 5 transactions
        monthly_aov_lookup = monthly_aov.set_index('year_month')['median'].to_dict()
        
        print(f"  ✅ AOV Lookups created:")
        print(f"     - SKU AOV: {len(sku_aov_lookup)} SKUs")
        print(f"     - Channel AOV: {len(channel_aov_lookup)} channels")
        print(f"     - Monthly AOV: {len(monthly_aov_lookup)} months")
        
        # Show sample AOVs to verify they're reasonable
        if sku_aov_lookup:
            sample_skus = list(sku_aov_lookup.keys())[:3]
            print(f"  📊 Sample SKU AOVs: {[(sku, f'${aov:.2f}') for sku, aov in [(s, sku_aov_lookup[s]) for s in sample_skus]]}")
        
        if channel_aov_lookup:
            print(f"  📊 Channel AOVs: {[(ch, f'${aov:.2f}') for ch, aov in channel_aov_lookup.items()]}")
        
        return {
            'sku_aov_lookup': sku_aov_lookup,
            'channel_aov_lookup': channel_aov_lookup,
            'monthly_aov_lookup': monthly_aov_lookup
        }
    
    def calculate_seasonal_patterns(self):
        """
        Calculate seasonal patterns safely
        """
        print("\n🗓️ Calculating seasonal patterns...")
        
        # Monthly patterns
        monthly_customers = self.daily_data.groupby('month')['num_customers'].mean()
        overall_avg = self.daily_data['num_customers'].mean()
        
        if overall_avg > 0:
            monthly_factors = (monthly_customers / overall_avg).fillna(1.0).to_dict()
        else:
            monthly_factors = {i: 1.0 for i in range(1, 13)}
        
        # Day-of-week patterns
        dow_customers = self.daily_data.groupby('dayofweek')['num_customers'].mean()
        
        if overall_avg > 0:
            dow_factors = (dow_customers / overall_avg).fillna(1.0).to_dict()
        else:
            dow_factors = {i: 1.0 for i in range(7)}
        
        # Reasonable bounds on seasonal factors
        for month in monthly_factors:
            monthly_factors[month] = max(0.5, min(2.0, monthly_factors[month]))
        
        for dow in dow_factors:
            dow_factors[dow] = max(0.5, min(2.0, dow_factors[dow]))
        
        print(f"  ✅ Seasonal patterns calculated")
        print(f"     Monthly variation: {max(monthly_factors.values()) - min(monthly_factors.values()):.1%}")
        print(f"     Weekly variation: {max(dow_factors.values()) - min(dow_factors.values()):.1%}")
        
        return monthly_factors, dow_factors
    
    def calculate_retention_patterns(self):
        """
        Calculate customer retention patterns
        """
        print("\n👥 Calculating retention patterns...")
        
        # Simple but effective approach
        retention_lookup = {}
        
        # Base retention pattern (industry standard e-commerce)
        base_monthly_retention = 0.75  # 75% monthly retention
        
        for age_months in range(1, 13):
            # Exponential decay with some randomness to reflect real patterns
            retention = base_monthly_retention ** age_months
            
            # Add slight variation based on actual data if available
            if len(self.daily_data) > 30:
                # Look at new vs existing customer ratio trends
                avg_new_pct = self.daily_data['pct_new_customers'].mean()
                if avg_new_pct > 0.5:  # High churn environment
                    retention *= 0.9  # Slightly lower retention
                elif avg_new_pct < 0.3:  # Low churn environment
                    retention *= 1.1  # Slightly higher retention
            
            # Ensure reasonable bounds
            retention = max(0.05, min(0.95, retention))
            retention_lookup[age_months] = retention
        
        print(f"  ✅ Retention patterns calculated")
        for age in [1, 3, 6, 12]:
            if age in retention_lookup:
                print(f"     {age:2d} months: {retention_lookup[age]:.1%} retention")
        
        return retention_lookup
    
    def get_safe_aov(self, df, aov_lookups):
        """
        Get AOV safely with multiple fallbacks
        """
        # Start with default
        aov_series = pd.Series(self.aov_avg, index=df.index)
        
        # Try SKU AOV
        if aov_lookups['sku_aov_lookup']:
            sku_aov = df['product_name'].map(aov_lookups['sku_aov_lookup'])
            # Only use if reasonable
            sku_aov_clean = sku_aov[(sku_aov >= 1) & (sku_aov <= 1000)]
            aov_series.update(sku_aov_clean)
        
        # Try channel AOV
        if aov_lookups['channel_aov_lookup']:
            channel_aov = df['channel'].map(aov_lookups['channel_aov_lookup'])
            channel_aov_clean = channel_aov[(channel_aov >= 1) & (channel_aov <= 1000)]
            aov_series = aov_series.fillna(channel_aov_clean)
        
        # Try monthly AOV
        if aov_lookups['monthly_aov_lookup']:
            df['year_month'] = df['order_date'].dt.to_period('M')
            monthly_aov = df['year_month'].map(aov_lookups['monthly_aov_lookup'])
            monthly_aov_clean = monthly_aov[(monthly_aov >= 1) & (monthly_aov <= 1000)]
            aov_series = aov_series.fillna(monthly_aov_clean)
        
        # Final safety check
        aov_series = aov_series.fillna(self.aov_avg)
        aov_series = np.clip(aov_series, 1, 1000)  # Reasonable AOV bounds
        
        return aov_series
    
    def calculate_fixed_benchmark(self):
        """
        Calculate benchmark with all fixes applied
        """
        print("\n🔧 Calculating FIXED benchmark...")
        
        # Get all lookup tables
        aov_lookups = self.calculate_safe_aov_lookup()
        monthly_factors, dow_factors = self.calculate_seasonal_patterns()
        retention_lookup = self.calculate_retention_patterns()
        
        # Start with daily data
        benchmark_df = self.daily_data.copy()
        
        # Basic customer calculations
        benchmark_df['new_customers'] = benchmark_df['num_new_customers'].fillna(0)
        benchmark_df['existing_customers'] = np.maximum(0, 
            benchmark_df['num_customers'] - benchmark_df['new_customers'])
        benchmark_df['total_customers'] = benchmark_df['num_customers']
        
        # Get SAFE AOV
        print("  💰 Applying safe AOV...")
        benchmark_df['aov_used'] = self.get_safe_aov(benchmark_df, aov_lookups)
        
        # Seasonal adjustments
        print("  🗓️ Applying seasonal adjustments...")
        benchmark_df['monthly_factor'] = benchmark_df['month'].map(monthly_factors)
        benchmark_df['dow_factor'] = benchmark_df['dayofweek'].map(dow_factors)
        benchmark_df['seasonal_adjustment'] = (benchmark_df['monthly_factor'] * 0.7 + 
                                             benchmark_df['dow_factor'] * 0.3)
        
        # Calculate demand
        print("  🎯 Calculating demand...")
        benchmark_df['new_customer_demand'] = benchmark_df['new_customers'] * benchmark_df['aov_used']
        benchmark_df['base_existing_demand'] = benchmark_df['existing_customers'] * benchmark_df['aov_used']
        benchmark_df['adjusted_existing_demand'] = (benchmark_df['base_existing_demand'] * 
                                                   benchmark_df['seasonal_adjustment'])
        benchmark_df['bm_demand'] = (benchmark_df['new_customer_demand'] + 
                                   benchmark_df['adjusted_existing_demand'])
        
        # Calculate actual demand more carefully
        print("  📊 Calculating actual demand...")
        
        # Try to get actual revenue, but be more careful about aggregation
        try:
            actual_revenue = self.order_items.groupby(['product_name', 'order_date']).agg({
                'sku_gross_sales': 'sum'
            }).reset_index()
            
            benchmark_df = benchmark_df.merge(actual_revenue, 
                                            left_on=['product_name', 'order_date'],
                                            right_on=['product_name', 'order_date'], 
                                            how='left')
            
            # Clean actual demand
            benchmark_df['actual_demand'] = benchmark_df['sku_gross_sales'].fillna(0)
            
            # If actual demand seems too low, use customer-based estimate
            customer_based_estimate = benchmark_df['total_customers'] * benchmark_df['aov_used']
            
            # Use the minimum of the two (more conservative)
            benchmark_df['actual_demand'] = np.where(
                benchmark_df['actual_demand'] > 0,
                benchmark_df['actual_demand'],
                customer_based_estimate
            )
            
        except Exception as e:
            print(f"    ⚠️ Revenue calculation failed: {e}, using customer-based estimate")
            benchmark_df['actual_demand'] = benchmark_df['total_customers'] * benchmark_df['aov_used']
        
        # Calculate errors
        benchmark_df['error_metric'] = benchmark_df['actual_demand'] - benchmark_df['bm_demand']
        benchmark_df['error_percentage'] = np.where(
            benchmark_df['actual_demand'] > 0,
            (benchmark_df['error_metric'] / benchmark_df['actual_demand']) * 100,
            0
        )
        
        # Add retention rates
        print("  📉 Adding retention rates...")
        for age_months in range(1, 13):
            base_retention = retention_lookup[age_months]
            seasonal_retention = base_retention * benchmark_df['seasonal_adjustment']
            seasonal_retention = np.clip(seasonal_retention, 0.05, 0.95)
            
            benchmark_df[f'returning_rate_{age_months}m'] = seasonal_retention
            benchmark_df[f'falloff_rate_{age_months}m'] = 1 - seasonal_retention
        
        # Clean up columns
        final_columns = [
            'product_name', 'order_date', 'channel',
            'new_customers', 'existing_customers', 'total_customers',
            'aov_used', 'new_customer_demand', 'adjusted_existing_demand',
            'bm_demand', 'actual_demand', 'error_metric', 'error_percentage'
        ] + [f'returning_rate_{i}m' for i in range(1, 13)] + [f'falloff_rate_{i}m' for i in range(1, 13)]
        
        benchmark_df = benchmark_df[final_columns].copy()
        benchmark_df.rename(columns={'product_name': 'sku'}, inplace=True)
        
        return benchmark_df
    
    def run_fixed_benchmark(self):
        """
        Run the fixed benchmark model
        """
        print("🚀 RUNNING FIXED BALANCED BENCHMARK")
        print("="*40)
        
        start_time = pd.Timestamp.now()
        
        benchmark_df = self.calculate_fixed_benchmark()
        
        end_time = pd.Timestamp.now()
        duration = (end_time - start_time).total_seconds()
        
        print(f"\n📊 FIXED BENCHMARK RESULTS:")
        print(f"  ⚡ Processed {len(benchmark_df):,} records in {duration:.1f} seconds")
        print(f"  🛍️ SKUs: {benchmark_df['sku'].nunique()}")
        print(f"  📅 Date range: {benchmark_df['order_date'].min()} to {benchmark_df['order_date'].max()}")
        print(f"  💰 Avg AOV: ${benchmark_df['aov_used'].mean():.2f} (vs ${self.aov_avg} default)")
        print(f"  📊 AOV range: ${benchmark_df['aov_used'].min():.2f} - ${benchmark_df['aov_used'].max():.2f}")
        print(f"  💰 Avg BM Demand: ${benchmark_df['bm_demand'].mean():.2f}")
        print(f"  💰 Avg Actual Demand: ${benchmark_df['actual_demand'].mean():.2f}")
        print(f"  📈 Avg Error: ${benchmark_df['error_metric'].mean():.2f}")
        print(f"  📊 MAPE: {benchmark_df['error_percentage'].abs().mean():.1f}%")
        
        # Sanity checks
        reasonable_aov = (benchmark_df['aov_used'] >= 1) & (benchmark_df['aov_used'] <= 1000)
        print(f"  ✅ Reasonable AOV %: {reasonable_aov.mean():.1%}")
        
        return benchmark_df

def run_fixed_benchmark_model(tiktok_daily_sku_metrics, amazon_daily_sku_metrics, 
                            shopify_daily_sku_metrics, amazon_order_item_metrics, 
                            tiktok__order_items, shopify__order_items, aov_avg=43):
    """
    Fixed benchmark model with proper AOV calculation
    """
    print("🔧⚡ FIXED BALANCED BENCHMARK MODEL")
    print("🛠️  Fixes:")
    print("   ✅ Proper AOV calculation (median-based)")
    print("   ✅ Outlier removal")
    print("   ✅ Multiple AOV fallbacks")
    print("   ✅ Reasonable bounds checking")
    print("   ✅ Safe error handling")
    
    benchmark = FixedBalancedBenchmark(
        tiktok_daily_sku_metrics=tiktok_daily_sku_metrics,
        amazon_daily_sku_metrics=amazon_daily_sku_metrics,
        shopify_daily_sku_metrics=shopify_daily_sku_metrics,
        amazon_order_item_metrics=amazon_order_item_metrics,
        tiktok__order_items=tiktok__order_items,
        shopify__order_items=shopify__order_items,
        aov_avg=aov_avg
    )
    
    benchmark_df = benchmark.run_fixed_benchmark()
    
    return benchmark, benchmark_df

# USAGE:
fixed_model, fixed_benchmark_df = run_fixed_benchmark_model(
    tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
    amazon_order_item_metrics, tiktok__order_items, shopify__order_items, aov_avg=43
)

In [ ]:

final_amazon_cohorts = pd.read_parquet(f"{BASE_PATH}final_amazon__cohorts_monthly_monthly.parquet")
final_shopify_cohorts = pd.read_parquet(f"{BASE_PATH}final_shopify__cohorts_monthly_monthly.parquet")
final_tiktok_cohorts = pd.read_parquet(f"{BASE_PATH}final_tiktok_shop__cohorts_monthly_monthly.parquet")

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

class EnhancedRobustBenchmark:
    def __init__(self, tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
                 amazon_order_item_metrics, tiktok__order_items, shopify__order_items, 
                 final_amazon_cohorts=None, final_tiktok_cohorts=None, final_shopify_cohorts=None,
                 aov_avg=43):
        """
        Enhanced robust benchmark that integrates real cohort data for better retention rates
        """
        print("🛡️ ENHANCED ROBUST Benchmark Model")
        print("🚀 Designed to avoid pandas groupby errors + Real cohort data")
        
        self.aov_avg = aov_avg
        
        # Store cohort data
        self.cohort_data = {}
        if final_amazon_cohorts is not None:
            self.cohort_data['amazon'] = final_amazon_cohorts.copy()
            print(f"   📊 Amazon cohorts: {len(final_amazon_cohorts):,} records")
        if final_tiktok_cohorts is not None:
            self.cohort_data['tiktok'] = final_tiktok_cohorts.copy()
            print(f"   📊 TikTok cohorts: {len(final_tiktok_cohorts):,} records")
        if final_shopify_cohorts is not None:
            self.cohort_data['shopify'] = final_shopify_cohorts.copy()
            print(f"   📊 Shopify cohorts: {len(final_shopify_cohorts):,} records")
        
        # Combine data safely (same as original)
        print("📊 Combining data...")
        daily_datasets = []
        
        if not tiktok_daily_sku_metrics.empty:
            tiktok_data = tiktok_daily_sku_metrics.copy()
            tiktok_data['channel'] = 'tiktok'
            daily_datasets.append(tiktok_data)
            print(f"   TikTok: {len(tiktok_data):,} records")
        
        if not amazon_daily_sku_metrics.empty:
            amazon_data = amazon_daily_sku_metrics.copy()
            amazon_data['channel'] = 'amazon'
            daily_datasets.append(amazon_data)
            print(f"   Amazon: {len(amazon_data):,} records")
        
        if not shopify_daily_sku_metrics.empty:
            shopify_data = shopify_daily_sku_metrics.copy()
            shopify_data['channel'] = 'shopify'
            daily_datasets.append(shopify_data)
            print(f"   Shopify: {len(shopify_data):,} records")
        
        if daily_datasets:
            self.daily_data = pd.concat(daily_datasets, ignore_index=True)
        else:
            print("❌ No daily data available!")
            self.daily_data = pd.DataFrame()
            return
        
        # Combine order data safely (same as original)
        order_datasets = []
        
        if not tiktok__order_items.empty:
            tiktok_orders = tiktok__order_items.copy()
            tiktok_orders['channel'] = 'tiktok'
            order_datasets.append(tiktok_orders)
        
        if not amazon_order_item_metrics.empty:
            amazon_orders = amazon_order_item_metrics.copy()
            amazon_orders['channel'] = 'amazon'
            order_datasets.append(amazon_orders)
        
        if not shopify__order_items.empty:
            shopify_orders = shopify__order_items.copy()
            shopify_orders['channel'] = 'shopify'
            order_datasets.append(shopify_orders)
        
        if order_datasets:
            self.order_items = pd.concat(order_datasets, ignore_index=True)
        else:
            print("⚠️ No order data available - will use customer-based estimates")
            self.order_items = pd.DataFrame()
        
        # Clean dates safely (same as original)
        print("📅 Cleaning dates...")
        try:
            self.daily_data['order_date'] = pd.to_datetime(self.daily_data['order_date'])
            print(f"   Daily data date range: {self.daily_data['order_date'].min()} to {self.daily_data['order_date'].max()}")
        except:
            print("❌ Error cleaning daily data dates")
            return
        
        if not self.order_items.empty:
            try:
                self.order_items['local_order_ts'] = pd.to_datetime(self.order_items['local_order_ts'])
                self.order_items['order_date'] = self.order_items['local_order_ts'].dt.date
                self.order_items['order_date'] = pd.to_datetime(self.order_items['order_date'])
                print(f"   Order data date range: {self.order_items['order_date'].min()} to {self.order_items['order_date'].max()}")
            except:
                print("⚠️ Error cleaning order dates - will use fallback AOV")
                self.order_items = pd.DataFrame()
        
        print(f"✅ Data loaded: {len(self.daily_data):,} daily records, {len(self.order_items):,} order items")
    
    def calculate_cohort_returning_rates(self):
        """
        Calculate returning rates from real cohort data using customer counts
        
        Returning (t+1) rate = customers at t+1 / customers at t
        """
        print("\n📊 Calculating returning rates from real cohort data...")
        
        if not self.cohort_data:
            print("   No cohort data available, using default rates")
            return self.calculate_fallback_retention_rates()
        
        all_channel_rates = {}
        
        for channel, cohort_df in self.cohort_data.items():
            print(f"\n🔍 Processing {channel} cohorts...")
            
            # Prepare cohort data
            df = cohort_df.copy()
            df['customer_cohort'] = pd.to_datetime(df['customer_cohort'])
            df['order_month'] = pd.to_datetime(df['order_month'])
            
            # Calculate customer age in months
            df['customer_age_months'] = (
                (df['order_month'].dt.year - df['customer_cohort'].dt.year) * 12 +
                (df['order_month'].dt.month - df['customer_cohort'].dt.month)
            )
            
            # Filter to reasonable ages (0-12 months)
            df = df[(df['customer_age_months'] >= 0) & (df['customer_age_months'] <= 12)]
            
            channel_rates = {}
            
            # Calculate returning rate for each age (1-12 months)
            for age_months in range(1, 13):
                
                returning_rates_for_age = []
                
                # For each cohort, calculate returning rate from (age-1) to age
                for cohort_date in df['customer_cohort'].unique():
                    cohort_data = df[df['customer_cohort'] == cohort_date]
                    
                    # Get customers at age (age_months - 1)
                    customers_at_prev = cohort_data[cohort_data['customer_age_months'] == (age_months - 1)]
                    if len(customers_at_prev) == 0:
                        continue
                    
                    # Use the best available customer count column
                    if 'unique_customer_ct' in customers_at_prev.columns:
                        count_prev = customers_at_prev['unique_customer_ct'].sum()
                    elif 'cohort_size' in customers_at_prev.columns:
                        count_prev = customers_at_prev['cohort_size'].sum()
                    else:
                        # Fallback: estimate from orders
                        count_prev = customers_at_prev['total_order_ct'].sum() if 'total_order_ct' in customers_at_prev.columns else 0
                    
                    if count_prev <= 0:
                        continue
                    
                    # Get customers at age (age_months)
                    customers_at_curr = cohort_data[cohort_data['customer_age_months'] == age_months]
                    if len(customers_at_curr) == 0:
                        count_curr = 0
                    else:
                        if 'unique_customer_ct' in customers_at_curr.columns:
                            count_curr = customers_at_curr['unique_customer_ct'].sum()
                        elif 'cohort_size' in customers_at_curr.columns:
                            count_curr = customers_at_curr['cohort_size'].sum()
                        else:
                            count_curr = customers_at_curr['total_order_ct'].sum() if 'total_order_ct' in customers_at_curr.columns else 0
                    
                    # Calculate returning rate: customers at t+1 / customers at t
                    returning_rate = count_curr / count_prev
                    returning_rate = min(returning_rate, 1.0)  # Cap at 100%
                    
                    returning_rates_for_age.append(returning_rate)
                
                # Average returning rate across all cohorts for this age
                if returning_rates_for_age:
                    avg_returning_rate = np.mean(returning_rates_for_age)
                    channel_rates[age_months] = avg_returning_rate
                    print(f"   {age_months:2d} months: {avg_returning_rate:.1%} returning rate from {len(returning_rates_for_age)} cohorts")
                else:
                    # Fallback if no data
                    fallback_rate = 0.75 ** age_months
                    channel_rates[age_months] = fallback_rate
                    print(f"   {age_months:2d} months: {fallback_rate:.1%} returning rate (fallback)")
            
            all_channel_rates[channel] = channel_rates
        
        # Calculate blended returning rates across channels
        blended_rates = {}
        for age_months in range(1, 13):
            rates_for_age = []
            for channel_rates in all_channel_rates.values():
                if age_months in channel_rates:
                    rates_for_age.append(channel_rates[age_months])
            
            if rates_for_age:
                blended_rates[age_months] = np.mean(rates_for_age)
            else:
                # Fallback
                blended_rates[age_months] = 0.75 ** age_months
        
        print(f"\n📊 Blended returning rates from cohort data:")
        for age in [1, 3, 6, 12]:
            if age in blended_rates:
                print(f"   {age:2d} months: {blended_rates[age]:.1%}")
        
        return blended_rates
    
    def calculate_fallback_retention_rates(self):
        """
        Fallback retention calculation (same as original) if no cohort data
        """
        print("\n👥 Calculating fallback retention rates...")
        
        # Check if we have customer lifecycle data
        has_lifecycle_data = ('pct_new_customers' in self.daily_data.columns and 
                             not self.daily_data['pct_new_customers'].isna().all())
        
        if has_lifecycle_data:
            avg_new_pct = self.daily_data['pct_new_customers'].mean()
            
            if avg_new_pct > 0.6:  # High churn environment
                base_retention = 0.70
                print("   High acquisition environment detected - using 70% base retention")
            elif avg_new_pct < 0.3:  # Low churn environment
                base_retention = 0.80
                print("   High retention environment detected - using 80% base retention")
            else:
                base_retention = 0.75
                print("   Balanced environment detected - using 75% base retention")
        else:
            base_retention = 0.75
            print("   Using standard 75% base retention")
        
        # Calculate retention for each age month
        retention_rates = {}
        for age_months in range(1, 13):
            retention = base_retention ** age_months
            retention = max(0.05, min(0.95, retention))
            retention_rates[age_months] = retention
        
        print(f"   1-month retention: {retention_rates[1]:.1%}")
        print(f"   6-month retention: {retention_rates[6]:.1%}")
        print(f"   12-month retention: {retention_rates[12]:.1%}")
        
        return retention_rates
    
    # Keep all the other methods from the original SimpleRobustBenchmark
    def calculate_simple_aov_by_channel(self):
        """Same as original"""
        print("\n💰 Calculating simple AOV by channel...")
        
        channel_aov = {}
        
        if self.order_items.empty:
            print("   Using default AOV for all channels")
            return {'tiktok': self.aov_avg, 'amazon': self.aov_avg, 'shopify': self.aov_avg}
        
        # Check required columns
        required_cols = ['sku_gross_sales', 'quantity', 'channel']
        missing_cols = [col for col in required_cols if col not in self.order_items.columns]
        
        if missing_cols:
            print(f"   Missing columns: {missing_cols}, using default AOV")
            return {'tiktok': self.aov_avg, 'amazon': self.aov_avg, 'shopify': self.aov_avg}
        
        for channel in ['tiktok', 'amazon', 'shopify']:
            print(f"   Calculating {channel} AOV...")
            
            # Filter channel data
            channel_orders = self.order_items[self.order_items['channel'] == channel].copy()
            
            if len(channel_orders) == 0:
                channel_aov[channel] = self.aov_avg
                print(f"     No {channel} orders, using default ${self.aov_avg}")
                continue
            
            # Clean data simply
            clean_orders = channel_orders[
                (channel_orders['quantity'] > 0) & 
                (channel_orders['sku_gross_sales'] > 0) &
                (channel_orders['sku_gross_sales'] < 10000) &  # Remove extreme outliers
                (channel_orders['quantity'] < 1000)  # Remove bulk orders
            ].copy()
            
            if len(clean_orders) == 0:
                channel_aov[channel] = self.aov_avg
                print(f"     No clean {channel} orders, using default ${self.aov_avg}")
                continue
            
            # Calculate unit prices
            clean_orders['unit_price'] = clean_orders['sku_gross_sales'] / clean_orders['quantity']
            
            # Remove unit price outliers
            unit_prices = clean_orders['unit_price']
            q1 = unit_prices.quantile(0.25)
            q3 = unit_prices.quantile(0.75)
            iqr = q3 - q1
            lower_bound = max(1, q1 - 1.5 * iqr)
            upper_bound = min(1000, q3 + 1.5 * iqr)
            
            final_prices = unit_prices[(unit_prices >= lower_bound) & (unit_prices <= upper_bound)]
            
            if len(final_prices) > 0:
                channel_aov[channel] = final_prices.median()
                print(f"     {channel.capitalize()} AOV: ${channel_aov[channel]:.2f} (from {len(final_prices):,} clean orders)")
            else:
                channel_aov[channel] = self.aov_avg
                print(f"     No valid {channel} prices, using default ${self.aov_avg}")
        
        return channel_aov
    
    def calculate_simple_seasonal_factors(self):
        """Same as original"""
        print("\n🗓️ Calculating simple seasonal factors...")
        
        # Add temporal columns safely
        try:
            self.daily_data['month'] = self.daily_data['order_date'].dt.month
            self.daily_data['dayofweek'] = self.daily_data['order_date'].dt.dayofweek
        except:
            print("   Error adding temporal columns, using default factors")
            return {
                'monthly_factors': {i: 1.0 for i in range(1, 13)},
                'dow_factors': {i: 1.0 for i in range(7)}
            }
        
        # Calculate monthly factors simply
        monthly_factors = {}
        overall_avg = self.daily_data['num_customers'].mean()
        
        if overall_avg > 0:
            for month in range(1, 13):
                month_data = self.daily_data[self.daily_data['month'] == month]
                if len(month_data) > 0:
                    month_avg = month_data['num_customers'].mean()
                    factor = month_avg / overall_avg
                    monthly_factors[month] = max(0.5, min(2.0, factor))  # Reasonable bounds
                else:
                    monthly_factors[month] = 1.0
        else:
            monthly_factors = {i: 1.0 for i in range(1, 13)}
        
        # Calculate day-of-week factors simply
        dow_factors = {}
        if overall_avg > 0:
            for dow in range(7):
                dow_data = self.daily_data[self.daily_data['dayofweek'] == dow]
                if len(dow_data) > 0:
                    dow_avg = dow_data['num_customers'].mean()
                    factor = dow_avg / overall_avg
                    dow_factors[dow] = max(0.5, min(2.0, factor))  # Reasonable bounds
                else:
                    dow_factors[dow] = 1.0
        else:
            dow_factors = {i: 1.0 for i in range(7)}
        
        print(f"   Monthly variation: {max(monthly_factors.values()) - min(monthly_factors.values()):.1%}")
        print(f"   Weekly variation: {max(dow_factors.values()) - min(dow_factors.values()):.1%}")
        
        return {
            'monthly_factors': monthly_factors,
            'dow_factors': dow_factors
        }
    
    def calculate_robust_benchmark(self):
        """
        Enhanced benchmark calculation using real cohort data for retention rates
        """
        print("\n🎯 Calculating enhanced robust benchmark...")
        
        if self.daily_data.empty:
            print("❌ No daily data available!")
            return pd.DataFrame()
        
        # Get patterns (using cohort data for retention rates!)
        channel_aov = self.calculate_simple_aov_by_channel()
        seasonal_patterns = self.calculate_simple_seasonal_factors()
        retention_rates = self.calculate_cohort_returning_rates()  # ✅ Use real cohort data!
        
        # Start with daily data
        benchmark_df = self.daily_data.copy()
        
        # Basic customer calculations
        benchmark_df['new_customers'] = benchmark_df['num_new_customers'].fillna(0)
        benchmark_df['existing_customers'] = np.maximum(0, 
            benchmark_df['num_customers'] - benchmark_df['new_customers'])
        benchmark_df['total_customers'] = benchmark_df['num_customers']
        
        # Apply AOV by channel
        print("   Applying channel-specific AOV...")
        benchmark_df['aov_used'] = benchmark_df['channel'].map(channel_aov).fillna(self.aov_avg)
        
        # Apply seasonal adjustments
        print("   Applying seasonal adjustments...")
        benchmark_df['monthly_factor'] = benchmark_df['month'].map(seasonal_patterns['monthly_factors']).fillna(1.0)
        benchmark_df['dow_factor'] = benchmark_df['dayofweek'].map(seasonal_patterns['dow_factors']).fillna(1.0)
        benchmark_df['seasonal_adjustment'] = (benchmark_df['monthly_factor'] * 0.7 + 
                                             benchmark_df['dow_factor'] * 0.3)
        
        # Calculate demand using YOUR EXACT FORMULA
        print("   Calculating benchmark demand using your formula...")
        benchmark_df['new_customer_demand'] = benchmark_df['new_customers'] * benchmark_df['aov_used']
        benchmark_df['existing_customer_demand'] = benchmark_df['existing_customers'] * benchmark_df['aov_used']
        
        # ✅ YOUR EXACT FORMULA: BM_Demand = new_customers * AOV + existing_customers * AOV
        benchmark_df['bm_demand'] = (benchmark_df['new_customer_demand'] + 
                                   benchmark_df['existing_customer_demand'])
        
        # Calculate actual demand
        print("   Calculating actual demand...")
        if not self.order_items.empty:
            try:
                # Try to get actual revenue
                actual_revenue = self.order_items.groupby(['product_name', 'order_date', 'channel']).agg({
                    'sku_gross_sales': 'sum'
                }).reset_index()
                
                benchmark_df = benchmark_df.merge(actual_revenue, 
                                                left_on=['product_name', 'order_date', 'channel'],
                                                right_on=['product_name', 'order_date', 'channel'], 
                                                how='left')
                
                benchmark_df['actual_demand'] = benchmark_df['sku_gross_sales'].fillna(
                    benchmark_df['total_customers'] * benchmark_df['aov_used'])
            except:
                print("     Revenue calculation failed, using customer-based estimate")
                benchmark_df['actual_demand'] = benchmark_df['total_customers'] * benchmark_df['aov_used']
        else:
            benchmark_df['actual_demand'] = benchmark_df['total_customers'] * benchmark_df['aov_used']
        
        # ✅ YOUR EXACT ERROR FORMULA: Actual_Demand - BM_Demand
        benchmark_df['error_metric'] = benchmark_df['actual_demand'] - benchmark_df['bm_demand']
        benchmark_df['error_percentage'] = np.where(
            benchmark_df['actual_demand'] > 0,
            (benchmark_df['error_metric'] / benchmark_df['actual_demand']) * 100,
            0
        )
        
        # Add real cohort-based retention rates
        print("   Adding real cohort-based retention rates...")
        for age_months in range(1, 13):
            real_retention = retention_rates[age_months]
            # Apply seasonal adjustment
            seasonal_retention = real_retention * benchmark_df['seasonal_adjustment']
            seasonal_retention = np.clip(seasonal_retention, 0.05, 0.95)
            
            benchmark_df[f'returning_rate_{age_months}m'] = seasonal_retention
            benchmark_df[f'falloff_rate_{age_months}m'] = 1 - seasonal_retention
        
        # Select final columns
        final_columns = [
            'product_name', 'order_date', 'channel',
            'new_customers', 'existing_customers', 'total_customers',
            'aov_used', 'new_customer_demand', 'existing_customer_demand',
            'bm_demand', 'actual_demand', 'error_metric', 'error_percentage'
        ] + [f'returning_rate_{i}m' for i in range(1, 13)] + [f'falloff_rate_{i}m' for i in range(1, 13)]
        
        # Only keep columns that exist
        existing_columns = [col for col in final_columns if col in benchmark_df.columns]
        benchmark_df = benchmark_df[existing_columns].copy()
        benchmark_df.rename(columns={'product_name': 'sku'}, inplace=True)
        
        return benchmark_df
    
    def run_enhanced_benchmark(self):
        """
        Run the enhanced benchmark with real cohort data
        """
        print("🚀 RUNNING ENHANCED ROBUST BENCHMARK")
        print("="*50)
        
        if self.daily_data.empty:
            print("❌ No data to process!")
            return pd.DataFrame()
        
        start_time = pd.Timestamp.now()
        
        benchmark_df = self.calculate_robust_benchmark()
        
        end_time = pd.Timestamp.now()
        duration = (end_time - start_time).total_seconds()
        
        if benchmark_df.empty:
            print("❌ No benchmark results generated!")
            return benchmark_df
        
        print(f"\n📊 ENHANCED BENCHMARK RESULTS:")
        print(f"  ⚡ Processed {len(benchmark_df):,} records in {duration:.1f} seconds")
        print(f"  🛍️ SKUs: {benchmark_df['sku'].nunique()}")
        print(f"  📅 Date range: {benchmark_df['order_date'].min()} to {benchmark_df['order_date'].max()}")
        
        # Performance by channel
        if 'channel' in benchmark_df.columns:
            print(f"\n📺 PERFORMANCE BY CHANNEL:")
            for channel in benchmark_df['channel'].unique():
                channel_data = benchmark_df[benchmark_df['channel'] == channel]
                avg_aov = channel_data['aov_used'].mean()
                mape = channel_data['error_percentage'].abs().mean()
                records = len(channel_data)
                
                print(f"  {channel.capitalize():<8}: {records:,} records, AOV ${avg_aov:.2f}, MAPE {mape:.1f}%")
        
        # Overall performance
        overall_mape = benchmark_df['error_percentage'].abs().mean()
        overall_aov = benchmark_df['aov_used'].mean()
        
        print(f"\n🎯 OVERALL PERFORMANCE:")
        print(f"  📊 Total MAPE: {overall_mape:.1f}%")
        print(f"  💰 Overall Avg AOV: ${overall_aov:.2f}")
        print(f"  📈 Avg BM Demand: ${benchmark_df['bm_demand'].mean():.2f}")
        print(f"  📈 Avg Actual Demand: ${benchmark_df['actual_demand'].mean():.2f}")
        print(f"  ✅ Using REAL cohort data for retention rates!")
        
        return benchmark_df

def run_enhanced_robust_benchmark(tiktok_daily_sku_metrics, amazon_daily_sku_metrics, 
                                shopify_daily_sku_metrics, amazon_order_item_metrics, 
                                tiktok__order_items, shopify__order_items,
                                final_amazon_cohorts=None, final_tiktok_cohorts=None, 
                                final_shopify_cohorts=None, aov_avg=43):
    """
    Enhanced robust benchmark model with real cohort data integration
    """
    print("🛡️ ENHANCED ROBUST BENCHMARK MODEL")
    print("🔧 Features:")
    print("   ✅ Safe data handling")
    print("   ✅ Simple operations (no complex groupby)")
    print("   ✅ Multiple fallbacks")
    print("   ✅ Channel-specific AOV")
    print("   ✅ Seasonal adjustments")
    print("   ✅ REAL cohort-based retention rates")
    print("   ✅ Your exact formulas: BM_Demand = new_customers * AOV + existing_customers * AOV")
    print("   ✅ Your exact error: Actual_Demand - BM_Demand")
    
    benchmark = EnhancedRobustBenchmark(
        tiktok_daily_sku_metrics=tiktok_daily_sku_metrics,
        amazon_daily_sku_metrics=amazon_daily_sku_metrics,
        shopify_daily_sku_metrics=shopify_daily_sku_metrics,
        amazon_order_item_metrics=amazon_order_item_metrics,
        tiktok__order_items=tiktok__order_items,
        shopify__order_items=shopify__order_items,
        final_amazon_cohorts=final_amazon_cohorts,
        final_tiktok_cohorts=final_tiktok_cohorts,
        final_shopify_cohorts=final_shopify_cohorts,
        aov_avg=aov_avg
    )
    
    benchmark_df = benchmark.run_enhanced_benchmark()
    
    return benchmark, benchmark_df

# USAGE:
enhanced_model, enhanced_benchmark_df = run_enhanced_robust_benchmark(
    tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
    amazon_order_item_metrics, tiktok__order_items, shopify__order_items,
    final_amazon_cohorts, final_tiktok_cohorts, final_shopify_cohorts, aov_avg=43
)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

class EnhancedRobustBenchmark:
    def __init__(self, tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
                 amazon_order_item_metrics, tiktok__order_items, shopify__order_items, 
                 final_amazon_cohorts=None, final_tiktok_cohorts=None, final_shopify_cohorts=None,
                 aov_avg=43):
        """
        Enhanced robust benchmark that integrates real cohort data for better retention rates
        """
        print("🛡️ ENHANCED ROBUST Benchmark Model")
        print("🚀 Designed to avoid pandas groupby errors + Real cohort data")
        
        self.aov_avg = aov_avg
        
        # Store cohort data
        self.cohort_data = {}
        if final_amazon_cohorts is not None:
            self.cohort_data['amazon'] = final_amazon_cohorts.copy()
            print(f"   📊 Amazon cohorts: {len(final_amazon_cohorts):,} records")
        if final_tiktok_cohorts is not None:
            self.cohort_data['tiktok'] = final_tiktok_cohorts.copy()
            print(f"   📊 TikTok cohorts: {len(final_tiktok_cohorts):,} records")
        if final_shopify_cohorts is not None:
            self.cohort_data['shopify'] = final_shopify_cohorts.copy()
            print(f"   📊 Shopify cohorts: {len(final_shopify_cohorts):,} records")
        
        # Combine data safely (same as original)
        print("📊 Combining data...")
        daily_datasets = []
        
        if not tiktok_daily_sku_metrics.empty:
            tiktok_data = tiktok_daily_sku_metrics.copy()
            tiktok_data['channel'] = 'tiktok'
            daily_datasets.append(tiktok_data)
            print(f"   TikTok: {len(tiktok_data):,} records")
        
        if not amazon_daily_sku_metrics.empty:
            amazon_data = amazon_daily_sku_metrics.copy()
            amazon_data['channel'] = 'amazon'
            daily_datasets.append(amazon_data)
            print(f"   Amazon: {len(amazon_data):,} records")
        
        if not shopify_daily_sku_metrics.empty:
            shopify_data = shopify_daily_sku_metrics.copy()
            shopify_data['channel'] = 'shopify'
            daily_datasets.append(shopify_data)
            print(f"   Shopify: {len(shopify_data):,} records")
        
        if daily_datasets:
            self.daily_data = pd.concat(daily_datasets, ignore_index=True)
        else:
            print("❌ No daily data available!")
            self.daily_data = pd.DataFrame()
            return
        
        # Combine order data safely (same as original)
        order_datasets = []
        
        if not tiktok__order_items.empty:
            tiktok_orders = tiktok__order_items.copy()
            tiktok_orders['channel'] = 'tiktok'
            order_datasets.append(tiktok_orders)
        
        if not amazon_order_item_metrics.empty:
            amazon_orders = amazon_order_item_metrics.copy()
            amazon_orders['channel'] = 'amazon'
            order_datasets.append(amazon_orders)
        
        if not shopify__order_items.empty:
            shopify_orders = shopify__order_items.copy()
            shopify_orders['channel'] = 'shopify'
            order_datasets.append(shopify_orders)
        
        if order_datasets:
            self.order_items = pd.concat(order_datasets, ignore_index=True)
        else:
            print("⚠️ No order data available - will use customer-based estimates")
            self.order_items = pd.DataFrame()
        
        # Clean dates safely (same as original)
        print("📅 Cleaning dates...")
        try:
            self.daily_data['order_date'] = pd.to_datetime(self.daily_data['order_date'])
            print(f"   Daily data date range: {self.daily_data['order_date'].min()} to {self.daily_data['order_date'].max()}")
        except:
            print("❌ Error cleaning daily data dates")
            return
        
        if not self.order_items.empty:
            try:
                self.order_items['local_order_ts'] = pd.to_datetime(self.order_items['local_order_ts'])
                self.order_items['order_date'] = self.order_items['local_order_ts'].dt.date
                self.order_items['order_date'] = pd.to_datetime(self.order_items['order_date'])
                print(f"   Order data date range: {self.order_items['order_date'].min()} to {self.order_items['order_date'].max()}")
            except:
                print("⚠️ Error cleaning order dates - will use fallback AOV")
                self.order_items = pd.DataFrame()
        
        print(f"✅ Data loaded: {len(self.daily_data):,} daily records, {len(self.order_items):,} order items")
    
    def calculate_cohort_returning_rates(self):
       
        print("\n📊 Calculating returning rates based on sales fall-off...")
        
        if not self.cohort_data:
            print("   No cohort data available, using default rates")
            return self.calculate_fallback_retention_rates()
        
        all_channel_rates = {}
        
        for channel, cohort_df in self.cohort_data.items():
            print(f"\n🔍 Processing {channel} cohorts...")
            
            # Prepare cohort data
            df = cohort_df.copy()
            df['customer_cohort'] = pd.to_datetime(df['customer_cohort'])
            df['order_month'] = pd.to_datetime(df['order_month'])
            
            # Calculate customer age in months
            df['customer_age_months'] = (
                (df['order_month'].dt.year - df['customer_cohort'].dt.year) * 12 +
                (df['order_month'].dt.month - df['customer_cohort'].dt.month)
            )
            
            # Filter to reasonable ages (0-12 months)
            df = df[(df['customer_age_months'] >= 0) & (df['customer_age_months'] <= 12)]
            
            # DEBUGGING: Show sample data to understand the pattern
            print(f"   📊 Sample cohort sales progression:")
            sample_cohort = df['customer_cohort'].unique()[0]
            sample_data = df[df['customer_cohort'] == sample_cohort].sort_values('customer_age_months')
            
            for _, row in sample_data.head(8).iterrows():
                sales_amount = row['total_gross_sales']
                print(f"     Age {row['customer_age_months']:2d}: ${sales_amount:8.2f}")
            
            channel_rates = {}
            
            # Calculate sales retention rate for each age (1-12 months)
            for age_months in range(1, 13):
                
                retention_rates_for_age = []
                
                # For each cohort, calculate sales retention from initial spend
                for cohort_date in df['customer_cohort'].unique():
                    cohort_data = df[df['customer_cohort'] == cohort_date]
                    
                    # Find initial spend (where customer_cohort = order_month, i.e., age 0)
                    initial_data = cohort_data[cohort_data['customer_age_months'] == 0]
                    if len(initial_data) == 0:
                        continue
                    
                    initial_sales = initial_data['total_gross_sales'].sum()
                    if initial_sales <= 0:
                        continue
                    
                    # Find current spend at this age
                    current_data = cohort_data[cohort_data['customer_age_months'] == age_months]
                    if len(current_data) == 0:
                        current_sales = 0
                    else:
                        current_sales = current_data['total_gross_sales'].sum()
                    
                    # Calculate sales retention: current_sales / initial_sales
                    sales_retention = current_sales / initial_sales
                    sales_retention = min(sales_retention, 1.0)  # Cap at 100%
                    
                    # DEBUG: Show calculation for first few cohorts and ages
                    if age_months <= 3 and len(retention_rates_for_age) < 3:
                        print(f"     Debug {cohort_date.strftime('%Y-%m')} age {age_months}: ${initial_sales:.0f} → ${current_sales:.0f} = {sales_retention:.1%}")
                    
                    retention_rates_for_age.append(sales_retention)
                
                # Average sales retention rate across all cohorts for this age
                if retention_rates_for_age:
                    avg_retention_rate = np.mean(retention_rates_for_age)
                    channel_rates[age_months] = avg_retention_rate
                    print(f"   {age_months:2d} months: {avg_retention_rate:.1%} sales retention from {len(retention_rates_for_age)} cohorts")
                else:
                    # Fallback if no data - ensure it declines
                    fallback_rate = max(0.05, 0.75 ** age_months)
                    channel_rates[age_months] = fallback_rate
                    print(f"   {age_months:2d} months: {fallback_rate:.1%} sales retention (fallback - no data)")
            
            all_channel_rates[channel] = channel_rates
        
        # Calculate blended retention rates across channels
        blended_rates = {}
        for age_months in range(1, 13):
            rates_for_age = []
            for channel_rates in all_channel_rates.values():
                if age_months in channel_rates:
                    rates_for_age.append(channel_rates[age_months])
            
            if rates_for_age:
                blended_rates[age_months] = np.mean(rates_for_age)
            else:
                # Fallback that ensures declining pattern
                blended_rates[age_months] = max(0.05, 0.75 ** age_months)
        
        print(f"\n📊 Blended sales retention rates from cohort data:")
        for age in [1, 3, 6, 12]:
            if age in blended_rates:
                print(f"   {age:2d} months: {blended_rates[age]:.1%} (vs initial spend)")
        
        # SANITY CHECK: Ensure rates generally decline
        print(f"\n🔍 Sanity check - sales retention should generally decline:")
        declining_pattern = True
        for age in range(1, 12):
            curr_rate = blended_rates.get(age, 0)
            next_rate = blended_rates.get(age + 1, 0)
            if next_rate > curr_rate * 1.3:  # Allow some variation but flag big increases
                print(f"   ⚠️ WARNING: Month {age} to {age+1} shows unexpected increase: {curr_rate:.1%} → {next_rate:.1%}")
                declining_pattern = False
        
        if declining_pattern:
            print(f"   ✅ Sales retention pattern looks realistic (generally declining)")
        
        return blended_rates
    
    def calculate_fallback_retention_rates(self):
        """
        Fallback retention calculation (same as original) if no cohort data
        """
        print("\n👥 Calculating fallback retention rates...")
        
        # Check if we have customer lifecycle data
        has_lifecycle_data = ('pct_new_customers' in self.daily_data.columns and 
                             not self.daily_data['pct_new_customers'].isna().all())
        
        if has_lifecycle_data:
            avg_new_pct = self.daily_data['pct_new_customers'].mean()
            
            if avg_new_pct > 0.6:  # High churn environment
                base_retention = 0.70
                print("   High acquisition environment detected - using 70% base retention")
            elif avg_new_pct < 0.3:  # Low churn environment
                base_retention = 0.80
                print("   High retention environment detected - using 80% base retention")
            else:
                base_retention = 0.75
                print("   Balanced environment detected - using 75% base retention")
        else:
            base_retention = 0.75
            print("   Using standard 75% base retention")
        
        # Calculate retention for each age month
        retention_rates = {}
        for age_months in range(1, 13):
            retention = base_retention ** age_months
            retention = max(0.05, min(0.95, retention))
            retention_rates[age_months] = retention
        
        print(f"   1-month retention: {retention_rates[1]:.1%}")
        print(f"   6-month retention: {retention_rates[6]:.1%}")
        print(f"   12-month retention: {retention_rates[12]:.1%}")
        
        return retention_rates
    
    # Keep all the other methods from the original SimpleRobustBenchmark
    def calculate_simple_aov_by_channel(self):
        """Same as original"""
        print("\n💰 Calculating simple AOV by channel...")
        
        channel_aov = {}
        
        if self.order_items.empty:
            print("   Using default AOV for all channels")
            return {'tiktok': self.aov_avg, 'amazon': self.aov_avg, 'shopify': self.aov_avg}
        
        # Check required columns
        required_cols = ['sku_gross_sales', 'quantity', 'channel']
        missing_cols = [col for col in required_cols if col not in self.order_items.columns]
        
        if missing_cols:
            print(f"   Missing columns: {missing_cols}, using default AOV")
            return {'tiktok': self.aov_avg, 'amazon': self.aov_avg, 'shopify': self.aov_avg}
        
        for channel in ['tiktok', 'amazon', 'shopify']:
            print(f"   Calculating {channel} AOV...")
            
            # Filter channel data
            channel_orders = self.order_items[self.order_items['channel'] == channel].copy()
            
            if len(channel_orders) == 0:
                channel_aov[channel] = self.aov_avg
                print(f"     No {channel} orders, using default ${self.aov_avg}")
                continue
            
            # Clean data simply
            clean_orders = channel_orders[
                (channel_orders['quantity'] > 0) & 
                (channel_orders['sku_gross_sales'] > 0) &
                (channel_orders['sku_gross_sales'] < 10000) &  # Remove extreme outliers
                (channel_orders['quantity'] < 1000)  # Remove bulk orders
            ].copy()
            
            if len(clean_orders) == 0:
                channel_aov[channel] = self.aov_avg
                print(f"     No clean {channel} orders, using default ${self.aov_avg}")
                continue
            
            # Calculate unit prices
            clean_orders['unit_price'] = clean_orders['sku_gross_sales'] / clean_orders['quantity']
            
            # Remove unit price outliers
            unit_prices = clean_orders['unit_price']
            q1 = unit_prices.quantile(0.25)
            q3 = unit_prices.quantile(0.75)
            iqr = q3 - q1
            lower_bound = max(1, q1 - 1.5 * iqr)
            upper_bound = min(1000, q3 + 1.5 * iqr)
            
            final_prices = unit_prices[(unit_prices >= lower_bound) & (unit_prices <= upper_bound)]
            
            if len(final_prices) > 0:
                channel_aov[channel] = final_prices.median()
                print(f"     {channel.capitalize()} AOV: ${channel_aov[channel]:.2f} (from {len(final_prices):,} clean orders)")
            else:
                channel_aov[channel] = self.aov_avg
                print(f"     No valid {channel} prices, using default ${self.aov_avg}")
        
        return channel_aov
    
    def calculate_simple_seasonal_factors(self):
        """Same as original"""
        print("\n🗓️ Calculating simple seasonal factors...")
        
        # Add temporal columns safely
        try:
            self.daily_data['month'] = self.daily_data['order_date'].dt.month
            self.daily_data['dayofweek'] = self.daily_data['order_date'].dt.dayofweek
        except:
            print("   Error adding temporal columns, using default factors")
            return {
                'monthly_factors': {i: 1.0 for i in range(1, 13)},
                'dow_factors': {i: 1.0 for i in range(7)}
            }
        
        # Calculate monthly factors simply
        monthly_factors = {}
        overall_avg = self.daily_data['num_customers'].mean()
        
        if overall_avg > 0:
            for month in range(1, 13):
                month_data = self.daily_data[self.daily_data['month'] == month]
                if len(month_data) > 0:
                    month_avg = month_data['num_customers'].mean()
                    factor = month_avg / overall_avg
                    monthly_factors[month] = max(0.5, min(2.0, factor))  # Reasonable bounds
                else:
                    monthly_factors[month] = 1.0
        else:
            monthly_factors = {i: 1.0 for i in range(1, 13)}
        
        # Calculate day-of-week factors simply
        dow_factors = {}
        if overall_avg > 0:
            for dow in range(7):
                dow_data = self.daily_data[self.daily_data['dayofweek'] == dow]
                if len(dow_data) > 0:
                    dow_avg = dow_data['num_customers'].mean()
                    factor = dow_avg / overall_avg
                    dow_factors[dow] = max(0.5, min(2.0, factor))  # Reasonable bounds
                else:
                    dow_factors[dow] = 1.0
        else:
            dow_factors = {i: 1.0 for i in range(7)}
        
        print(f"   Monthly variation: {max(monthly_factors.values()) - min(monthly_factors.values()):.1%}")
        print(f"   Weekly variation: {max(dow_factors.values()) - min(dow_factors.values()):.1%}")
        
        return {
            'monthly_factors': monthly_factors,
            'dow_factors': dow_factors
        }
    
    def calculate_robust_benchmark(self):
        """
        Enhanced benchmark calculation using real cohort data for retention rates
        """
        print("\n🎯 Calculating enhanced robust benchmark...")
        
        if self.daily_data.empty:
            print("❌ No daily data available!")
            return pd.DataFrame()
        
        # Get patterns (using cohort data for retention rates!)
        channel_aov = self.calculate_simple_aov_by_channel()
        seasonal_patterns = self.calculate_simple_seasonal_factors()
        retention_rates = self.calculate_cohort_returning_rates()  # ✅ Use real cohort data!
        
        # Start with daily data
        benchmark_df = self.daily_data.copy()
        
        # Basic customer calculations
        benchmark_df['new_customers'] = benchmark_df['num_new_customers'].fillna(0)
        benchmark_df['existing_customers'] = np.maximum(0, 
            benchmark_df['num_customers'] - benchmark_df['new_customers'])
        benchmark_df['total_customers'] = benchmark_df['num_customers']
        
        # Apply AOV by channel
        print("   Applying channel-specific AOV...")
        benchmark_df['aov_used'] = benchmark_df['channel'].map(channel_aov).fillna(self.aov_avg)
        
        # Apply seasonal adjustments
        print("   Applying seasonal adjustments...")
        benchmark_df['monthly_factor'] = benchmark_df['month'].map(seasonal_patterns['monthly_factors']).fillna(1.0)
        benchmark_df['dow_factor'] = benchmark_df['dayofweek'].map(seasonal_patterns['dow_factors']).fillna(1.0)
        benchmark_df['seasonal_adjustment'] = (benchmark_df['monthly_factor'] * 0.7 + 
                                             benchmark_df['dow_factor'] * 0.3)
        
        # Calculate demand using YOUR EXACT FORMULA
        print("   Calculating benchmark demand using your formula...")
        benchmark_df['new_customer_demand'] = benchmark_df['new_customers'] * benchmark_df['aov_used']
        benchmark_df['existing_customer_demand'] = benchmark_df['existing_customers'] * benchmark_df['aov_used']
        
        # ✅ YOUR EXACT FORMULA: BM_Demand = new_customers * AOV + existing_customers * AOV
        benchmark_df['bm_demand'] = (benchmark_df['new_customer_demand'] + 
                                   benchmark_df['existing_customer_demand'])
        
        # Calculate actual demand
        print("   Calculating actual demand...")
        if not self.order_items.empty:
            try:
                # Try to get actual revenue
                actual_revenue = self.order_items.groupby(['product_name', 'order_date', 'channel']).agg({
                    'sku_gross_sales': 'sum'
                }).reset_index()
                
                benchmark_df = benchmark_df.merge(actual_revenue, 
                                                left_on=['product_name', 'order_date', 'channel'],
                                                right_on=['product_name', 'order_date', 'channel'], 
                                                how='left')
                
                benchmark_df['actual_demand'] = benchmark_df['sku_gross_sales'].fillna(
                    benchmark_df['total_customers'] * benchmark_df['aov_used'])
            except:
                print("     Revenue calculation failed, using customer-based estimate")
                benchmark_df['actual_demand'] = benchmark_df['total_customers'] * benchmark_df['aov_used']
        else:
            benchmark_df['actual_demand'] = benchmark_df['total_customers'] * benchmark_df['aov_used']
        
        # ✅ YOUR EXACT ERROR FORMULA: Actual_Demand - BM_Demand
        benchmark_df['error_metric'] = benchmark_df['actual_demand'] - benchmark_df['bm_demand']
        benchmark_df['error_percentage'] = np.where(
            benchmark_df['actual_demand'] > 0,
            (benchmark_df['error_metric'] / benchmark_df['actual_demand']) * 100,
            0
        )
        
        # Add real cohort-based retention rates
        print("   Adding real cohort-based retention rates...")
        for age_months in range(1, 13):
            real_retention = retention_rates[age_months]
            # Apply seasonal adjustment
            seasonal_retention = real_retention * benchmark_df['seasonal_adjustment']
            seasonal_retention = np.clip(seasonal_retention, 0.05, 0.95)
            
            benchmark_df[f'returning_rate_{age_months}m'] = seasonal_retention
            benchmark_df[f'falloff_rate_{age_months}m'] = 1 - seasonal_retention
        
        # Select final columns
        final_columns = [
            'product_name', 'order_date', 'channel',
            'new_customers', 'existing_customers', 'total_customers',
            'aov_used', 'new_customer_demand', 'existing_customer_demand',
            'bm_demand', 'actual_demand', 'error_metric', 'error_percentage'
        ] + [f'returning_rate_{i}m' for i in range(1, 13)] + [f'falloff_rate_{i}m' for i in range(1, 13)]
        
        # Only keep columns that exist
        existing_columns = [col for col in final_columns if col in benchmark_df.columns]
        benchmark_df = benchmark_df[existing_columns].copy()
        benchmark_df.rename(columns={'product_name': 'sku'}, inplace=True)
        
        return benchmark_df
    
    def run_enhanced_benchmark(self):
        """
        Run the enhanced benchmark with real cohort data
        """
        print("🚀 RUNNING ENHANCED ROBUST BENCHMARK")
        print("="*50)
        
        if self.daily_data.empty:
            print("❌ No data to process!")
            return pd.DataFrame()
        
        start_time = pd.Timestamp.now()
        
        benchmark_df = self.calculate_robust_benchmark()
        
        end_time = pd.Timestamp.now()
        duration = (end_time - start_time).total_seconds()
        
        if benchmark_df.empty:
            print("❌ No benchmark results generated!")
            return benchmark_df
        
        print(f"\n📊 ENHANCED BENCHMARK RESULTS:")
        print(f"  ⚡ Processed {len(benchmark_df):,} records in {duration:.1f} seconds")
        print(f"  🛍️ SKUs: {benchmark_df['sku'].nunique()}")
        print(f"  📅 Date range: {benchmark_df['order_date'].min()} to {benchmark_df['order_date'].max()}")
        
        # Performance by channel
        if 'channel' in benchmark_df.columns:
            print(f"\n📺 PERFORMANCE BY CHANNEL:")
            for channel in benchmark_df['channel'].unique():
                channel_data = benchmark_df[benchmark_df['channel'] == channel]
                avg_aov = channel_data['aov_used'].mean()
                mape = channel_data['error_percentage'].abs().mean()
                records = len(channel_data)
                
                print(f"  {channel.capitalize():<8}: {records:,} records, AOV ${avg_aov:.2f}, MAPE {mape:.1f}%")
        
        # Overall performance
        overall_mape = benchmark_df['error_percentage'].abs().mean()
        overall_aov = benchmark_df['aov_used'].mean()
        
        print(f"\n🎯 OVERALL PERFORMANCE:")
        print(f"  📊 Total MAPE: {overall_mape:.1f}%")
        print(f"  💰 Overall Avg AOV: ${overall_aov:.2f}")
        print(f"  📈 Avg BM Demand: ${benchmark_df['bm_demand'].mean():.2f}")
        print(f"  📈 Avg Actual Demand: ${benchmark_df['actual_demand'].mean():.2f}")
        print(f"  ✅ Using REAL cohort data for retention rates!")
        
        return benchmark_df

def run_enhanced_robust_benchmark(tiktok_daily_sku_metrics, amazon_daily_sku_metrics, 
                                shopify_daily_sku_metrics, amazon_order_item_metrics, 
                                tiktok__order_items, shopify__order_items,
                                final_amazon_cohorts=None, final_tiktok_cohorts=None, 
                                final_shopify_cohorts=None, aov_avg=43):
    """
    Enhanced robust benchmark model with real cohort data integration
    """
    print("🛡️ ENHANCED ROBUST BENCHMARK MODEL")
    print("🔧 Features:")
    print("   ✅ Safe data handling")
    print("   ✅ Simple operations (no complex groupby)")
    print("   ✅ Multiple fallbacks")
    print("   ✅ Channel-specific AOV")
    print("   ✅ Seasonal adjustments")
    print("   ✅ REAL cohort-based retention rates")
    print("   ✅ Your exact formulas: BM_Demand = new_customers * AOV + existing_customers * AOV")
    print("   ✅ Your exact error: Actual_Demand - BM_Demand")
    
    benchmark = EnhancedRobustBenchmark(
        tiktok_daily_sku_metrics=tiktok_daily_sku_metrics,
        amazon_daily_sku_metrics=amazon_daily_sku_metrics,
        shopify_daily_sku_metrics=shopify_daily_sku_metrics,
        amazon_order_item_metrics=amazon_order_item_metrics,
        tiktok__order_items=tiktok__order_items,
        shopify__order_items=shopify__order_items,
        final_amazon_cohorts=final_amazon_cohorts,
        final_tiktok_cohorts=final_tiktok_cohorts,
        final_shopify_cohorts=final_shopify_cohorts,
        aov_avg=aov_avg
    )
    
    benchmark_df = benchmark.run_enhanced_benchmark()
    
    return benchmark, benchmark_df

# USAGE:
enhanced_model, enhanced_benchmark_df = run_enhanced_robust_benchmark(
    tiktok_daily_sku_metrics, amazon_daily_sku_metrics, shopify_daily_sku_metrics,
    amazon_order_item_metrics, tiktok__order_items, shopify__order_items,
    final_amazon_cohorts, final_tiktok_cohorts, final_shopify_cohorts, aov_avg=43
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class ModelEvaluationFramework:
    def __init__(self, benchmark_df, baseline_model_df=None):
        
        print("📊 COMPREHENSIVE MODEL EVALUATION FRAMEWORK")
        print("="*60)
        
        self.benchmark_df = benchmark_df.copy()
        self.baseline_df = baseline_model_df.copy() if baseline_model_df is not None else None
        
        # Clean and prepare data
        self.prepare_evaluation_data()
        
        print(f"✅ Loaded benchmark data: {len(self.benchmark_df):,} records")
        if self.baseline_df is not None:
            print(f"✅ Loaded baseline data: {len(self.baseline_df):,} records")
        else:
            print("ℹ️ No baseline model provided - will use naive benchmarks")
    
    def prepare_evaluation_data(self):
        """
        Clean and prepare data for evaluation
        """
        print("\n🔧 Preparing evaluation data...")
        
        # Ensure required columns exist
        required_cols = ['order_date', 'sku', 'actual_demand', 'bm_demand', 'error_metric']
        missing_cols = [col for col in required_cols if col not in self.benchmark_df.columns]
        
        if missing_cols:
            print(f"❌ Missing required columns: {missing_cols}")
            return
        
        # Clean data
        self.benchmark_df = self.benchmark_df.dropna(subset=['actual_demand', 'bm_demand'])
        self.benchmark_df = self.benchmark_df[self.benchmark_df['actual_demand'] > 0]  # Remove zero demand
        
        # Add derived metrics
        self.benchmark_df['absolute_error'] = np.abs(self.benchmark_df['error_metric'])
        self.benchmark_df['percentage_error'] = np.where(
            self.benchmark_df['actual_demand'] > 0,
            (self.benchmark_df['error_metric'] / self.benchmark_df['actual_demand']) * 100,
            0
        )
        self.benchmark_df['absolute_percentage_error'] = np.abs(self.benchmark_df['percentage_error'])
        
        # Add temporal columns
        self.benchmark_df['order_date'] = pd.to_datetime(self.benchmark_df['order_date'])
        self.benchmark_df['year_month'] = self.benchmark_df['order_date'].dt.to_period('M')
        self.benchmark_df['quarter'] = self.benchmark_df['order_date'].dt.to_period('Q')
        self.benchmark_df['year'] = self.benchmark_df['order_date'].dt.year
        self.benchmark_df['month'] = self.benchmark_df['order_date'].dt.month
        
        # Calculate relative performance metrics
        if 'channel' in self.benchmark_df.columns:
            self.benchmark_df['channel'] = self.benchmark_df['channel'].fillna('unknown')
        
        print(f"   ✅ Clean records: {len(self.benchmark_df):,}")
        print(f"   📅 Date range: {self.benchmark_df['order_date'].min()} to {self.benchmark_df['order_date'].max()}")
        print(f"   🛍️ Unique SKUs: {self.benchmark_df['sku'].nunique()}")
        if 'channel' in self.benchmark_df.columns:
            print(f"   📺 Channels: {self.benchmark_df['channel'].unique()}")
    
    def calculate_overall_accuracy(self):
        """
        Calculate overall model accuracy metrics
        """
        print("\n📈 OVERALL ACCURACY METRICS")
        print("-" * 40)
        
        # Basic accuracy metrics
        mae = self.benchmark_df['absolute_error'].mean()
        mape = self.benchmark_df['absolute_percentage_error'].mean()
        rmse = np.sqrt((self.benchmark_df['error_metric'] ** 2).mean())
        
        # Relative metrics
        total_actual = self.benchmark_df['actual_demand'].sum()
        total_predicted = self.benchmark_df['bm_demand'].sum()
        total_error = self.benchmark_df['error_metric'].sum()
        
        bias = (total_predicted - total_actual) / total_actual * 100
        
        # Accuracy scores
        accuracy_within_10pct = (self.benchmark_df['absolute_percentage_error'] <= 10).mean() * 100
        accuracy_within_20pct = (self.benchmark_df['absolute_percentage_error'] <= 20).mean() * 100
        accuracy_within_50pct = (self.benchmark_df['absolute_percentage_error'] <= 50).mean() * 100
        
        # Directional accuracy
        over_predictions = (self.benchmark_df['error_metric'] < 0).sum()
        under_predictions = (self.benchmark_df['error_metric'] > 0).sum()
        perfect_predictions = (self.benchmark_df['error_metric'] == 0).sum()
        
        print(f"📊 ACCURACY SUMMARY:")
        print(f"   Mean Absolute Error (MAE):     ${mae:,.2f}")
        print(f"   Mean Absolute % Error (MAPE):  {mape:.1f}%")
        print(f"   Root Mean Square Error (RMSE): ${rmse:,.2f}")
        print(f"")
        print(f"📈 BIAS ANALYSIS:")
        print(f"   Overall Bias:                  {bias:+.1f}% ({'Over' if bias > 0 else 'Under'}-prediction)")
        print(f"   Total Actual Demand:           ${total_actual:,.2f}")
        print(f"   Total Predicted Demand:        ${total_predicted:,.2f}")
        print(f"")
        print(f"🎯 PREDICTION ACCURACY:")
        print(f"   Within 10%:  {accuracy_within_10pct:.1f}% of predictions")
        print(f"   Within 20%:  {accuracy_within_20pct:.1f}% of predictions")
        print(f"   Within 50%:  {accuracy_within_50pct:.1f}% of predictions")
        print(f"")
        print(f"📊 PREDICTION DIRECTION:")
        print(f"   Over-predictions:  {over_predictions:,} ({over_predictions/len(self.benchmark_df)*100:.1f}%)")
        print(f"   Under-predictions: {under_predictions:,} ({under_predictions/len(self.benchmark_df)*100:.1f}%)")
        print(f"   Perfect predictions: {perfect_predictions:,} ({perfect_predictions/len(self.benchmark_df)*100:.1f}%)")\
        
        return {
            'mae': mae,
            'mape': mape,
            'rmse': rmse,
            'bias': bias,
            'accuracy_10pct': accuracy_within_10pct,
            'accuracy_20pct': accuracy_within_20pct,
            'accuracy_50pct': accuracy_within_50pct
        }
    
    def calculate_weighted_accuracy(self):
        """
        Calculate accuracy weighted by product importance (revenue/volume)
        """
        print(f"\n💰 WEIGHTED ACCURACY BY PRODUCT IMPORTANCE")
        print("-" * 50)
        
        # Calculate product weights by total revenue
        product_revenue = self.benchmark_df.groupby('sku')['actual_demand'].sum().sort_values(ascending=False)
        total_revenue = product_revenue.sum()
        product_weights = product_revenue / total_revenue
        
        # Calculate weighted errors
        sku_errors = self.benchmark_df.groupby('sku').agg({
            'absolute_percentage_error': 'mean',
            'absolute_error': 'mean',
            'actual_demand': 'sum'
        }).reset_index()
        
        sku_errors['weight'] = sku_errors['sku'].map(product_weights)
        sku_errors = sku_errors.dropna()
        
        # Weighted metrics
        weighted_mape = (sku_errors['absolute_percentage_error'] * sku_errors['weight']).sum()
        weighted_mae = (sku_errors['absolute_error'] * sku_errors['weight']).sum()
        
        # Top product analysis
        top_10_skus = product_revenue.head(10)
        top_10_performance = self.benchmark_df[self.benchmark_df['sku'].isin(top_10_skus.index)]
        top_10_mape = top_10_performance['absolute_percentage_error'].mean()
        
        bottom_skus = product_revenue.tail(len(product_revenue)//2)  # Bottom 50%
        bottom_performance = self.benchmark_df[self.benchmark_df['sku'].isin(bottom_skus.index)]
        bottom_mape = bottom_performance['absolute_percentage_error'].mean()
        
        print(f"💎 WEIGHTED PERFORMANCE:")
        print(f"   Revenue-Weighted MAPE:    {weighted_mape:.1f}%")
        print(f"   Revenue-Weighted MAE:     ${weighted_mae:,.2f}")
        print(f"")
        print(f"🏆 TOP 10 PRODUCTS (by revenue):")
        print(f"   MAPE:                     {top_10_mape:.1f}%")
        print(f"   Revenue Share:            {top_10_skus.sum()/total_revenue*100:.1f}%")
        print(f"")
        print(f"📉 BOTTOM 50% PRODUCTS:")
        print(f"   MAPE:                     {bottom_mape:.1f}%")
        print(f"   Revenue Share:            {bottom_skus.sum()/total_revenue*100:.1f}%")
        print(f"")
        print(f"📊 PERFORMANCE SPREAD:")
        print(f"   Top vs Bottom Difference: {top_10_mape - bottom_mape:+.1f}% MAPE")\
        
        return {
            'weighted_mape': weighted_mape,
            'weighted_mae': weighted_mae,
            'top_10_mape': top_10_mape,
            'bottom_50_mape': bottom_mape,
            'top_products': top_10_skus
        }
    
    def analyze_channel_performance(self):
        """
        Analyze model performance across different channels
        """
        print(f"\n📺 CHANNEL PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        if 'channel' not in self.benchmark_df.columns:
            print("   ❌ No channel data available")
            return {}
        
        channel_performance = self.benchmark_df.groupby('channel').agg({
            'absolute_percentage_error': ['mean', 'std', 'count'],
            'absolute_error': 'mean',
            'actual_demand': 'sum',
            'bm_demand': 'sum',
            'error_metric': 'sum'
        }).round(2)
        
        channel_performance.columns = ['mape', 'mape_std', 'record_count', 'mae', 'actual_total', 'predicted_total', 'total_error']
        channel_performance = channel_performance.reset_index()
        
        # Calculate channel bias
        channel_performance['bias_pct'] = (
            (channel_performance['predicted_total'] - channel_performance['actual_total']) / 
            channel_performance['actual_total'] * 100
        )
        
        # Revenue share
        total_revenue = channel_performance['actual_total'].sum()
        channel_performance['revenue_share'] = channel_performance['actual_total'] / total_revenue * 100
        
        # Sort by performance
        channel_performance = channel_performance.sort_values('mape')
        
        print("📊 CHANNEL RANKINGS (by MAPE):")
        for _, row in channel_performance.iterrows():
            print(f"   {row['channel'].capitalize():<10}: {row['mape']:.1f}% MAPE, {row['bias_pct']:+.1f}% bias, {row['revenue_share']:.1f}% revenue share")
        
        best_channel = channel_performance.iloc[0]['channel']
        worst_channel = channel_performance.iloc[-1]['channel']
        performance_gap = channel_performance.iloc[-1]['mape'] - channel_performance.iloc[0]['mape']
        
        print(f"")
        print(f"🏆 CHANNEL INSIGHTS:")
        print(f"   Best Performing:  {best_channel.capitalize()} ({channel_performance.iloc[0]['mape']:.1f}% MAPE)")
        print(f"   Worst Performing: {worst_channel.capitalize()} ({channel_performance.iloc[-1]['mape']:.1f}% MAPE)")
        print(f"   Performance Gap:  {performance_gap:.1f}% MAPE difference")\
        
        return channel_performance.to_dict('records')
    
    def analyze_sku_performance(self):
        """
        Analyze model performance for top SKUs
        """
        print(f"\n🛍️ SKU PERFORMANCE ANALYSIS")
        print("-" * 35)
        
        # Calculate SKU-level performance
        sku_performance = self.benchmark_df.groupby('sku').agg({
            'absolute_percentage_error': 'mean',
            'absolute_error': 'mean',
            'actual_demand': ['sum', 'count'],
            'bm_demand': 'sum',
            'error_metric': 'sum'
        }).round(2)
        
        sku_performance.columns = ['mape', 'mae', 'total_revenue', 'record_count', 'predicted_total', 'total_error']
        sku_performance = sku_performance.reset_index()
        
        # Calculate bias
        sku_performance['bias_pct'] = (
            (sku_performance['predicted_total'] - sku_performance['total_revenue']) / 
            sku_performance['total_revenue'] * 100
        )
        
        # Filter to SKUs with significant volume (top 50% by revenue)
        revenue_threshold = sku_performance['total_revenue'].quantile(0.5)
        significant_skus = sku_performance[sku_performance['total_revenue'] >= revenue_threshold]
        
        # Top and bottom performers
        top_10_accurate = significant_skus.nsmallest(10, 'mape')
        bottom_10_accurate = significant_skus.nlargest(10, 'mape')
        
        print("🎯 TOP 10 MOST ACCURATE SKUs (among significant SKUs):")
        for _, row in top_10_accurate.iterrows():
            sku_short = row['sku'][:20] + "..." if len(row['sku']) > 20 else row['sku']
            print(f"   {sku_short:<25}: {row['mape']:.1f}% MAPE, ${row['total_revenue']:,.0f} revenue")
        
        print("")
        print("❌ BOTTOM 10 LEAST ACCURATE SKUs (among significant SKUs):")
        for _, row in bottom_10_accurate.iterrows():
            sku_short = row['sku'][:20] + "..." if len(row['sku']) > 20 else row['sku']
            print(f"   {sku_short:<25}: {row['mape']:.1f}% MAPE, ${row['total_revenue']:,.0f} revenue")
        
        # Summary stats
        avg_mape_all = sku_performance['mape'].mean()
        avg_mape_significant = significant_skus['mape'].mean()
        
        print("")
        print("📊 SKU PERFORMANCE SUMMARY:")
        print(f"   Average MAPE (all SKUs):        {avg_mape_all:.1f}%")
        print(f"   Average MAPE (significant SKUs): {avg_mape_significant:.1f}%")
        print(f"   SKUs analyzed:                  {len(sku_performance):,}")
        print(f"   Significant SKUs:               {len(significant_skus):,}")\
        
        return {
            'top_accurate': top_10_accurate.to_dict('records'),
            'bottom_accurate': bottom_10_accurate.to_dict('records'),
            'avg_mape_all': avg_mape_all,
            'avg_mape_significant': avg_mape_significant
        }
    
    def analyze_temporal_performance(self):
        """
        Analyze model performance over time
        """
        print(f"\n📅 TEMPORAL PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        # Monthly performance
        monthly_performance = self.benchmark_df.groupby('year_month').agg({
            'absolute_percentage_error': 'mean',
            'absolute_error': 'mean',
            'actual_demand': 'sum',
            'bm_demand': 'sum',
            'error_metric': 'sum'
        }).reset_index()
        
        monthly_performance['bias_pct'] = (
            (monthly_performance['bm_demand'] - monthly_performance['actual_demand']) / 
            monthly_performance['actual_demand'] * 100
        )
        
        # Trend analysis
        monthly_performance['month_num'] = range(len(monthly_performance))
        mape_trend = np.corrcoef(monthly_performance['month_num'], monthly_performance['absolute_percentage_error'])[0,1]
        
        # Best and worst months
        best_month = monthly_performance.loc[monthly_performance['absolute_percentage_error'].idxmin()]
        worst_month = monthly_performance.loc[monthly_performance['absolute_percentage_error'].idxmax()]
        
        # Consistency metrics
        mape_std = monthly_performance['absolute_percentage_error'].std()
        mape_cv = mape_std / monthly_performance['absolute_percentage_error'].mean()  # Coefficient of variation
        
        print("📈 TEMPORAL TRENDS:")
        print(f"   MAPE Trend Correlation:  {mape_trend:+.3f} ({'Improving' if mape_trend < 0 else 'Deteriorating'} over time)")
        print(f"   MAPE Standard Deviation: {mape_std:.1f}%")
        print(f"   MAPE Consistency (CV):   {mape_cv:.2f} ({'Consistent' if mape_cv < 0.3 else 'Variable'})")
        print("")
        print(f"🏆 BEST MONTH:  {best_month['year_month']} ({best_month['absolute_percentage_error']:.1f}% MAPE)")
        print(f"❌ WORST MONTH: {worst_month['year_month']} ({worst_month['absolute_percentage_error']:.1f}% MAPE)")
        print("")
        print(f"📊 PERFORMANCE RANGE: {worst_month['absolute_percentage_error'] - best_month['absolute_percentage_error']:.1f}% MAPE spread")\
        
        return {
            'monthly_performance': monthly_performance,
            'mape_trend': mape_trend,
            'best_month': best_month,
            'worst_month': worst_month,
            'consistency': mape_cv
        }
    
    def cross_validation_analysis(self):
        """
        Perform time-series cross validation analysis
        """
        print(f"\n🔄 CROSS-VALIDATION ANALYSIS")
        print("-" * 35)
        
        # Sort by date for time series CV
        df_sorted = self.benchmark_df.sort_values('order_date')
        
        # Define train/test splits (expanding window)
        months = sorted(df_sorted['year_month'].unique())
        
        if len(months) < 6:
            print("   ❌ Need at least 6 months of data for cross-validation")
            return {}
        
        cv_results = []
        min_train_months = 3
        
        for i in range(min_train_months, len(months)):
            train_months = months[:i]
            test_month = months[i]
            
            train_data = df_sorted[df_sorted['year_month'].isin(train_months)]
            test_data = df_sorted[df_sorted['year_month'] == test_month]
            
            if len(test_data) == 0:
                continue
            
            # Calculate baseline (simple average from training data)
            baseline_avg = train_data['actual_demand'].mean()
            
            # Performance metrics for test month
            test_mape = test_data['absolute_percentage_error'].mean()
            test_mae = test_data['absolute_error'].mean()
            
            # Baseline comparison (naive forecast = training average)
            baseline_errors = np.abs(test_data['actual_demand'] - baseline_avg)
            baseline_mape = (baseline_errors / test_data['actual_demand']).mean() * 100
            
            model_beats_baseline = test_mape < baseline_mape
            
            cv_results.append({
                'test_month': test_month,
                'train_months': len(train_months),
                'test_records': len(test_data),
                'model_mape': test_mape,
                'baseline_mape': baseline_mape,
                'improvement': baseline_mape - test_mape,
                'beats_baseline': model_beats_baseline
            })
        
        cv_df = pd.DataFrame(cv_results)
        
        # Summary statistics
        avg_improvement = cv_df['improvement'].mean()
        win_rate = cv_df['beats_baseline'].mean() * 100
        consistent_months = cv_df['beats_baseline'].sum()
        total_months = len(cv_df)
        
        print("🎯 CROSS-VALIDATION RESULTS:")
        print(f"   Months Tested:           {total_months}")
        print(f"   Win Rate vs Baseline:    {win_rate:.1f}% ({consistent_months}/{total_months} months)")
        print(f"   Average Improvement:     {avg_improvement:+.1f}% MAPE vs naive baseline")
        print("")
        print("📊 MONTH-BY-MONTH PERFORMANCE:")
        
        for _, row in cv_df.tail(12).iterrows():  # Show last 12 months
            status = "✅ WIN" if row['beats_baseline'] else "❌ LOSS"
            print(f"   {row['test_month']}: {row['model_mape']:.1f}% vs {row['baseline_mape']:.1f}% baseline = {row['improvement']:+.1f}% {status}")\
        
        return cv_df
    
    def cumulative_error_analysis(self):
        """
        Analyze cumulative error impact over time
        """
        print(f"\n📈 CUMULATIVE ERROR ANALYSIS")
        print("-" * 35)
        
        # Monthly aggregated data
        monthly_data = self.benchmark_df.groupby('year_month').agg({
            'actual_demand': 'sum',
            'bm_demand': 'sum',
            'error_metric': 'sum'
        }).reset_index()
        
        # Calculate cumulative metrics
        monthly_data['cumulative_actual'] = monthly_data['actual_demand'].cumsum()
        monthly_data['cumulative_predicted'] = monthly_data['bm_demand'].cumsum()
        monthly_data['cumulative_error'] = monthly_data['error_metric'].cumsum()
        monthly_data['cumulative_error_pct'] = (monthly_data['cumulative_error'] / monthly_data['cumulative_actual']) * 100
        
        # If following model for full year projection
        last_month_error_rate = monthly_data['error_metric'].iloc[-1] / monthly_data['actual_demand'].iloc[-1]
        avg_monthly_demand = monthly_data['actual_demand'].mean()
        
        # Project annual impact
        annual_error_projection = last_month_error_rate * avg_monthly_demand * 12
        annual_demand_projection = avg_monthly_demand * 12
        annual_error_pct = (annual_error_projection / annual_demand_projection) * 100
        
        # Final cumulative error
        final_cumulative_error = monthly_data['cumulative_error_pct'].iloc[-1]
        
        print("💰 CUMULATIVE IMPACT ANALYSIS:")
        print("")
        print("📊 CURRENT CUMULATIVE PERFORMANCE:")
        print(f"   Total Actual Demand:     ${monthly_data['cumulative_actual'].iloc[-1]:,.0f}")
        print(f"   Total Predicted Demand:  ${monthly_data['cumulative_predicted'].iloc[-1]:,.0f}")
        print(f"   Cumulative Error:        ${monthly_data['cumulative_error'].iloc[-1]:,.0f}")
        print(f"   Cumulative Error %:      {final_cumulative_error:+.1f}%")
        print("")
        print("🔮 ANNUAL PROJECTION (if following model):")
        print(f"   Projected Annual Demand: ${annual_demand_projection:,.0f}")
        print(f"   Projected Annual Error:  ${annual_error_projection:,.0f}")
        print(f"   Projected Error Rate:    {annual_error_pct:+.1f}%")
        print("")
        print("⚠️ RISK ASSESSMENT:")
        
        if abs(annual_error_pct) < 5:
            risk_level = "🟢 LOW RISK"
        elif abs(annual_error_pct) < 15:
            risk_level = "🟡 MODERATE RISK"
        else:
            risk_level = "🔴 HIGH RISK"
        
        print(f"   {risk_level} - {abs(annual_error_pct):.1f}% projected annual error")
        
        # Show monthly progression
        print("")
        print("📅 MONTHLY ERROR PROGRESSION:")
        for _, row in monthly_data.tail(6).iterrows():
            print(f"   {row['year_month']}: {row['cumulative_error_pct']:+.1f}% cumulative error")\
        
        return {
            'monthly_data': monthly_data,
            'annual_projection': annual_error_pct,
            'cumulative_error': final_cumulative_error,
            'risk_level': risk_level
        }
    
    def run_comprehensive_evaluation(self):
        """
        Run all evaluation analyses
        """
        print("🚀 RUNNING COMPREHENSIVE MODEL EVALUATION")
        print("="*60)
        
        results = {}
        
        try:
            results['overall'] = self.calculate_overall_accuracy()
            results['weighted'] = self.calculate_weighted_accuracy()
            results['channels'] = self.analyze_channel_performance()
            results['skus'] = self.analyze_sku_performance()
            results['temporal'] = self.analyze_temporal_performance()
            results['cross_validation'] = self.cross_validation_analysis()
            results['cumulative'] = self.cumulative_error_analysis()
            
            print(f"\n🎉 EVALUATION COMPLETE!")
            print(f"="*60)
            
            return results
            
        except Exception as e:
            print(f"❌ Error during evaluation: {str(e)}")
            return results

# Usage function
def evaluate_model_performance(benchmark_df, baseline_df=None):
    """
    Main function to evaluate model performance comprehensively
    
    Args:
        benchmark_df: Your enhanced benchmark model results
        baseline_df: Optional baseline model for comparison
    
    Returns:
        ModelEvaluationFramework instance with all results
    """
    
    evaluator = ModelEvaluationFramework(benchmark_df, baseline_df)
    results = evaluator.run_comprehensive_evaluation()
    
    return evaluator, results

# To use:
evaluator, evaluation_results = evaluate_model_performance(enhanced_benchmark_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

class MLModelEvaluationAnalysis:
    def __init__(self, test_data, model_results, benchmark_col='bm_demand', actual_col='actual_demand'):
        
        print("🔍 COMPREHENSIVE ML MODEL PERFORMANCE ANALYSIS")
        print("="*60)
        
        self.test_data = test_data.copy()
        self.model_results = model_results
        self.benchmark_col = benchmark_col
        self.actual_col = actual_col
        
        self.add_ml_predictions_to_test_data()
        
        self.prepare_analysis_data()
        
    def add_ml_predictions_to_test_data(self):
        
        print("\n📊 Adding ML predictions to test data...")
        
        for model_name, results in self.model_results.items():
            if 'predictions' in results:
                # Ensure we have the right number of predictions
                predictions = results['predictions']
                if len(predictions) == len(self.test_data):
                    self.test_data[f'{model_name}_pred'] = predictions
                    print(f"   ✅ Added {model_name} predictions")
                else:
                    print(f"   ⚠️ {model_name} prediction length mismatch: {len(predictions)} vs {len(self.test_data)}")
    
    def prepare_analysis_data(self):
        """
        Prepare data for comprehensive analysis
        """
        print("\n🔧 Preparing analysis data...")
        
        # Ensure required columns exist
        if self.actual_col not in self.test_data.columns:
            print(f"❌ Missing actual demand column: {self.actual_col}")
            return
        
        if self.benchmark_col not in self.test_data.columns:
            print(f"❌ Missing benchmark column: {self.benchmark_col}")
            return
        
        # Add temporal columns
        if 'order_date' in self.test_data.columns:
            self.test_data['order_date'] = pd.to_datetime(self.test_data['order_date'])
            self.test_data['year_month'] = self.test_data['order_date'].dt.to_period('M')
            self.test_data['month'] = self.test_data['order_date'].dt.month
            self.test_data['quarter'] = self.test_data['order_date'].dt.quarter
        
        # Calculate benchmark errors
        self.test_data['benchmark_error'] = self.test_data[self.actual_col] - self.test_data[self.benchmark_col]
        self.test_data['benchmark_abs_error'] = np.abs(self.test_data['benchmark_error'])
        self.test_data['benchmark_pct_error'] = np.where(
            self.test_data[self.actual_col] > 0,
            (self.test_data['benchmark_error'] / self.test_data[self.actual_col]) * 100,
            0
        )
        self.test_data['benchmark_abs_pct_error'] = np.abs(self.test_data['benchmark_pct_error'])
        
        # Calculate ML model errors
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            self.test_data[f'{model_name}_error'] = self.test_data[self.actual_col] - self.test_data[pred_col]
            self.test_data[f'{model_name}_abs_error'] = np.abs(self.test_data[f'{model_name}_error'])
            self.test_data[f'{model_name}_pct_error'] = np.where(
                self.test_data[self.actual_col] > 0,
                (self.test_data[f'{model_name}_error'] / self.test_data[self.actual_col]) * 100,
                0
            )
            self.test_data[f'{model_name}_abs_pct_error'] = np.abs(self.test_data[f'{model_name}_pct_error'])
        
        print(f"   ✅ Analysis data prepared: {len(self.test_data):,} records")
        print(f"   📊 ML models found: {len(ml_pred_cols)}")
        
    def analyze_overall_accuracy(self):
        """
        1. Overall Accuracy Analysis
        """
        print("\n📈 1. OVERALL ACCURACY ANALYSIS")
        print("-" * 45)
        
        # Get ML model prediction columns
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        if not ml_pred_cols:
            print("❌ No ML model predictions found")
            return {}
        
        accuracy_results = {}
        
        # Benchmark performance
        benchmark_mae = self.test_data['benchmark_abs_error'].mean()
        benchmark_mape = self.test_data['benchmark_abs_pct_error'].mean()
        
        print(f"🎯 BENCHMARK PERFORMANCE:")
        print(f"   MAE:  ${benchmark_mae:,.2f}")
        print(f"   MAPE: {benchmark_mape:.1f}%")
        print("")
        
        accuracy_results['benchmark'] = {'mae': benchmark_mae, 'mape': benchmark_mape}
        
        # ML model performance
        print(f"🤖 ML MODEL PERFORMANCE:")
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            ml_mae = self.test_data[f'{model_name}_abs_error'].mean()
            ml_mape = self.test_data[f'{model_name}_abs_pct_error'].mean()
            
            # Improvement calculations
            mae_improvement = benchmark_mae - ml_mae
            mae_improvement_pct = (mae_improvement / benchmark_mae) * 100
            
            mape_improvement = benchmark_mape - ml_mape
            
            status = "✅ BETTER" if mae_improvement > 0 else "❌ WORSE"
            
            print(f"   {model_name}:")
            print(f"     MAE:  ${ml_mae:,.2f} (${mae_improvement:+,.2f}, {mae_improvement_pct:+.1f}%) {status}")
            print(f"     MAPE: {ml_mape:.1f}% ({mape_improvement:+.1f}%)")
            print("")
            
            accuracy_results[model_name] = {
                'mae': ml_mae,
                'mape': ml_mape,
                'mae_improvement': mae_improvement,
                'mae_improvement_pct': mae_improvement_pct,
                'mape_improvement': mape_improvement
            }
        
        return accuracy_results
    
    def analyze_weighted_accuracy(self):
        """
        2. Weighted Accuracy by Product Revenue
        """
        print("\n💰 2. WEIGHTED ACCURACY BY PRODUCT IMPORTANCE")
        print("-" * 55)
        
        if 'sku' not in self.test_data.columns:
            print("❌ No SKU column found for product analysis")
            return {}
        
        # Calculate product weights by revenue
        product_revenue = self.test_data.groupby('sku')[self.actual_col].sum().sort_values(ascending=False)
        total_revenue = product_revenue.sum()
        product_weights = product_revenue / total_revenue
        
        # Top products analysis
        top_20_pct_threshold = product_revenue.quantile(0.8)
        top_products = product_revenue[product_revenue >= top_20_pct_threshold].index
        
        top_product_data = self.test_data[self.test_data['sku'].isin(top_products)]
        bottom_product_data = self.test_data[~self.test_data['sku'].isin(top_products)]
        
        print(f"📊 PRODUCT SEGMENTATION:")
        print(f"   Top 20% Products: {len(top_products)} SKUs, ${product_revenue[product_revenue >= top_20_pct_threshold].sum():,.0f} revenue ({product_revenue[product_revenue >= top_20_pct_threshold].sum()/total_revenue*100:.1f}%)")
        print(f"   Bottom 80% Products: {len(product_revenue) - len(top_products)} SKUs, ${product_revenue[product_revenue < top_20_pct_threshold].sum():,.0f} revenue ({product_revenue[product_revenue < top_20_pct_threshold].sum()/total_revenue*100:.1f}%)")
        print("")
        
        weighted_results = {}
        
        # Analyze performance on top vs bottom products
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for segment_name, segment_data in [('Top 20% Products', top_product_data), ('Bottom 80% Products', bottom_product_data)]:
            print(f"🎯 {segment_name.upper()}:")
            
            # Benchmark performance
            bench_mae = segment_data['benchmark_abs_error'].mean()
            bench_mape = segment_data['benchmark_abs_pct_error'].mean()
            print(f"   Benchmark: MAE ${bench_mae:.2f}, MAPE {bench_mape:.1f}%")
            
            segment_results = {'benchmark': {'mae': bench_mae, 'mape': bench_mape}}
            
            # ML model performance
            for pred_col in ml_pred_cols:
                model_name = pred_col.replace('_pred', '')
                ml_mae = segment_data[f'{model_name}_abs_error'].mean()
                ml_mape = segment_data[f'{model_name}_abs_pct_error'].mean()
                
                mae_improvement = bench_mae - ml_mae
                mae_improvement_pct = (mae_improvement / bench_mae) * 100
                
                status = "✅" if mae_improvement > 0 else "❌"
                print(f"   {model_name}: MAE ${ml_mae:.2f} ({mae_improvement_pct:+.1f}%) {status}")
                
                segment_results[model_name] = {
                    'mae': ml_mae,
                    'mape': ml_mape,
                    'improvement_pct': mae_improvement_pct
                }
            
            weighted_results[segment_name] = segment_results
            print("")
        
        return weighted_results
    
    def analyze_channel_performance(self):
        """
        3. Channel Performance Analysis
        """
        print("\n📺 3. CHANNEL PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        if 'channel' not in self.test_data.columns:
            print("❌ No channel column found")
            return {}
        
        channel_results = {}
        channels = self.test_data['channel'].unique()
        
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for channel in channels:
            channel_data = self.test_data[self.test_data['channel'] == channel]
            
            print(f"📻 {channel.upper()} CHANNEL:")
            print(f"   Records: {len(channel_data):,}")
            print(f"   Revenue: ${channel_data[self.actual_col].sum():,.0f}")
            
            # Benchmark performance
            bench_mae = channel_data['benchmark_abs_error'].mean()
            bench_mape = channel_data['benchmark_abs_pct_error'].mean()
            print(f"   Benchmark: MAE ${bench_mae:.2f}, MAPE {bench_mape:.1f}%")
            
            channel_results[channel] = {'benchmark': {'mae': bench_mae, 'mape': bench_mape}}
            
            # ML model performance  
            for pred_col in ml_pred_cols:
                model_name = pred_col.replace('_pred', '')
                ml_mae = channel_data[f'{model_name}_abs_error'].mean()
                ml_mape = channel_data[f'{model_name}_abs_pct_error'].mean()
                
                mae_improvement = bench_mae - ml_mae
                mae_improvement_pct = (mae_improvement / bench_mae) * 100
                
                status = "✅ BETTER" if mae_improvement > 0 else "❌ WORSE"
                print(f"   {model_name}: MAE ${ml_mae:.2f} ({mae_improvement_pct:+.1f}%) {status}")
                
                channel_results[channel][model_name] = {
                    'mae': ml_mae,
                    'mape': ml_mape,
                    'improvement_pct': mae_improvement_pct
                }
            
            print("")
        
        return channel_results
    
    def analyze_temporal_performance(self):
        """
        4. Temporal Performance Analysis
        """
        print("\n📅 4. TEMPORAL PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        if 'year_month' not in self.test_data.columns:
            print("❌ No temporal data available")
            return {}
        
        # Monthly performance analysis
        monthly_data = self.test_data.groupby('year_month').agg({
            self.actual_col: 'sum',
            self.benchmark_col: 'sum',
            'benchmark_abs_error': 'mean',
            'benchmark_abs_pct_error': 'mean'
        }).round(2)
        
        # Add ML model monthly performance
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            monthly_ml = self.test_data.groupby('year_month').agg({
                f'{model_name}_abs_error': 'mean',
                f'{model_name}_abs_pct_error': 'mean'
            }).round(2)
            
            monthly_data = monthly_data.join(monthly_ml)
        
        # Calculate month-by-month wins/losses
        temporal_results = {}
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            # Count wins per month
            monthly_comparison = []
            
            for month in monthly_data.index:
                month_data = self.test_data[self.test_data['year_month'] == month]
                
                if len(month_data) == 0:
                    continue
                
                bench_mae = month_data['benchmark_abs_error'].mean()
                ml_mae = month_data[f'{model_name}_abs_error'].mean()
                
                wins_benchmark = ml_mae < bench_mae
                improvement = bench_mae - ml_mae
                improvement_pct = (improvement / bench_mae) * 100 if bench_mae > 0 else 0
                
                monthly_comparison.append({
                    'month': month,
                    'benchmark_mae': bench_mae,
                    'ml_mae': ml_mae,
                    'wins_benchmark': wins_benchmark,
                    'improvement': improvement,
                    'improvement_pct': improvement_pct,
                    'records': len(month_data)
                })
            
            comparison_df = pd.DataFrame(monthly_comparison)
            
            # Summary statistics
            total_months = len(comparison_df)
            months_won = comparison_df['wins_benchmark'].sum()
            win_rate = (months_won / total_months) * 100 if total_months > 0 else 0
            avg_improvement = comparison_df['improvement_pct'].mean()
            
            print(f"📊 {model_name.upper()} MONTHLY PERFORMANCE:")
            print(f"   Months tested: {total_months}")
            print(f"   Months won: {months_won}/{total_months} ({win_rate:.1f}%)")
            print(f"   Average improvement: {avg_improvement:+.1f}%")
            
            # Show recent monthly performance
            print(f"   Recent performance:")
            for _, row in comparison_df.tail(6).iterrows():
                status = "✅ WIN" if row['wins_benchmark'] else "❌ LOSS"
                print(f"     {row['month']}: {row['improvement_pct']:+.1f}% {status}")
            
            temporal_results[model_name] = {
                'total_months': total_months,
                'months_won': months_won,
                'win_rate': win_rate,
                'avg_improvement': avg_improvement,
                'monthly_details': comparison_df
            }
            print("")
        
        return temporal_results
    
    def analyze_cumulative_error_impact(self):
        """
        5. Cumulative Error & Annual Risk Analysis
        """
        print("\n💰 5. CUMULATIVE ERROR & ANNUAL RISK ANALYSIS")
        print("-" * 50)
        
        if 'year_month' not in self.test_data.columns:
            print("❌ No temporal data for cumulative analysis")
            return {}
        
        # Monthly aggregated data
        monthly_data = self.test_data.groupby('year_month').agg({
            self.actual_col: 'sum',
            self.benchmark_col: 'sum',
            'benchmark_error': 'sum'
        }).reset_index()
        
        # Add ML model data
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        cumulative_results = {}
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            # Calculate monthly ML predictions and errors
            monthly_ml = self.test_data.groupby('year_month').agg({
                pred_col: 'sum',
                f'{model_name}_error': 'sum'
            }).reset_index()
            
            # Merge with main monthly data
            monthly_combined = monthly_data.merge(monthly_ml, on='year_month')
            
            # Calculate cumulative metrics
            monthly_combined['cumulative_actual'] = monthly_combined[self.actual_col].cumsum()
            monthly_combined['cumulative_benchmark'] = monthly_combined[self.benchmark_col].cumsum()
            monthly_combined['cumulative_ml'] = monthly_combined[pred_col].cumsum()
            
            monthly_combined['cumulative_benchmark_error'] = monthly_combined['benchmark_error'].cumsum()
            monthly_combined['cumulative_ml_error'] = monthly_combined[f'{model_name}_error'].cumsum()
            
            monthly_combined['cumulative_benchmark_error_pct'] = (monthly_combined['cumulative_benchmark_error'] / monthly_combined['cumulative_actual']) * 100
            monthly_combined['cumulative_ml_error_pct'] = (monthly_combined['cumulative_ml_error'] / monthly_combined['cumulative_actual']) * 100
            
            # Annual projections
            avg_monthly_demand = monthly_combined[self.actual_col].mean()
            last_month_ml_error_rate = monthly_combined[f'{model_name}_error'].iloc[-1] / monthly_combined[self.actual_col].iloc[-1]
            last_month_benchmark_error_rate = monthly_combined['benchmark_error'].iloc[-1] / monthly_combined[self.actual_col].iloc[-1]
            
            # Project full year impact
            annual_demand_projection = avg_monthly_demand * 12
            annual_ml_error_projection = last_month_ml_error_rate * avg_monthly_demand * 12
            annual_benchmark_error_projection = last_month_benchmark_error_rate * avg_monthly_demand * 12
            
            annual_ml_error_pct = (annual_ml_error_projection / annual_demand_projection) * 100
            annual_benchmark_error_pct = (annual_benchmark_error_projection / annual_demand_projection) * 100
            
            # Final cumulative error
            final_ml_cumulative_error = monthly_combined['cumulative_ml_error_pct'].iloc[-1]
            final_benchmark_cumulative_error = monthly_combined['cumulative_benchmark_error_pct'].iloc[-1]
            
            print(f"📊 {model_name.upper()} CUMULATIVE ANALYSIS:")
            print(f"   Current cumulative error: {final_ml_cumulative_error:+.1f}%")
            print(f"   Benchmark cumulative error: {final_benchmark_cumulative_error:+.1f}%")
            print(f"   Projected annual error: {annual_ml_error_pct:+.1f}%")
            print(f"   Benchmark annual error: {annual_benchmark_error_pct:+.1f}%")
            
            # Risk assessment
            if abs(annual_ml_error_pct) < 5:
                risk_level = "🟢 LOW RISK"
            elif abs(annual_ml_error_pct) < 15:
                risk_level = "🟡 MODERATE RISK"
            else:
                risk_level = "🔴 HIGH RISK"
            
            print(f"   Risk Level: {risk_level}")
            
            # Show if ML beats benchmark cumulatively
            cumulative_improvement = final_benchmark_cumulative_error - final_ml_cumulative_error
            annual_improvement = annual_benchmark_error_pct - annual_ml_error_pct
            
            print(f"   Cumulative improvement: {cumulative_improvement:+.1f}%")
            print(f"   Annual improvement: {annual_improvement:+.1f}%")
            
            cumulative_results[model_name] = {
                'final_cumulative_error': final_ml_cumulative_error,
                'projected_annual_error': annual_ml_error_pct,
                'risk_level': risk_level,
                'cumulative_improvement': cumulative_improvement,
                'annual_improvement': annual_improvement,
                'monthly_data': monthly_combined
            }
            print("")
        
        return cumulative_results
    
    def feature_importance_deep_dive(self):
    
        
        # Analyze prediction patterns
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        if ml_pred_cols and 'demand_ma_3d' in self.test_data.columns:
            print("📊 CORRELATION ANALYSIS:")
            
            for pred_col in ml_pred_cols[:2]:  # Check top 2 models
                model_name = pred_col.replace('_pred', '')
                
                # Correlation between ML prediction and 3-day moving average
                if 'demand_ma_3d' in self.test_data.columns:
                    correlation = self.test_data[pred_col].corr(self.test_data['demand_ma_3d'])
                    print(f"   {model_name} vs demand_ma_3d: {correlation:.3f} correlation")
                
                # Correlation with actual demand
                correlation_actual = self.test_data[pred_col].corr(self.test_data[self.actual_col])
                print(f"   {model_name} vs actual demand: {correlation_actual:.3f} correlation")
                
                # Correlation with benchmark
                correlation_benchmark = self.test_data[pred_col].corr(self.test_data[self.benchmark_col])
                print(f"   {model_name} vs benchmark: {correlation_benchmark:.3f} correlation")
                print("")
    
    def run_comprehensive_analysis(self):
        """
        Run all analyses and provide summary
        """
        print("🚀 RUNNING COMPREHENSIVE ML MODEL ANALYSIS")
        print("="*60)
        
        results = {}
        
        try:
            results['overall_accuracy'] = self.analyze_overall_accuracy()
            results['weighted_accuracy'] = self.analyze_weighted_accuracy()
            results['channel_performance'] = self.analyze_channel_performance()
            results['temporal_performance'] = self.analyze_temporal_performance()
            results['cumulative_analysis'] = self.analyze_cumulative_error_impact()
            self.feature_importance_deep_dive()
            
            # Summary conclusions
            self.provide_final_recommendations(results)
            
            return results
            
        except Exception as e:
            print(f"❌ Error during analysis: {str(e)}")
            return results
    
    def provide_final_recommendations(self, results):
        """
        Provide final recommendations based on analysis
        """
        print("\n🎯 FINAL RECOMMENDATIONS")
        print("="*30)
        
        # Find best performing ML model
        if 'overall_accuracy' in results:
            best_model = None
            best_improvement = -float('inf')
            
            for model_name, metrics in results['overall_accuracy'].items():
                if model_name != 'benchmark' and 'mae_improvement_pct' in metrics:
                    if metrics['mae_improvement_pct'] > best_improvement:
                        best_improvement = metrics['mae_improvement_pct']
                        best_model = model_name
            
            if best_model and best_improvement > 0:
                print(f"✅ BEST MODEL: {best_model}")
                print(f"   Overall improvement: {best_improvement:+.1f}%")
                
                # Check consistency
                if 'temporal_performance' in results and best_model in results['temporal_performance']:
                    win_rate = results['temporal_performance'][best_model]['win_rate']
                    print(f"   Monthly win rate: {win_rate:.1f}%")
                    
                    if win_rate >= 70:
                        print("   🟢 CONSISTENT performer - safe to deploy")
                    elif win_rate >= 50:
                        print("   🟡 MODERATELY consistent - monitor closely")
                    else:
                        print("   🔴 INCONSISTENT - risky for production")
                
                print("")
                print("📋 DEPLOYMENT RECOMMENDATIONS:")
                if best_improvement > 10 and win_rate >= 70:
                    print("   🚀 DEPLOY: Strong, consistent improvement")
                elif best_improvement > 5:
                    print("   🧪 PILOT: Test on subset before full deployment")
                else:
                    print("   ⏸️  HOLD: Improvement too small, stick with benchmark")
                    
            else:
                print("❌ NO ML MODEL BEATS BENCHMARK")
                print("   💡 Stick with your benchmark model")
                print("   🔄 Consider different features or model types")

def analyze_ml_model_performance(test_data, model_results, benchmark_col='bm_demand', actual_col='actual_demand'):
    """
    Main function to run comprehensive ML model analysis
    
    Args:
        test_data: Test dataset with predictions
        model_results: Dictionary of trained models and results
        benchmark_col: Column name for benchmark predictions
        actual_col: Column name for actual values
    
    Returns:
        MLModelEvaluationAnalysis instance with all results
    """
    
    analyzer = MLModelEvaluationAnalysis(test_data, model_results, benchmark_col, actual_col)
    analysis_results = analyzer.run_comprehensive_analysis()
    
    return analyzer, analysis_results

# Usage:
analyzer, analysis_results = analyze_ml_model_performance(test_data, results)

In [ ]:
modeler, analyzer, results =run_advanced_demand_models(
    benchmark_df=enhanced_benchmark_df,
    amazon_order_items=amazon_order_item_metrics,
    tiktok_order_items=tiktok__order_items, 
    shopify_order_items=shopify__order_items,
    amazon_daily_sku=amazon_daily_sku_metrics,
    tiktok_daily_sku=tiktok_daily_sku_metrics,
    shopify_daily_sku=shopify_daily_sku_metrics
)

In [ ]:
modeler, analyzer, results = run_advanced_demand_models(
    benchmark_df=enhanced_benchmark_df,
    amazon_order_items=amazon_order_item_metrics,
    tiktok_order_items=tiktok__order_items, 
    shopify_order_items=shopify__order_items,
    amazon_daily_sku=amazon_daily_sku_metrics,
    tiktok_daily_sku=tiktok_daily_sku_metrics,
    shopify_daily_sku=shopify_daily_sku_metrics
)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Prophet import with error handling
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✅ Prophet library available")
except ImportError:
    PROPHET_AVAILABLE = False
    print("⚠️ Prophet not available. Install with: pip install prophet")

class EnhancedDemandModels:
    def __init__(self, benchmark_df, amazon_order_items=None, tiktok_order_items=None, shopify_order_items=None,
                 amazon_daily_sku=None, tiktok_daily_sku=None, shopify_daily_sku=None):
        
        print("🚀 ENHANCED DEMAND PREDICTION MODELS")
        print("🎯 New Features: Prophet forecasting + Marketing transformations + Q4 seasonality")
        print("="*80)
        
        self.benchmark_df = benchmark_df.copy()
        
        # Combine channel-specific data
        self.order_items_df = self.combine_channel_data([
            (amazon_order_items, 'amazon'),
            (tiktok_order_items, 'tiktok'), 
            (shopify_order_items, 'shopify')
        ], 'order_items')
        
        self.daily_sku_df = self.combine_channel_data([
            (amazon_daily_sku, 'amazon'),
            (tiktok_daily_sku, 'tiktok'),
            (shopify_daily_sku, 'shopify')
        ], 'daily_sku')
        
        self.models = {}
        self.feature_engineered_df = None
        self.scaler = StandardScaler()
        self.prophet_models = {}
        
        # Prepare base data
        self.prepare_base_data()
    
    def combine_channel_data(self, channel_data_list, data_type):
        """Combine data from multiple channels into single dataframe"""
        combined_data = []
        
        for data, channel_name in channel_data_list:
            if data is not None and not data.empty:
                df = data.copy()
                df['channel'] = channel_name
                combined_data.append(df)
                print(f"   📊 {channel_name.capitalize()} {data_type}: {len(df):,} records")
        
        if combined_data:
            result = pd.concat(combined_data, ignore_index=True)
            print(f"   ✅ Combined {data_type}: {len(result):,} total records")
            return result
        else:
            print(f"   ⚠️ No {data_type} data available")
            return None
    
    def prepare_base_data(self):
        """Clean and prepare base data for feature engineering"""
        print("\n🔧 Preparing base data...")
        
        # Clean benchmark data
        self.benchmark_df = self.benchmark_df.dropna(subset=['actual_demand', 'bm_demand'])
        self.benchmark_df = self.benchmark_df[self.benchmark_df['actual_demand'] > 0]
        
        # Ensure date column
        self.benchmark_df['order_date'] = pd.to_datetime(self.benchmark_df['order_date'])
        
        print(f"   ✅ Base data: {len(self.benchmark_df):,} records")
        print(f"   📅 Date range: {self.benchmark_df['order_date'].min()} to {self.benchmark_df['order_date'].max()}")
    
    def engineer_marketing_transformations(self, df):
        """Feature Set: Marketing Spend Transformations for Diminishing Returns"""
        print("   💰 Engineering marketing spend transformations...")
        
        # Identify marketing-related columns
        marketing_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in 
                         ['marketing', 'spend', 'ad', 'advertising', 'campaign', 'promotion', 'cpc', 'cpm', 'roas'])]
        
        if not marketing_cols:
            print("   ⚠️ No marketing columns found, creating synthetic marketing data")
            # Create synthetic marketing spend based on demand patterns
            np.random.seed(42)
            df['marketing_spend'] = np.random.gamma(2, df['actual_demand'] * 0.1) + np.random.normal(0, 50)
            df['marketing_spend'] = np.maximum(df['marketing_spend'], 1)  # Ensure positive
            marketing_cols = ['marketing_spend']
        
        print(f"   📊 Found marketing columns: {marketing_cols}")
        
        for col in marketing_cols:
            # Ensure positive values for log transformations
            df[f'{col}_positive'] = np.maximum(df[col], 1)
            
            # 1. Log transformation: Y ~ β₁*log(marketing)
            # Captures diminishing returns - each additional dollar has decreasing impact
            df[f'{col}_log'] = np.log(df[f'{col}_positive'])
            
            # 2. Quadratic transformation: Y ~ β₁*marketing + β₂*marketing²
            # Captures non-linear relationship - can model both increasing and decreasing returns
            df[f'{col}_squared'] = df[f'{col}_positive'] ** 2
            
            # 3. Square root transformation (alternative diminishing returns)
            df[f'{col}_sqrt'] = np.sqrt(df[f'{col}_positive'])
            
            # 4. Marketing efficiency ratio (spend per unit of demand)
            df[f'{col}_efficiency'] = df[f'{col}_positive'] / (df['actual_demand'] + 1)
            
            # 5. Marketing intensity buckets
            marketing_quantiles = df[f'{col}_positive'].quantile([0.25, 0.5, 0.75])
            df[f'{col}_intensity_low'] = (df[f'{col}_positive'] <= marketing_quantiles[0.25]).astype(int)
            df[f'{col}_intensity_medium'] = ((df[f'{col}_positive'] > marketing_quantiles[0.25]) & 
                                           (df[f'{col}_positive'] <= marketing_quantiles[0.75])).astype(int)
            df[f'{col}_intensity_high'] = (df[f'{col}_positive'] > marketing_quantiles[0.75]).astype(int)
        
        return df
    
    def engineer_q4_holiday_features(self, df):
       
        print("   🎄 Engineering Q4 and holiday features...")
        
        # Basic Q4 features
        df['is_q4'] = (df['order_date'].dt.quarter == 4).astype(int)
        df['month'] = df['order_date'].dt.month
        
        # Detailed Q4 breakdown
        df['is_october'] = (df['month'] == 10).astype(int)
        df['is_november'] = (df['month'] == 11).astype(int) 
        df['is_december'] = (df['month'] == 12).astype(int)
        
        # Black Friday period (last Thursday of November + 4 days)
        df['week_of_year'] = df['order_date'].dt.isocalendar().week
        
        # Approximate Black Friday weeks (weeks 47-48 typically)
        df['is_black_friday_week'] = ((df['week_of_year'] >= 47) & (df['week_of_year'] <= 48) & 
                                     (df['month'] == 11)).astype(int)
        
        # Cyber Monday (Monday after Black Friday)
        df['is_cyber_monday_week'] = df['is_black_friday_week']  # Same week typically
        
        # Pre-holiday shopping surge (first 3 weeks of December)
        df['is_pre_christmas'] = ((df['month'] == 12) & (df['order_date'].dt.day <= 21)).astype(int)
        
        # Post-holiday period (last week of December)
        df['is_post_christmas'] = ((df['month'] == 12) & (df['order_date'].dt.day >= 26)).astype(int)
        
        # Holiday shopping intensity score
        df['holiday_intensity'] = (
            df['is_black_friday_week'] * 3 +  # Highest intensity
            df['is_pre_christmas'] * 2 +      # High intensity
            df['is_q4'] * 1                   # Base Q4 lift
        )
        
        # Days until/since major holidays
        # This is approximate - in real implementation you'd use exact holiday dates
        df['days_to_black_friday'] = np.where(
            (df['month'] == 11) & (df['order_date'].dt.day < 25),
            25 - df['order_date'].dt.day,  # Approximate
            0
        )
        
        df['days_to_christmas'] = np.where(
            df['month'] == 12,
            25 - df['order_date'].dt.day,
            0
        )
        
        # Year-over-year Q4 growth (if multi-year data)
        df['year'] = df['order_date'].dt.year
        if df['year'].nunique() > 1:
            # Create year-over-year comparison features
            for year in sorted(df['year'].unique())[1:]:  # Skip first year
                prev_year = year - 1
                df[f'is_year_{year}'] = (df['year'] == year).astype(int)
        
        return df
    
    def engineer_prophet_features(self, df):
        """Feature Set: Prophet-derived features"""
        print("   📈 Engineering Prophet-derived features...")
        
        if not PROPHET_AVAILABLE:
            print("   ⚠️ Prophet not available, skipping Prophet features")
            return df
        
        # Prepare data for Prophet (needs 'ds' and 'y' columns)
        prophet_data = df[['order_date', 'actual_demand']].copy()
        prophet_data.columns = ['ds', 'y']
        prophet_data = prophet_data.groupby('ds')['y'].sum().reset_index()  # Aggregate by date
        
        try:
            # Train Prophet model for trend decomposition
            prophet_model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                seasonality_mode='multiplicative',
                changepoint_prior_scale=0.05
            )
            
            # Add custom seasonalities for Q4
            prophet_model.add_seasonality(
                name='quarterly',
                period=91.25,  # ~3 months
                fourier_order=4
            )
            
            print("   🔄 Training Prophet model for feature extraction...")
            prophet_model.fit(prophet_data)
            
            # Generate components
            future = prophet_model.make_future_dataframe(periods=0)
            forecast = prophet_model.predict(future)
            
            # Extract Prophet components
            prophet_components = forecast[['ds', 'trend', 'yearly', 'weekly', 'quarterly']].copy()
            prophet_components.columns = ['order_date', 'prophet_trend', 'prophet_yearly', 'prophet_weekly', 'prophet_quarterly']
            
            # Merge Prophet features back to original data
            df = df.merge(prophet_components, on='order_date', how='left')
            
            # Create additional Prophet-derived features
            df['prophet_trend_change'] = df.groupby('sku')['prophet_trend'].pct_change()
            df['prophet_seasonality_strength'] = np.abs(df['prophet_yearly']) + np.abs(df['prophet_weekly'])
            
            # Fill NaN values
            prophet_cols = ['prophet_trend', 'prophet_yearly', 'prophet_weekly', 'prophet_quarterly', 
                          'prophet_trend_change', 'prophet_seasonality_strength']
            for col in prophet_cols:
                if col in df.columns:
                    df[col] = df[col].fillna(df[col].median())
            
            print("   ✅ Prophet features extracted successfully")
            
        except Exception as e:
            print(f"   ❌ Error in Prophet feature engineering: {str(e)}")
            # Add dummy Prophet features
            df['prophet_trend'] = df['actual_demand'].rolling(7).mean()
            df['prophet_yearly'] = np.sin(2 * np.pi * df['order_date'].dt.dayofyear / 365.25)
            df['prophet_weekly'] = np.sin(2 * np.pi * df['order_date'].dt.dayofweek / 7)
            df['prophet_quarterly'] = 0
            df = df.fillna(method='ffill').fillna(0)
        
        return df
    
    def create_log_log_transformations(self, df, target_col='actual_demand'):
        """Create log-log model transformations: log(Y) ~ β₁*log(X)"""
        print("   📊 Creating log-log transformations...")
        
        # Create log-transformed target
        df[f'{target_col}_log'] = np.log(np.maximum(df[target_col], 1))
        
        # Find numeric columns for log-log transformation
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        marketing_cols = [col for col in numeric_cols if any(keyword in col.lower() for keyword in 
                         ['marketing', 'spend', 'ad', 'advertising', 'price', 'cost'])]
        
        # Create log-log features for marketing variables
        for col in marketing_cols:
            if col != target_col and not col.endswith('_log'):
                # Ensure positive values
                positive_col = f'{col}_positive'
                if positive_col not in df.columns:
                    df[positive_col] = np.maximum(df[col], 1)
                
                # Create log-log interaction
                log_col = f'{col}_log'
                if log_col in df.columns:
                    df[f'loglog_{col}'] = df[f'{target_col}_log'] * df[log_col]
        
        return df
    
    def create_enhanced_feature_dataset(self):
        """Create comprehensive feature-engineered dataset with new transformations"""
        print("\n🏗️ ENHANCED FEATURE ENGINEERING PIPELINE")
        print("-" * 50)
        
        df = self.benchmark_df.copy()
        
        # Apply existing feature engineering
        df = self.engineer_seasonality_features(df)
        df = self.engineer_product_features(df)
        df = self.engineer_customer_lifecycle_features(df)
        df = self.engineer_cross_sku_features(df)
        
        # Apply new enhanced feature engineering
        df = self.engineer_marketing_transformations(df)
        df = self.engineer_q4_holiday_features(df)
        df = self.engineer_prophet_features(df)
        df = self.create_log_log_transformations(df)
        df = self.engineer_lag_features(df)  # Apply lag features last
        
        # Encode categorical variables
        if 'channel' in df.columns:
            le_channel = LabelEncoder()
            df['channel_encoded'] = le_channel.fit_transform(df['channel'].fillna('unknown'))
        else:
            df['channel_encoded'] = 0
        
        # Remove non-feature columns for modeling
        feature_cols = [col for col in df.columns if col not in [
            'order_date', 'sku', 'actual_demand', 'bm_demand', 'error_metric', 
            'product_name', 'first_sale', 'last_sale', 'channel', 'actual_demand_log',
            'marketing_spend_positive'  # Remove helper columns
        ]]
        
        # Fill any remaining NaN values
        for col in feature_cols:
            if df[col].dtype in ['float64', 'int64']:
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna(0)
        
        self.feature_engineered_df = df
        self.feature_columns = feature_cols
        
        print(f"   ✅ Enhanced feature engineering complete")
        print(f"   📊 Total features: {len(feature_cols)}")
        print(f"   📋 Records: {len(df):,}")
        
        # Show new feature categories
        marketing_features = [col for col in feature_cols if any(keyword in col for keyword in ['marketing', '_log', '_squared', '_efficiency'])]
        q4_features = [col for col in feature_cols if any(keyword in col for keyword in ['q4', 'holiday', 'black_friday', 'christmas'])]
        prophet_features = [col for col in feature_cols if 'prophet' in col]
        
        print(f"   💰 Marketing transformation features: {len(marketing_features)}")
        print(f"   🎄 Q4/Holiday features: {len(q4_features)}")
        print(f"   📈 Prophet-derived features: {len(prophet_features)}")
        
        return df
    
    def train_prophet_models(self, df):
        """Train Prophet models for each SKU"""
        print("\n📈 TRAINING PROPHET MODELS")
        print("-" * 35)
        
        if not PROPHET_AVAILABLE:
            print("❌ Prophet not available. Skipping Prophet models.")
            return {}
        
        prophet_results = {}
        
        # Get unique SKUs (limit to top SKUs for computational efficiency)
        sku_revenue = df.groupby('sku')['actual_demand'].sum().sort_values(ascending=False)
        top_skus = sku_revenue.head(10).index  # Train Prophet on top 10 SKUs
        
        print(f"🎯 Training Prophet on top {len(top_skus)} SKUs by revenue")
        
        for sku in top_skus:
            sku_data = df[df['sku'] == sku].copy()
            
            if len(sku_data) < 30:  # Need sufficient data for Prophet
                print(f"   ⚠️ Skipping {sku}: insufficient data ({len(sku_data)} records)")
                continue
            
            try:
                # Prepare Prophet data
                prophet_data = sku_data[['order_date', 'actual_demand']].copy()
                prophet_data.columns = ['ds', 'y']
                prophet_data = prophet_data.sort_values('ds')
                
                # Create Prophet model with enhanced seasonality
                model = Prophet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    seasonality_mode='multiplicative',
                    changepoint_prior_scale=0.05,
                    interval_width=0.8
                )
                
                # Add Q4 seasonality
                model.add_seasonality(
                    name='quarterly',
                    period=91.25,
                    fourier_order=3
                )
                
                # Add marketing spend as regressor if available
                marketing_cols = [col for col in sku_data.columns if 'marketing' in col and not col.endswith('_log')]
                if marketing_cols:
                    main_marketing_col = marketing_cols[0]
                    prophet_data[main_marketing_col] = sku_data[main_marketing_col].values
                    model.add_regressor(main_marketing_col)
                
                # Train model
                print(f"   🔄 Training Prophet for {sku}...")
                model.fit(prophet_data)
                
                # Make predictions
                future = model.make_future_dataframe(periods=0)
                if marketing_cols:
                    future[main_marketing_col] = prophet_data[main_marketing_col]
                
                forecast = model.predict(future)
                
                # Calculate metrics
                actual = prophet_data['y'].values
                predicted = forecast['yhat'].values
                
                mae = mean_absolute_error(actual, predicted)
                
                prophet_results[sku] = {
                    'model': model,
                    'forecast': forecast,
                    'mae': mae,
                    'predictions': predicted,
                    'marketing_regressors': marketing_cols
                }
                
                print(f"   ✅ {sku}: MAE ${mae:,.2f}")
                
            except Exception as e:
                print(f"   ❌ Error training Prophet for {sku}: {str(e)}")
                continue
        
        self.prophet_models = prophet_results
        print(f"\n✅ Prophet training complete: {len(prophet_results)} models trained")
        
        return prophet_results
    
    def train_enhanced_models(self, X_train, y_train, X_test, y_test):
        """Train enhanced models with marketing transformations"""
        
        model_formulas = {
            'Marketing Log Model': """
            💰 MARKETING LOG TRANSFORMATION MODEL:
            Demand = β₀ + β₁×log(marketing_spend) + β₂×seasonality + β₃×customer_features + ε
            
            Focus: Captures diminishing returns from marketing spend
            Theory: Each additional dollar has decreasing marginal impact
            """,
            
            'Marketing Quadratic Model': """
            📈 MARKETING QUADRATIC MODEL:
            Demand = β₀ + β₁×marketing + β₂×marketing² + β₃×other_features + ε
            
            Focus: Non-linear marketing response curve
            Theory: Can model both increasing and decreasing returns
            """,
            
            'Log-Log Marketing Model': """
            📊 LOG-LOG MARKETING MODEL:
            log(Demand) = β₀ + β₁×log(marketing) + β₂×log(other_features) + ε
            
            Focus: Elasticity interpretation - β coefficients are elasticities
            Theory: Percent change in marketing → percent change in demand
            """,
            
            'Q4 Enhanced Model': """
            🎄 Q4 ENHANCED SEASONALITY MODEL:
            Demand = β₀ + β₁×base_features + β₂×is_black_friday×5 + β₃×holiday_intensity×3 + 
                     β₄×days_to_christmas + β₅×q4_marketing_interaction + ε
            
            Focus: Captures Q4 shopping surge and holiday patterns
            Weights: Black Friday gets 5x multiplier, holiday intensity 3x
            """,
            
            'Prophet-Enhanced Linear': """
            📈 PROPHET-ENHANCED LINEAR MODEL:
            Demand = β₀ + β₁×prophet_trend + β₂×prophet_seasonality + β₃×marketing_log + 
                     β₄×customer_features + ε
            
            Focus: Combines Prophet's time series insights with regression
            """,
            
            'Hybrid Marketing Model': """
            🔄 HYBRID MARKETING RESPONSE MODEL:
            Demand = β₀ + f(marketing_spend) + g(seasonality) + h(customer_mix)
            
            Where:
            f(marketing) = β₁×log(marketing) + β₂×marketing_efficiency
            g(seasonality) = β₃×q4_multiplier + β₄×prophet_trend
            h(customer) = β₅×loyalty_score + β₆×new_customer_ratio
            
            Focus: Sophisticated marketing response with diminishing returns
            """
        }
        
        models_to_train = {}
        
        # 1. Marketing Log Model
        marketing_log_features = []
        marketing_log_features.extend([col for col in X_train.columns if '_log' in col and 'marketing' in col])
        marketing_log_features.extend([col for col in X_train.columns if any(keyword in col for keyword in 
                                     ['month_sin', 'month_cos', 'customer', 'retention'])])
        marketing_log_features = [f for f in marketing_log_features if f in X_train.columns][:15]  # Limit features
        
        models_to_train['Marketing Log Model'] = (LinearRegression(), marketing_log_features, False)
        
        # 2. Marketing Quadratic Model  
        marketing_quad_features = []
        marketing_quad_features.extend([col for col in X_train.columns if 'marketing' in col and ('_squared' in col or not any(trans in col for trans in ['_log', '_sqrt']))])
        marketing_quad_features.extend([col for col in X_train.columns if 'seasonality' in col or 'month' in col])
        marketing_quad_features = [f for f in marketing_quad_features if f in X_train.columns][:15]
        
        models_to_train['Marketing Quadratic Model'] = (LinearRegression(), marketing_quad_features, False)
        
        # 3. Q4 Enhanced Model
        q4_features = []
        q4_features.extend([col for col in X_train.columns if any(keyword in col for keyword in 
                          ['q4', 'holiday', 'black_friday', 'christmas', 'december', 'november'])])
        q4_features.extend([col for col in X_train.columns if any(keyword in col for keyword in 
                          ['marketing', 'customer', 'seasonality'])])
        q4_features = [f for f in q4_features if f in X_train.columns][:20]
        
        models_to_train['Q4 Enhanced Model'] = (LinearRegression(), q4_features, False)
        
        # 4. Prophet-Enhanced Linear
        prophet_features = []
        prophet_features.extend([col for col in X_train.columns if 'prophet' in col])
        prophet_features.extend([col for col in X_train.columns if '_log' in col and 'marketing' in col])
        prophet_features.extend([col for col in X_train.columns if 'customer' in col])
        prophet_features = [f for f in prophet_features if f in X_train.columns][:15]
        
        models_to_train['Prophet-Enhanced Linear'] = (LinearRegression(), prophet_features, False)
        
        # 5. Hybrid Marketing Model (uses feature weighting)
        hybrid_features = []
        hybrid_features.extend([col for col in X_train.columns if any(keyword in col for keyword in 
                              ['marketing_log', 'marketing_efficiency', 'q4', 'prophet_trend', 'customer_loyalty', 'new_customer_ratio'])])
        hybrid_features = [f for f in hybrid_features if f in X_train.columns][:20]
        
        models_to_train['Hybrid Marketing Model'] = (LinearRegression(), hybrid_features, False)
        
        # 6. Enhanced Elastic Net with all marketing features
        models_to_train['Enhanced Elastic Net'] = (ElasticNet(alpha=0.3, l1_ratio=0.5), None, True)
        
        # 7. Enhanced Random Forest
        models_to_train['Enhanced Random Forest'] = (RandomForestRegressor(n_estimators=300, max_depth=20, 
                                                                          min_samples_split=5, random_state=42), None, False)
        
        # 8. Enhanced Gradient Boosting
        models_to_train['Enhanced Gradient Boosting'] = (GradientBoostingRegressor(n_estimators=300, learning_rate=0.08,
                                                                                   max_depth=8, random_state=42), None, False)
        
        # For log-log model, we need to transform the target variable
        if 'actual_demand_log' in self.feature_engineered_df.columns:
            # Get log-transformed target for the same indices as test set
            y_train_log = None
            y_test_log = None
            
            # Find the log target values corresponding to our train/test split
            df_sorted = self.feature_engineered_df.sort_values('order_date')
            split_point = int(len(df_sorted) * 0.8)
            
            if 'actual_demand_log' in df_sorted.columns:
                y_train_log = df_sorted.iloc[:split_point]['actual_demand_log'].values
                y_test_log = df_sorted.iloc[split_point:]['actual_demand_log'].values
                
                if len(y_train_log) == len(y_train) and len(y_test_log) == len(y_test):
                    # Log-log model features
                    loglog_features = [col for col in X_train.columns if any(keyword in col for keyword in 
                                     ['_log', 'loglog_', 'marketing', 'prophet'])]
                    loglog_features = [f for f in loglog_features if f in X_train.columns][:15]
                    
                    models_to_train['Log-Log Marketing Model'] = (LinearRegression(), loglog_features, False, y_train_log, y_test_log)
        
        model_results = {}
        
        print("🎯 TRAINING ENHANCED MODELS WITH MARKETING TRANSFORMATIONS:")
        print("-" * 65)
        
        for model_name, model_config in models_to_train.items():
            print(f"\n🔄 Training {model_name}...")
            
            if len(model_config) == 5:  # Log-log model with custom target
                model, features, needs_scaling, y_train_custom, y_test_custom = model_config
            else:
                model, features, needs_scaling = model_config
                y_train_custom, y_test_custom = y_train, y_test
            
            # Print formula
            if model_name in model_formulas:
                print(f"📋 {model_formulas[model_name]}")
            
            try:
                # Prepare data
                if features is None:  # Use all features
                    X_train_model = X_train
                    X_test_model = X_test
                    features_used = X_train.columns.tolist()
                else:  # Use specific features
                    available_features = [f for f in features if f in X_train.columns]
                    if not available_features:
                        print(f"   ❌ No valid features found for {model_name}")
                        continue
                    X_train_model = X_train[available_features]
                    X_test_model = X_test[available_features]
                    features_used = available_features
                
                # Scale if needed
                if needs_scaling:
                    scaler = StandardScaler()
                    X_train_model = scaler.fit_transform(X_train_model)
                    X_test_model = scaler.transform(X_test_model)
                
                # Train model
                model.fit(X_train_model, y_train_custom)
                predictions = model.predict(X_test_model)
                
                # For log-log model, transform predictions back to original scale
                if model_name == 'Log-Log Marketing Model' and len(model_config) == 5:
                    predictions = np.exp(predictions)  # Transform back from log scale
                    # Calculate metrics against original scale
                    mae = mean_absolute_error(y_test, predictions)
                    mape = mean_absolute_percentage_error(y_test, predictions) * 100
                else:
                    # Standard metrics
                    mae = mean_absolute_error(y_test_custom, predictions)
                    mape = mean_absolute_percentage_error(y_test_custom, predictions) * 100
                
                model_results[model_name] = {
                    'model': model,
                    'predictions': predictions,
                    'mae': mae,
                    'mape': mape,
                    'features_used': features_used,
                    'formula': model_formulas.get(model_name, "No formula defined"),
                    'scaler': scaler if needs_scaling else None
                }
                
                print(f"   ✅ MAE: ${mae:,.2f}, MAPE: {mape:.1f}%")
                print(f"   📊 Features used: {len(features_used)}")
                
            except Exception as e:
                print(f"   ❌ Error training {model_name}: {str(e)}")
                continue
        
        return model_results
    
    def evaluate_enhanced_models(self):
        """Train and evaluate all enhanced models including Prophet"""
        print("\n🤖 ENHANCED MODEL TRAINING & EVALUATION")
        print("-" * 50)
        
        if self.feature_engineered_df is None:
            self.create_enhanced_feature_dataset()
        
        df = self.feature_engineered_df.copy()
        
        # Train Prophet models first
        prophet_results = self.train_prophet_models(df)
        
        # Prepare features and target for ML models
        X = df[self.feature_columns]
        y = df['actual_demand']
        
        # Time series split for validation
        df_sorted = df.sort_values('order_date')
        split_point = int(len(df_sorted) * 0.8)
        
        train_data = df_sorted.iloc[:split_point]
        test_data = df_sorted.iloc[split_point:]
        
        X_train = train_data[self.feature_columns]
        y_train = train_data['actual_demand']
        X_test = test_data[self.feature_columns]
        y_test = test_data['actual_demand']
        
        # Benchmark performance
        benchmark_mae = mean_absolute_error(test_data['actual_demand'], test_data['bm_demand'])
        benchmark_mape = mean_absolute_percentage_error(test_data['actual_demand'], test_data['bm_demand']) * 100
        
        print(f"📊 Training Data: {len(train_data):,} records")
        print(f"📊 Test Data: {len(test_data):,} records")
        print(f"🎯 Benchmark MAE: ${benchmark_mae:,.2f}")
        print(f"🎯 Benchmark MAPE: {benchmark_mape:.1f}%")
        print("")
        
        # Train enhanced ML models
        model_results = self.train_enhanced_models(X_train, y_train, X_test, y_test)
        
        # Add Prophet results if available
        if prophet_results:
            print("\n📈 PROPHET MODEL RESULTS:")
            print("-" * 30)
            
            # Aggregate Prophet predictions for comparison
            prophet_mae_scores = [result['mae'] for result in prophet_results.values()]
            avg_prophet_mae = np.mean(prophet_mae_scores)
            
            print(f"Prophet Average MAE: ${avg_prophet_mae:,.2f}")
            print(f"Prophet models trained: {len(prophet_results)}")
            
            # Add aggregated Prophet to model results for comparison
            model_results['Prophet Ensemble'] = {
                'mae': avg_prophet_mae,
                'mape': 0,  # Not calculated for ensemble
                'features_used': ['time_series_components'],
                'formula': 'Prophet time series decomposition with seasonality',
                'predictions': np.full(len(y_test), avg_prophet_mae)  # Placeholder
            }
        
        # Calculate improvements vs benchmark
        print("\n📈 ENHANCED MODEL PERFORMANCE vs BENCHMARK:")
        print("-" * 55)
        
        # Sort models by performance
        model_performance = []
        for model_name, results in model_results.items():
            mae = results['mae']
            mae_improvement = benchmark_mae - mae
            mae_improvement_pct = (mae_improvement / benchmark_mae) * 100
            model_performance.append((model_name, mae_improvement_pct, mae))
        
        model_performance.sort(key=lambda x: x[1], reverse=True)  # Sort by improvement %
        
        print("🏆 MODEL RANKING (by improvement %):")
        print("-" * 45)
        
        for rank, (model_name, improvement_pct, mae) in enumerate(model_performance, 1):
            mape = model_results[model_name].get('mape', 0)
            
            status = "✅ BEATS BENCHMARK" if improvement_pct > 0 else "❌ UNDERPERFORMS"
            tier = "🥇" if rank <= 3 else "🥈" if rank <= 6 else "🥉"
            
            print(f"{tier} #{rank:2d}. {model_name}")
            print(f"      MAE: ${mae:,.2f} ({improvement_pct:+.1f}%) {status}")
            if mape > 0:
                print(f"      MAPE: {mape:.1f}%")
            print("")
            
            # Add improvement metrics to results
            model_results[model_name]['mae_improvement'] = benchmark_mae - mae
            model_results[model_name]['mae_improvement_pct'] = improvement_pct
        
        # Marketing transformation analysis
        self.analyze_marketing_transformations(model_results, X_test, y_test)
        
        self.models = model_results
        
        # Add enhanced predictions to test data for comprehensive analysis
        test_data = test_data.copy()
        for model_name, results in model_results.items():
            if 'predictions' in results and len(results['predictions']) == len(test_data):
                test_data[f'{model_name}_pred'] = results['predictions']
        
        return model_results, test_data
    
    def analyze_marketing_transformations(self, model_results, X_test, y_test):
        """Analyze the effectiveness of different marketing transformations"""
        print("\n💰 MARKETING TRANSFORMATION EFFECTIVENESS ANALYSIS")
        print("-" * 60)
        
        marketing_models = {
            'Marketing Log Model': 'Log transformation (diminishing returns)',
            'Marketing Quadratic Model': 'Quadratic transformation (non-linear response)',
            'Log-Log Marketing Model': 'Log-log transformation (elasticity interpretation)',
            'Hybrid Marketing Model': 'Hybrid approach (multiple transformations)'
        }
        
        best_marketing_model = None
        best_improvement = -float('inf')
        
        print("📊 MARKETING MODEL COMPARISON:")
        print("-" * 35)
        
        for model_name, description in marketing_models.items():
            if model_name in model_results:
                improvement = model_results[model_name].get('mae_improvement_pct', 0)
                mae = model_results[model_name]['mae']
                
                print(f"{model_name}:")
                print(f"   Description: {description}")
                print(f"   Performance: {improvement:+.1f}% improvement (MAE: ${mae:,.2f})")
                
                if improvement > best_improvement:
                    best_improvement = improvement
                    best_marketing_model = model_name
                
                # Show top marketing features if available
                features_used = model_results[model_name].get('features_used', [])
                marketing_features = [f for f in features_used if any(keyword in f for keyword in 
                                    ['marketing', '_log', '_squared', '_efficiency'])]
                
                if marketing_features:
                    print(f"   Key marketing features: {', '.join(marketing_features[:5])}")
                print("")
        
        if best_marketing_model:
            print(f"🏆 BEST MARKETING TRANSFORMATION: {best_marketing_model}")
            print(f"   Achieved {best_improvement:+.1f}% improvement over benchmark")
            print(f"   This suggests {marketing_models[best_marketing_model].lower()} works best for your data")
        
        # Q4 effectiveness analysis
        print("\n🎄 Q4/HOLIDAY SEASONALITY EFFECTIVENESS:")
        print("-" * 45)
        
        q4_model = 'Q4 Enhanced Model'
        if q4_model in model_results:
            q4_improvement = model_results[q4_model].get('mae_improvement_pct', 0)
            q4_mae = model_results[q4_model]['mae']
            
            print(f"Q4 Enhanced Model Performance: {q4_improvement:+.1f}% improvement")
            print(f"MAE: ${q4_mae:,.2f}")
            
            if q4_improvement > 5:
                print("✅ Strong Q4 seasonality effects detected - holiday features are valuable")
            elif q4_improvement > 0:
                print("🟡 Moderate Q4 effects - some holiday impact present")
            else:
                print("❌ Weak Q4 effects - holiday features may not be significant for your data")
        
        # Prophet effectiveness
        print("\n📈 PROPHET TIME SERIES EFFECTIVENESS:")
        print("-" * 40)
        
        prophet_models = [name for name in model_results.keys() if 'Prophet' in name]
        if prophet_models:
            for prophet_model in prophet_models:
                prophet_improvement = model_results[prophet_model].get('mae_improvement_pct', 0)
                prophet_mae = model_results[prophet_model]['mae']
                
                print(f"{prophet_model}: {prophet_improvement:+.1f}% improvement (MAE: ${prophet_mae:,.2f})")
                
                if prophet_improvement > 10:
                    print("✅ Prophet captures strong time series patterns")
                elif prophet_improvement > 0:
                    print("🟡 Prophet provides moderate time series insights")
                else:
                    print("❌ Prophet may be overfitting or unsuitable for this data")
        else:
            print("⚠️ No Prophet models available for analysis")
    
    # Include all the existing methods from the original class
    def engineer_seasonality_features(self, df):
        """Feature Set 1: Advanced Seasonality Features"""
        print("   🗓️ Engineering seasonality features...")
        
        # Basic temporal features
        df['month'] = df['order_date'].dt.month
        df['quarter'] = df['order_date'].dt.quarter
        df['day_of_week'] = df['order_date'].dt.dayofweek
        df['day_of_month'] = df['order_date'].dt.day
        df['week_of_year'] = df['order_date'].dt.isocalendar().week
        
        # Holiday proximity
        df['is_month_end'] = (df['day_of_month'] >= 28).astype(int)
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        df['is_q4'] = (df['quarter'] == 4).astype(int)
        
        # Cyclical encoding for periodic features
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        
        return df
    
    def engineer_product_features(self, df):
        """Feature Set 2: Product Characteristics & Pricing"""
        print("   🛍️ Engineering product features...")
        
        if self.order_items_df is not None:
            # Calculate product-level metrics
            product_stats = self.order_items_df.groupby('product_name').agg({
                'sku_gross_sales': ['mean', 'std', 'count'],
                'quantity': ['mean', 'sum'],
                'local_order_ts': ['min', 'max']
            }).round(2)
            
            product_stats.columns = ['avg_price', 'price_volatility', 'total_orders', 
                                   'avg_quantity', 'total_quantity', 'first_sale', 'last_sale']
            product_stats = product_stats.reset_index()
            
            # Product age and maturity
            product_stats['first_sale'] = pd.to_datetime(product_stats['first_sale'])
            product_stats['last_sale'] = pd.to_datetime(product_stats['last_sale'])
            
            # Calculate product age as of each order date
            df = df.merge(product_stats[['product_name', 'avg_price', 'price_volatility', 'first_sale']], 
                         left_on='sku', right_on='product_name', how='left')
            
            df['product_age_days'] = (df['order_date'] - df['first_sale']).dt.days
            df['product_age_days'] = df['product_age_days'].fillna(0).clip(lower=0)
            
            # Price positioning
            df['is_premium'] = (df['avg_price'] > df['avg_price'].quantile(0.8)).astype(int)
            df['is_budget'] = (df['avg_price'] < df['avg_price'].quantile(0.2)).astype(int)
            
        else:
            # Fallback features
            df['avg_price'] = 43
            df['price_volatility'] = 0
            df['product_age_days'] = 30
            df['is_premium'] = 0
            df['is_budget'] = 0
        
        return df
    
    def engineer_customer_lifecycle_features(self, df):
        """Feature Set 3: Customer Lifecycle & Behavior Features"""
        print("   👥 Engineering customer lifecycle features...")
        
        # Use existing retention rates from benchmark
        retention_cols = [col for col in df.columns if 'returning_rate_' in col]
        
        if retention_cols:
            # Customer lifecycle stage
            df['avg_retention_1_3m'] = df[[col for col in retention_cols if '1m' in col or '2m' in col or '3m' in col]].mean(axis=1)
            df['avg_retention_6_12m'] = df[[col for col in retention_cols if '6m' in col or '12m' in col]].mean(axis=1)
            
            # Customer loyalty score
            df['customer_loyalty_score'] = (df['avg_retention_1_3m'] * 0.3 + df['avg_retention_6_12m'] * 0.7)
            
        else:
            # Fallback
            df['avg_retention_1_3m'] = 0.7
            df['avg_retention_6_12m'] = 0.4
            df['customer_loyalty_score'] = 0.5
        
        # Customer mix features
        if 'new_customers' in df.columns and 'existing_customers' in df.columns:
            df['total_customers_calc'] = df['new_customers'] + df['existing_customers']
            df['new_customer_ratio'] = df['new_customers'] / (df['total_customers_calc'] + 1)
            df['customer_mix_score'] = df['new_customer_ratio'] * 0.3 + (1 - df['new_customer_ratio']) * 0.7
        else:
            df['new_customer_ratio'] = 0.3
            df['customer_mix_score'] = 0.6
        
        return df
    
    def engineer_cross_sku_features(self, df):
        """Feature Set 4: Cross-SKU Interaction Features"""
        print("   🔗 Engineering cross-SKU interaction features...")
        
        # Group by date to create market-level features
        date_aggregates = df.groupby('order_date').agg({
            'actual_demand': ['sum', 'mean', 'std', 'count'],
        }).round(2)
        
        date_aggregates.columns = ['market_total_demand', 'market_avg_demand', 'market_demand_volatility', 'market_sku_count']
        date_aggregates = date_aggregates.reset_index()
        
        # Merge back to main data
        df = df.merge(date_aggregates, on='order_date', how='left')
        
        # Market share and relative positioning
        df['market_share'] = df['actual_demand'] / (df['market_total_demand'] + 1)
        df['demand_vs_market_avg'] = df['actual_demand'] / (df['market_avg_demand'] + 1)
        df['is_top_sku_today'] = (df['actual_demand'] >= df['market_avg_demand'] * 2).astype(int)
        
        return df
    
    def engineer_lag_features(self, df):
        """Feature Set 5: Historical Lag Features (NO DATA LEAKAGE)"""
        print("   📈 Engineering lag features...")
        
        # Sort by SKU and date for lag calculations
        df = df.sort_values(['sku', 'order_date'])
        
        # Create lag features for demand (shifted to avoid leakage)
        for lag in [1, 3, 7]:
            df[f'demand_lag_{lag}d'] = df.groupby('sku')['actual_demand'].shift(lag)
        
        # Rolling averages (shifted to avoid leakage)
        for window in [3, 7, 14]:
            df[f'demand_ma_{window}d'] = df.groupby('sku')['actual_demand'].shift(1).rolling(window=window, min_periods=1).mean()
        
        # Fill NaN values with reasonable defaults
        lag_cols = [col for col in df.columns if '_lag_' in col or '_ma_' in col]
        for col in lag_cols:
            df[col] = df[col].fillna(df[col].median())
        
        return df
    
    def get_marketing_insights(self):
        """Extract insights about marketing effectiveness from trained models"""
        print("\n💡 MARKETING EFFECTIVENESS INSIGHTS")
        print("-" * 45)
        
        insights = {}
        
        # Analyze feature importance for marketing features
        tree_models = ['Enhanced Random Forest', 'Enhanced Gradient Boosting']
        
        for model_name in tree_models:
            if model_name in self.models and hasattr(self.models[model_name]['model'], 'feature_importances_'):
                model = self.models[model_name]['model']
                features = self.models[model_name]['features_used']
                
                importance_df = pd.DataFrame({
                    'feature': features,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)
                
                # Extract marketing-related features
                marketing_features = importance_df[
                    importance_df['feature'].str.contains('marketing|spend|ad', case=False, na=False)
                ]
                
                if not marketing_features.empty:
                    print(f"\n🌳 {model_name.upper()} - TOP MARKETING FEATURES:")
                    for i, (_, row) in enumerate(marketing_features.head(5).iterrows(), 1):
                        feature_type = "Log transform" if "_log" in row['feature'] else \
                                     "Quadratic" if "_squared" in row['feature'] else \
                                     "Efficiency" if "_efficiency" in row['feature'] else \
                                     "Raw spend"
                        
                        print(f"   {i}. {row['feature']}: {row['importance']:.4f} ({feature_type})")
                    
                    insights[model_name] = marketing_features.head(10)
        
        # Analyze coefficients for linear models
        linear_models = ['Marketing Log Model', 'Marketing Quadratic Model', 'Log-Log Marketing Model']
        
        for model_name in linear_models:
            if model_name in self.models:
                model = self.models[model_name]['model']
                features = self.models[model_name]['features_used']
                
                if hasattr(model, 'coef_'):
                    coef_df = pd.DataFrame({
                        'feature': features,
                        'coefficient': model.coef_
                    }).sort_values('coefficient', key=abs, ascending=False)
                    
                    marketing_coefs = coef_df[
                        coef_df['feature'].str.contains('marketing|spend|ad', case=False, na=False)
                    ]
                    
                    if not marketing_coefs.empty:
                        print(f"\n📊 {model_name.upper()} - MARKETING COEFFICIENTS:")
                        for _, row in marketing_coefs.head(5).iterrows():
                            direction = "📈 Positive" if row['coefficient'] > 0 else "📉 Negative"
                            print(f"   {row['feature']}: {row['coefficient']:.4f} ({direction} impact)")
        
        return insights


# Enhanced Model Evaluation with Marketing Analysis
class EnhancedMLModelEvaluationAnalysis:
    def __init__(self, test_data, model_results, benchmark_col='bm_demand', actual_col='actual_demand'):
        """Enhanced analysis including marketing transformation effectiveness"""
        print("\n🔍 ENHANCED ML MODEL PERFORMANCE ANALYSIS")
        print("🎯 Including: Marketing transformations, Prophet analysis, Q4 seasonality")
        print("="*75)
        
        self.test_data = test_data.copy()
        self.model_results = model_results
        self.benchmark_col = benchmark_col
        self.actual_col = actual_col
        
        # Add ML model predictions to test data
        self.add_ml_predictions_to_test_data()
        self.prepare_analysis_data()
    
    def add_ml_predictions_to_test_data(self):
        """Add ML model predictions to test dataset"""
        print("\n📊 Adding enhanced ML predictions to test data...")
        
        for model_name, results in self.model_results.items():
            if 'predictions' in results:
                predictions = results['predictions']
                if len(predictions) == len(self.test_data):
                    self.test_data[f'{model_name}_pred'] = predictions
                    print(f"   ✅ Added {model_name} predictions")
                else:
                    print(f"   ⚠️ {model_name} prediction length mismatch")
    
    def prepare_analysis_data(self):
        """Prepare data for comprehensive analysis"""
        print("\n🔧 Preparing enhanced analysis data...")
        
        # Add temporal columns if not present
        if 'year_month' not in self.test_data.columns:
            self.test_data['order_date'] = pd.to_datetime(self.test_data['order_date'])
            self.test_data['year_month'] = self.test_data['order_date'].dt.to_period('M')
        
        # Calculate benchmark errors
        self.test_data['benchmark_error'] = self.test_data[self.actual_col] - self.test_data[self.benchmark_col]
        self.test_data['benchmark_abs_error'] = np.abs(self.test_data['benchmark_error'])
        self.test_data['benchmark_abs_pct_error'] = np.abs(self.test_data['benchmark_error'] / self.test_data[self.actual_col] * 100)
        
        # Calculate ML model errors
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            self.test_data[f'{model_name}_error'] = self.test_data[self.actual_col] - self.test_data[pred_col]
            self.test_data[f'{model_name}_abs_error'] = np.abs(self.test_data[f'{model_name}_error'])
            self.test_data[f'{model_name}_abs_pct_error'] = np.abs(self.test_data[f'{model_name}_error'] / self.test_data[self.actual_col] * 100)
        
        print(f"   ✅ Enhanced analysis data prepared: {len(self.test_data):,} records")
        print(f"   📊 Enhanced ML models found: {len(ml_pred_cols)}")
    
    def analyze_marketing_model_effectiveness(self):
        """Analyze effectiveness of different marketing transformation approaches"""
        print("\n💰 MARKETING TRANSFORMATION MODEL EFFECTIVENESS")
        print("-" * 55)
        
        marketing_models = [name for name in self.model_results.keys() if 
                          any(keyword in name for keyword in ['Marketing', 'Log-Log', 'Hybrid'])]
        
        if not marketing_models:
            print("❌ No marketing transformation models found")
            return {}
        
        # Benchmark performance
        benchmark_mae = self.test_data['benchmark_abs_error'].mean()
        
        marketing_analysis = {}
        
        print("📊 MARKETING MODEL COMPARISON:")
        print("-" * 35)
        
        for model_name in marketing_models:
            if f'{model_name}_abs_error' in self.test_data.columns:
                ml_mae = self.test_data[f'{model_name}_abs_error'].mean()
                improvement = ((benchmark_mae - ml_mae) / benchmark_mae) * 100
                
                # Analyze by marketing spend levels
                if 'marketing_spend' in self.test_data.columns or any('marketing' in col for col in self.test_data.columns):
                    # Find a marketing spend column
                    marketing_col = None
                    for col in self.test_data.columns:
                        if 'marketing' in col.lower() and 'spend' in col.lower():
                            marketing_col = col
                            break
                    
                    if marketing_col:
                        # Analyze performance by marketing spend quintiles
                        quintiles = pd.qcut(self.test_data[marketing_col], 5, labels=['Low', 'Low-Med', 'Medium', 'Med-High', 'High'])
                        quintile_performance = {}
                        
                        for quintile in ['Low', 'Low-Med', 'Medium', 'Med-High', 'High']:
                            quintile_data = self.test_data[quintiles == quintile]
                            if len(quintile_data) > 0:
                                quintile_mae = quintile_data[f'{model_name}_abs_error'].mean()
                                quintile_benchmark = quintile_data['benchmark_abs_error'].mean()
                                quintile_improvement = ((quintile_benchmark - quintile_mae) / quintile_benchmark) * 100
                                quintile_performance[quintile] = quintile_improvement
                
                transformation_type = "Log transformation" if "Log" in model_name else \
                                    "Quadratic transformation" if "Quadratic" in model_name else \
                                    "Log-log transformation" if "Log-Log" in model_name else \
                                    "Hybrid approach"
                
                print(f"{model_name}:")
                print(f"   Transformation: {transformation_type}")
                print(f"   Overall improvement: {improvement:+.1f}%")
                print(f"   MAE: ${ml_mae:,.2f}")
                
                if 'quintile_performance' in locals():
                    print(f"   Performance by marketing spend:")
                    for quintile, perf in quintile_performance.items():
                        print(f"     {quintile} spend: {perf:+.1f}%")
                
                marketing_analysis[model_name] = {
                    'improvement': improvement,
                    'mae': ml_mae,
                    'transformation_type': transformation_type,
                    'quintile_performance': quintile_performance if 'quintile_performance' in locals() else {}
                }
                print("")
        
        # Determine best marketing approach
        if marketing_analysis:
            best_marketing_model = max(marketing_analysis.keys(), 
                                     key=lambda x: marketing_analysis[x]['improvement'])
            
            print(f"🏆 BEST MARKETING TRANSFORMATION:")
            print(f"   Model: {best_marketing_model}")
            print(f"   Improvement: {marketing_analysis[best_marketing_model]['improvement']:+.1f}%")
            print(f"   Approach: {marketing_analysis[best_marketing_model]['transformation_type']}")
        
        return marketing_analysis
    
    def analyze_q4_seasonality_effectiveness(self):
        """Analyze Q4 and holiday seasonality model effectiveness"""
        print("\n🎄 Q4 SEASONALITY EFFECTIVENESS ANALYSIS")
        print("-" * 45)
        
        # Filter test data for Q4 months
        q4_data = self.test_data[self.test_data['order_date'].dt.quarter == 4]
        non_q4_data = self.test_data[self.test_data['order_date'].dt.quarter != 4]
        
        if len(q4_data) == 0:
            print("❌ No Q4 data available for analysis")
            return {}
        
        print(f"📊 Q4 Data: {len(q4_data):,} records")
        print(f"📊 Non-Q4 Data: {len(non_q4_data):,} records")
        print("")
        
        q4_analysis = {}
        
        # Find Q4-enhanced models
        q4_models = [name for name in self.model_results.keys() if 'Q4' in name or 'Enhanced' in name]
        
        benchmark_mae_q4 = q4_data['benchmark_abs_error'].mean()
        benchmark_mae_non_q4 = non_q4_data['benchmark_abs_error'].mean()
        
        print("🎯 BENCHMARK PERFORMANCE:")
        print(f"   Q4 MAE: ${benchmark_mae_q4:,.2f}")
        print(f"   Non-Q4 MAE: ${benchmark_mae_non_q4:,.2f}")
        print(f"   Q4 vs Non-Q4 difference: {((benchmark_mae_q4 - benchmark_mae_non_q4) / benchmark_mae_non_q4) * 100:+.1f}%")
        print("")
        
        # Analyze all models for Q4 effectiveness
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        print("🤖 ML MODEL Q4 vs NON-Q4 PERFORMANCE:")
        print("-" * 40)
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            # Q4 performance
            q4_ml_mae = q4_data[f'{model_name}_abs_error'].mean()
            q4_improvement = ((benchmark_mae_q4 - q4_ml_mae) / benchmark_mae_q4) * 100
            
            # Non-Q4 performance
            non_q4_ml_mae = non_q4_data[f'{model_name}_abs_error'].mean()
            non_q4_improvement = ((benchmark_mae_non_q4 - non_q4_ml_mae) / benchmark_mae_non_q4) * 100
            
            # Q4 specialization score (how much better in Q4 vs non-Q4)
            q4_specialization = q4_improvement - non_q4_improvement
            
            print(f"{model_name}:")
            print(f"   Q4 improvement: {q4_improvement:+.1f}% (MAE: ${q4_ml_mae:,.2f})")
            print(f"   Non-Q4 improvement: {non_q4_improvement:+.1f}% (MAE: ${non_q4_ml_mae:,.2f})")
            print(f"   Q4 specialization: {q4_specialization:+.1f}% {'✅' if q4_specialization > 0 else '❌'}")
            print("")
            
            q4_analysis[model_name] = {
                'q4_improvement': q4_improvement,
                'non_q4_improvement': non_q4_improvement,
                'q4_specialization': q4_specialization,
                'q4_mae': q4_ml_mae,
                'non_q4_mae': non_q4_ml_mae
            }
        
        # Find best Q4 specialist
        if q4_analysis:
            best_q4_model = max(q4_analysis.keys(), 
                               key=lambda x: q4_analysis[x]['q4_specialization'])
            
            print(f"🏆 BEST Q4 SPECIALIST MODEL:")
            print(f"   Model: {best_q4_model}")
            print(f"   Q4 specialization score: {q4_analysis[best_q4_model]['q4_specialization']:+.1f}%")
            
            if q4_analysis[best_q4_model]['q4_specialization'] > 5:
                print("   ✅ Strong Q4 seasonality captured")
            elif q4_analysis[best_q4_model]['q4_specialization'] > 0:
                print("   🟡 Moderate Q4 seasonality effects")
            else:
                print("   ❌ No significant Q4 specialization")
        
        return q4_analysis
    
    def analyze_prophet_effectiveness(self):
        """Analyze Prophet model effectiveness and time series insights"""
        print("\n📈 PROPHET MODEL EFFECTIVENESS ANALYSIS")
        print("-" * 45)
        
        prophet_models = [name for name in self.model_results.keys() if 'Prophet' in name]
        
        if not prophet_models:
            print("❌ No Prophet models found for analysis")
            return {}
        
        prophet_analysis = {}
        benchmark_mae = self.test_data['benchmark_abs_error'].mean()
        
        for prophet_model in prophet_models:
            if f'{prophet_model}_abs_error' in self.test_data.columns:
                prophet_mae = self.test_data[f'{prophet_model}_abs_error'].mean()
                improvement = ((benchmark_mae - prophet_mae) / benchmark_mae) * 100
                
                print(f"{prophet_model}:")
                print(f"   Overall improvement: {improvement:+.1f}%")
                print(f"   MAE: ${prophet_mae:,.2f}")
                
                # Analyze Prophet performance by time periods
                monthly_performance = []
                
                for month in sorted(self.test_data['year_month'].unique()):
                    month_data = self.test_data[self.test_data['year_month'] == month]
                    
                    if len(month_data) > 0:
                        month_benchmark = month_data['benchmark_abs_error'].mean()
                        month_prophet = month_data[f'{prophet_model}_abs_error'].mean()
                        month_improvement = ((month_benchmark - month_prophet) / month_benchmark) * 100
                        
                        monthly_performance.append({
                            'month': month,
                            'improvement': month_improvement,
                            'mae': month_prophet
                        })
                
                # Calculate consistency
                improvements = [perf['improvement'] for perf in monthly_performance]
                consistency_score = 100 - np.std(improvements)  # Lower std = higher consistency
                
                print(f"   Monthly consistency: {consistency_score:.1f}/100")
                print(f"   Best month: {max(monthly_performance, key=lambda x: x['improvement'])['month']} "
                      f"({max(improvements):+.1f}%)")
                print(f"   Worst month: {min(monthly_performance, key=lambda x: x['improvement'])['month']} "
                      f"({min(improvements):+.1f}%)")
                print("")
                
                prophet_analysis[prophet_model] = {
                    'overall_improvement': improvement,
                    'mae': prophet_mae,
                    'monthly_performance': monthly_performance,
                    'consistency_score': consistency_score
                }
        
        return prophet_analysis
    
    def run_enhanced_comprehensive_analysis(self):
        """Run all enhanced analyses"""
        print("🚀 RUNNING ENHANCED COMPREHENSIVE ML MODEL ANALYSIS")
        print("="*65)
        
        results = {}
        
        try:
            # Standard analyses
            results['overall_accuracy'] = self.analyze_overall_accuracy()
            results['weighted_accuracy'] = self.analyze_weighted_accuracy()
            results['channel_performance'] = self.analyze_channel_performance()
            results['top_sku_performance'] = self.analyze_top_sku_performance()
            results['temporal_performance'] = self.analyze_temporal_performance()
            
            # Enhanced analyses
            results['marketing_effectiveness'] = self.analyze_marketing_model_effectiveness()
            results['q4_seasonality'] = self.analyze_q4_seasonality_effectiveness()
            results['prophet_effectiveness'] = self.analyze_prophet_effectiveness()
            
            # Final enhanced recommendations
            self.provide_enhanced_recommendations(results)
            
            return results
            
        except Exception as e:
            print(f"❌ Error during enhanced analysis: {str(e)}")
            return results
    
    def provide_enhanced_recommendations(self, results):
        """Provide enhanced recommendations based on all analyses"""
        print("\n🎯 ENHANCED MODEL RECOMMENDATIONS")
        print("="*45)
        
        print("📊 MARKETING TRANSFORMATION RECOMMENDATIONS:")
        marketing_results = results.get('marketing_effectiveness', {})
        if marketing_results:
            best_marketing = max(marketing_results.keys(), 
                               key=lambda x: marketing_results[x]['improvement'])
            print(f"   🏆 Best marketing approach: {best_marketing}")
            print(f"   📈 Improvement: {marketing_results[best_marketing]['improvement']:+.1f}%")
            print(f"   💡 Strategy: {marketing_results[best_marketing]['transformation_type']}")
        else:
            print("   ⚠️ No marketing transformation analysis available")
        
        print("\n🎄 Q4 SEASONALITY RECOMMENDATIONS:")
        q4_results = results.get('q4_seasonality', {})
        if q4_results:
            best_q4_specialist = max(q4_results.keys(), 
                                   key=lambda x: q4_results[x]['q4_specialization'])
            specialization_score = q4_results[best_q4_specialist]['q4_specialization']
            
            print(f"   🏆 Best Q4 model: {best_q4_specialist}")
            print(f"   🎯 Q4 specialization: {specialization_score:+.1f}%")
            
            if specialization_score > 5:
                print("   ✅ Strong Q4 effects - implement separate holiday models")
            elif specialization_score > 0:
                print("   🟡 Moderate Q4 effects - consider Q4 feature engineering")
            else:
                print("   ❌ Weak Q4 effects - focus on other factors")
        
        print("\n📈 PROPHET RECOMMENDATIONS:")
        prophet_results = results.get('prophet_effectiveness', {})
        if prophet_results:
            best_prophet = max(prophet_results.keys(), 
                             key=lambda x: prophet_results[x]['overall_improvement'])
            prophet_improvement = prophet_results[best_prophet]['overall_improvement']
            consistency = prophet_results[best_prophet]['consistency_score']
            
            print(f"   🏆 Best Prophet model: {best_prophet}")
            print(f"   📈 Improvement: {prophet_improvement:+.1f}%")
            print(f"   📊 Consistency: {consistency:.1f}/100")
            
            if prophet_improvement > 10 and consistency > 70:
                print("   ✅ Prophet highly effective - use for forecasting")
            elif prophet_improvement > 0:
                print("   🟡 Prophet moderately effective - use as ensemble component")
            else:
                print("   ❌ Prophet ineffective - stick to ML models")
        
        print("\n🏆 OVERALL STRATEGY RECOMMENDATIONS:")
        
        # Find overall best model
        overall_results = results.get('overall_accuracy', {})
        if overall_results:
            # Remove benchmark from comparison
            ml_results = {k: v for k, v in overall_results.items() if k != 'benchmark'}
            
            if ml_results:
                best_overall = max(ml_results.keys(), 
                                 key=lambda x: ml_results[x].get('mae_improvement_pct', 0))
                best_improvement = ml_results[best_overall].get('mae_improvement_pct', 0)
                
                print(f"   🥇 Best overall model: {best_overall}")
                print(f"   📈 Best improvement: {best_improvement:+.1f}%")
                
                if best_improvement > 15:
                    strategy = "DEPLOY IMMEDIATELY"
                    confidence = "HIGH"
                elif best_improvement > 5:
                    strategy = "PILOT DEPLOYMENT"
                    confidence = "MEDIUM"
                else:
                    strategy = "CONTINUE DEVELOPMENT"
                    confidence = "LOW"
                
                print(f"   🎯 Recommended strategy: {strategy}")
                print(f"   🎪 Confidence level: {confidence}")
    
    # Include all existing methods from the original MLModelEvaluationAnalysis class
    def analyze_overall_accuracy(self):
        """1. Overall Accuracy Analysis"""
        print("\n📈 1. OVERALL ACCURACY ANALYSIS")
        print("-" * 45)
        
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        if not ml_pred_cols:
            print("❌ No ML model predictions found")
            return {}
        
        accuracy_results = {}
        
        # Benchmark performance
        benchmark_mae = self.test_data['benchmark_abs_error'].mean()
        benchmark_mape = self.test_data['benchmark_abs_pct_error'].mean()
        
        print(f"🎯 BENCHMARK PERFORMANCE:")
        print(f"   MAE:  ${benchmark_mae:,.2f}")
        print(f"   MAPE: {benchmark_mape:.1f}%")
        print("")
        
        accuracy_results['benchmark'] = {'mae': benchmark_mae, 'mape': benchmark_mape}
        
        # ML model performance
        print(f"🤖 ML MODEL PERFORMANCE:")
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            ml_mae = self.test_data[f'{model_name}_abs_error'].mean()
            ml_mape = self.test_data[f'{model_name}_abs_pct_error'].mean()
            
            mae_improvement = benchmark_mae - ml_mae
            mae_improvement_pct = (mae_improvement / benchmark_mae) * 100
            mape_improvement = benchmark_mape - ml_mape
            
            status = "✅ BETTER" if mae_improvement > 0 else "❌ WORSE"
            
            print(f"   {model_name}:")
            print(f"     MAE:  ${ml_mae:,.2f} (${mae_improvement:+,.2f}, {mae_improvement_pct:+.1f}%) {status}")
            print(f"     MAPE: {ml_mape:.1f}% ({mape_improvement:+.1f}%)")
            print("")
            
            accuracy_results[model_name] = {
                'mae': ml_mae,
                'mape': ml_mape,
                'mae_improvement': mae_improvement,
                'mae_improvement_pct': mae_improvement_pct,
                'mape_improvement': mape_improvement
            }
        
        return accuracy_results
    
    def analyze_weighted_accuracy(self):
        """2. Weighted Accuracy by Product Revenue"""
        print("\n💰 2. WEIGHTED ACCURACY BY PRODUCT IMPORTANCE")
        print("-" * 55)
        
        if 'sku' not in self.test_data.columns:
            print("❌ No SKU column found for product analysis")
            return {}
        
        # Calculate product weights by revenue
        product_revenue = self.test_data.groupby('sku')[self.actual_col].sum().sort_values(ascending=False)
        top_20_pct_threshold = product_revenue.quantile(0.8)
        top_products = product_revenue[product_revenue >= top_20_pct_threshold].index
        
        top_product_data = self.test_data[self.test_data['sku'].isin(top_products)]
        bottom_product_data = self.test_data[~self.test_data['sku'].isin(top_products)]
        
        print(f"📊 PRODUCT SEGMENTATION:")
        print(f"   Top 20% Products: {len(top_products)} SKUs")
        print(f"   Bottom 80% Products: {len(product_revenue) - len(top_products)} SKUs")
        print("")
        
        weighted_results = {}
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for segment_name, segment_data in [('Top 20% Products', top_product_data), ('Bottom 80% Products', bottom_product_data)]:
            print(f"🎯 {segment_name.upper()}:")
            
            bench_mae = segment_data['benchmark_abs_error'].mean()
            print(f"   Benchmark MAE: ${bench_mae:.2f}")
            
            segment_results = {'benchmark': {'mae': bench_mae}}
            
            for pred_col in ml_pred_cols:
                model_name = pred_col.replace('_pred', '')
                ml_mae = segment_data[f'{model_name}_abs_error'].mean()
                improvement_pct = ((bench_mae - ml_mae) / bench_mae) * 100
                
                status = "✅" if improvement_pct > 0 else "❌"
                print(f"   {model_name}: ${ml_mae:.2f} ({improvement_pct:+.1f}%) {status}")
                
                segment_results[model_name] = {'mae': ml_mae, 'improvement_pct': improvement_pct}
            
            weighted_results[segment_name] = segment_results
            print("")
        
        return weighted_results
    
    def analyze_channel_performance(self):
        """3. Channel Performance Analysis"""
        print("\n📺 3. CHANNEL PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        if 'channel' not in self.test_data.columns:
            print("❌ No channel column found")
            return {}
        
        channel_results = {}
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for channel in self.test_data['channel'].unique():
            channel_data = self.test_data[self.test_data['channel'] == channel]
            
            print(f"📻 {channel.upper()} CHANNEL:")
            print(f"   Records: {len(channel_data):,}")
            
            bench_mae = channel_data['benchmark_abs_error'].mean()
            print(f"   Benchmark MAE: ${bench_mae:.2f}")
            
            channel_results[channel] = {'benchmark': {'mae': bench_mae}}
            
            for pred_col in ml_pred_cols:
                model_name = pred_col.replace('_pred', '')
                ml_mae = channel_data[f'{model_name}_abs_error'].mean()
                improvement_pct = ((bench_mae - ml_mae) / bench_mae) * 100
                
                status = "✅ BETTER" if improvement_pct > 0 else "❌ WORSE"
                print(f"   {model_name}: ${ml_mae:.2f} ({improvement_pct:+.1f}%) {status}")
                
                channel_results[channel][model_name] = {'mae': ml_mae, 'improvement_pct': improvement_pct}
            
            print("")
        
        return channel_results
    
    def analyze_top_sku_performance(self):
        """4. Top SKU Performance Analysis"""
        print("\n🏆 4. TOP SKU PERFORMANCE ANALYSIS")
        print("-" * 40)
        
        # Get top 10 SKUs by revenue
        sku_revenue = self.test_data.groupby('sku')[self.actual_col].sum().sort_values(ascending=False)
        top_10_skus = sku_revenue.head(10).index
        
        print(f"📊 TOP 10 SKUs by Revenue:")
        for i, sku in enumerate(top_10_skus, 1):
            revenue = sku_revenue[sku]
            pct_of_total = (revenue / sku_revenue.sum()) * 100
            print(f"   {i:2d}. {sku}: ${revenue:,.0f} ({pct_of_total:.1f}% of total)")
        print("")
        
        sku_results = {}
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        # Analyze each top SKU individually
        for rank, sku in enumerate(top_10_skus, 1):
            sku_data = self.test_data[self.test_data['sku'] == sku]
            
            if len(sku_data) < 5:
                continue
            
            bench_mae = sku_data['benchmark_abs_error'].mean()
            sku_results[sku] = {
                'rank': rank,
                'revenue': sku_revenue[sku],
                'benchmark': {'mae': bench_mae}
            }
            
            for pred_col in ml_pred_cols:
                model_name = pred_col.replace('_pred', '')
                ml_mae = sku_data[f'{model_name}_abs_error'].mean()
                improvement_pct = ((bench_mae - ml_mae) / bench_mae) * 100
                
                sku_results[sku][model_name] = {
                    'mae': ml_mae,
                    'improvement_pct': improvement_pct
                }
        
        return sku_results
    
    def analyze_temporal_performance(self):
        """5. Temporal Performance Analysis"""
        print("\n📅 5. TEMPORAL PERFORMANCE ANALYSIS")
        print("-" * 45)
        
        temporal_results = {}
        ml_pred_cols = [col for col in self.test_data.columns if col.endswith('_pred')]
        
        for pred_col in ml_pred_cols:
            model_name = pred_col.replace('_pred', '')
            
            monthly_comparison = []
            
            for month in sorted(self.test_data['year_month'].unique()):
                month_data = self.test_data[self.test_data['year_month'] == month]
                
                if len(month_data) == 0:
                    continue
                
                bench_mae = month_data['benchmark_abs_error'].mean()
                ml_mae = month_data[f'{model_name}_abs_error'].mean()
                improvement_pct = ((bench_mae - ml_mae) / bench_mae) * 100
                
                monthly_comparison.append({
                    'month': month,
                    'improvement_pct': improvement_pct,
                    'wins_benchmark': improvement_pct > 0
                })
            
            comparison_df = pd.DataFrame(monthly_comparison)
            
            if len(comparison_df) > 0:
                win_rate = (comparison_df['wins_benchmark'].sum() / len(comparison_df)) * 100
                avg_improvement = comparison_df['improvement_pct'].mean()
                
                temporal_results[model_name] = {
                    'win_rate': win_rate,
                    'avg_improvement': avg_improvement,
                    'monthly_details': comparison_df
                }
        
        return temporal_results


# Main Enhanced Pipeline Function
def run_enhanced_demand_analysis_with_prophet_and_marketing(
    benchmark_df, 
    amazon_order_items=None, tiktok_order_items=None, shopify_order_items=None,
    amazon_daily_sku=None, tiktok_daily_sku=None, shopify_daily_sku=None
):
    """
    Complete enhanced demand analysis pipeline with Prophet and marketing transformations
    
    New Features:
    - Prophet time series forecasting
    - Marketing spend transformations (log, quadratic, log-log)
    - Enhanced Q4/holiday seasonality
    - Diminishing returns modeling
    """
    
    print("🚀 ENHANCED SKU DEMAND ANALYSIS WITH PROPHET & MARKETING")
    print("🎯 NEW: Prophet forecasting + Marketing transformations + Q4 analysis")
    print("="*85)
    
    # Step 1: Train enhanced models
    print("\n" + "="*85)
    print("STEP 1: ENHANCED MODEL TRAINING WITH MARKETING & PROPHET")
    print("="*85)
    
    enhanced_modeler = EnhancedDemandModels(
        benchmark_df, 
        amazon_order_items, tiktok_order_items, shopify_order_items,
        amazon_daily_sku, tiktok_daily_sku, shopify_daily_sku
    )
    
    enhanced_model_results, enhanced_test_data = enhanced_modeler.evaluate_enhanced_models()
    
    # Step 2: Comprehensive enhanced evaluation
    print("\n" + "="*85)
    print("STEP 2: ENHANCED COMPREHENSIVE EVALUATION")
    print("="*85)
    
    enhanced_analyzer = EnhancedMLModelEvaluationAnalysis(
        enhanced_test_data, enhanced_model_results
    )
    enhanced_analysis_results = enhanced_analyzer.run_enhanced_comprehensive_analysis()
    
    # Step 3: Marketing insights extraction
    print("\n" + "="*85)
    print("STEP 3: MARKETING EFFECTIVENESS INSIGHTS")
    print("="*85)
    
    marketing_insights = enhanced_modeler.get_marketing_insights()
    
    print("\n🎉 ENHANCED ANALYSIS COMPLETE!")
    print("📊 Enhanced features delivered:")
    print("   ✅ Prophet time series forecasting")
    print("   ✅ Marketing spend transformations (log, quadratic, log-log)")
    print("   ✅ Enhanced Q4 and holiday seasonality")
    print("   ✅ Diminishing returns modeling")
    print("   ✅ Marketing effectiveness analysis")
    print("   ✅ Q4 specialization scoring")
    print("   ✅ Prophet vs ML model comparison")
    
    return enhanced_modeler, enhanced_analyzer, enhanced_analysis_results, marketing_insights


# Quick usage example:

enhanced_modeler, enhanced_analyzer, enhanced_results, marketing_insights = run_enhanced_demand_analysis_with_prophet_and_marketing(
    benchmark_df=your_benchmark_data,
    amazon_order_items=your_amazon_data,
    # ... other channel data
)

# Extract specific insights
marketing_effectiveness = enhanced_results['marketing_effectiveness']
q4_seasonality = enhanced_results['q4_seasonality']
prophet_performance = enhanced_results['prophet_effectiveness']

✅ Prophet library available


"\n# Run the enhanced analysis\nenhanced_modeler, enhanced_analyzer, enhanced_results, marketing_insights = run_enhanced_demand_analysis_with_prophet_and_marketing(\n    benchmark_df=your_benchmark_data,\n    amazon_order_items=your_amazon_data,\n    # ... other channel data\n)\n\n# Extract specific insights\nmarketing_effectiveness = enhanced_results['marketing_effectiveness']\nq4_seasonality = enhanced_results['q4_seasonality']\nprophet_performance = enhanced_results['prophet_effectiveness']\n"

In [ ]:
enhanced_modeler, enhanced_analyzer, enhanced_results, marketing_insights = run_enhanced_demand_analysis_with_prophet_and_marketing(
    benchmark_df=enhanced_benchmark_df,
    amazon_order_items=amazon_order_item_metrics,
    tiktok_order_items=tiktok__order_items,
    shopify_order_items=shopify__order_items,
    amazon_daily_sku=amazon_daily_sku_metrics,
    tiktok_daily_sku=tiktok_daily_sku_metrics,
    shopify_daily_sku=shopify_daily_sku_metrics
)

# Get specific insights
marketing_effectiveness = enhanced_results['marketing_effectiveness']
q4_performance = enhanced_results['q4_seasonality'] 
prophet_insights = enhanced_results['prophet_effectiveness']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

class ComprehensiveModelAnalysis:
    """Analyze performance patterns across all models to identify universal insights"""
    
    def __init__(self, test_data, model_results, target_col='demand_pct_change'):
        self.test_data = test_data.copy()
        self.model_results = model_results
        self.target_col = target_col
        self.model_names = list(model_results.keys())
        
        # Add all model predictions to test data
        for model_name, results in model_results.items():
            if 'predictions' in results and len(results['predictions']) == len(test_data):
                self.test_data[f'{model_name}_pred'] = results['predictions']
                self.test_data[f'{model_name}_error'] = self.test_data[target_col] - results['predictions']
                self.test_data[f'{model_name}_abs_error'] = np.abs(self.test_data[f'{model_name}_error'])
    
    def analyze_universal_sku_predictability(self):
        """Identify SKUs that are consistently predictable/unpredictable across ALL models"""
        print("UNIVERSAL SKU PREDICTABILITY ANALYSIS")
        print("="*45)
        
        # Calculate MAE for each SKU across all models
        sku_performance_all_models = {}
        
        for sku in self.test_data['sku'].unique():
            sku_data = self.test_data[self.test_data['sku'] == sku]
            
            if len(sku_data) < 5:  # Need minimum observations
                continue
                
            sku_performance = {}
            for model_name in self.model_names:
                if f'{model_name}_abs_error' in sku_data.columns:
                    sku_performance[model_name] = sku_data[f'{model_name}_abs_error'].mean()
            
            if len(sku_performance) >= 3:  # Need at least 3 models
                sku_performance_all_models[sku] = sku_performance
        
        # Create SKU performance matrix
        sku_performance_df = pd.DataFrame(sku_performance_all_models).T
        sku_performance_df['mean_mae'] = sku_performance_df.mean(axis=1)
        sku_performance_df['mae_std'] = sku_performance_df.std(axis=1)
        sku_performance_df['n_observations'] = [
            len(self.test_data[self.test_data['sku'] == sku]) for sku in sku_performance_df.index
        ]
        
        # Sort by mean performance across all models
        sku_performance_df = sku_performance_df.sort_values('mean_mae')
        
        print(f"Analyzed {len(sku_performance_df)} SKUs across {len(self.model_names)} models")
        print()
        
        print("TOP 10 UNIVERSALLY PREDICTABLE SKUs:")
        print("-" * 40)
        for sku, row in sku_performance_df.head(10).iterrows():
            consistency = "High" if row['mae_std'] < 2 else "Medium" if row['mae_std'] < 5 else "Low"
            print(f"{sku}: {row['mean_mae']:.1f}% avg MAE (consistency: {consistency}, n={row['n_observations']})")
        
        print("\nTOP 10 UNIVERSALLY UNPREDICTABLE SKUs:")
        print("-" * 42)
        for sku, row in sku_performance_df.tail(10).iterrows():
            consistency = "High" if row['mae_std'] < 2 else "Medium" if row['mae_std'] < 5 else "Low"
            print(f"{sku}: {row['mean_mae']:.1f}% avg MAE (consistency: {consistency}, n={row['n_observations']})")
        
        # Analyze what makes SKUs predictable
        self._analyze_predictability_drivers(sku_performance_df)
        
        return sku_performance_df
    
    def _analyze_predictability_drivers(self, sku_performance_df):
        """Analyze what characteristics drive SKU predictability"""
        print("\nPREDICTABILITY DRIVERS ANALYSIS:")
        print("-" * 35)
        
        # Merge with SKU characteristics
        sku_chars = self.test_data.groupby('sku').agg({
            'actual_demand': ['mean', 'std', 'sum'],
            self.target_col: ['mean', 'std'],
            'order_date': 'count'
        }).round(2)
        
        sku_chars.columns = ['avg_demand', 'demand_volatility', 'total_demand', 
                            'avg_pct_change', 'pct_change_volatility', 'n_days']
        
        # Merge performance with characteristics
        analysis_df = sku_performance_df.merge(sku_chars, left_index=True, right_index=True, how='left')
        
        # Correlation analysis
        correlations = analysis_df[['mean_mae', 'avg_demand', 'demand_volatility', 
                                   'total_demand', 'pct_change_volatility', 'n_days']].corr()['mean_mae'].drop('mean_mae')
        
        print("Correlation with Predictability (negative = more predictable):")
        for factor, corr in correlations.sort_values().items():
            direction = "Higher" if corr > 0 else "Lower"
            strength = "Strong" if abs(corr) > 0.3 else "Moderate" if abs(corr) > 0.1 else "Weak"
            print(f"  {factor}: {corr:.3f} ({strength} - {direction} = Less Predictable)")
        
        # Segment analysis
        print("\nPREDICTABILITY BY SEGMENTS:")
        print("-" * 28)
        
        # High vs Low volume
        high_volume = analysis_df[analysis_df['total_demand'] > analysis_df['total_demand'].quantile(0.8)]
        low_volume = analysis_df[analysis_df['total_demand'] < analysis_df['total_demand'].quantile(0.2)]
        
        print(f"High Volume SKUs (top 20%): {high_volume['mean_mae'].mean():.1f}% avg MAE")
        print(f"Low Volume SKUs (bottom 20%): {low_volume['mean_mae'].mean():.1f}% avg MAE")
        
        # High vs Low volatility
        high_volatility = analysis_df[analysis_df['pct_change_volatility'] > analysis_df['pct_change_volatility'].quantile(0.8)]
        low_volatility = analysis_df[analysis_df['pct_change_volatility'] < analysis_df['pct_change_volatility'].quantile(0.2)]
        
        print(f"High Volatility SKUs: {high_volatility['mean_mae'].mean():.1f}% avg MAE")
        print(f"Low Volatility SKUs: {low_volatility['mean_mae'].mean():.1f}% avg MAE")
    
    def analyze_universal_prediction_degradation(self):
        """Analyze when prediction accuracy starts degrading across ALL models"""
        print("\nUNIVERSAL PREDICTION DEGRADATION ANALYSIS")
        print("="*45)
        
        # Sort by date and calculate prediction horizons
        test_sorted = self.test_data.sort_values('order_date').copy()
        training_end = test_sorted['order_date'].min()
        test_sorted['days_from_training'] = (test_sorted['order_date'] - training_end).dt.days
        
        # Define time bins
        bins = [0, 7, 14, 30, 60, 90, float('inf')]
        labels = ['0-7 days', '8-14 days', '15-30 days', '31-60 days', '61-90 days', '90+ days']
        test_sorted['time_bin'] = pd.cut(test_sorted['days_from_training'], bins=bins, labels=labels, right=False)
        
        # Calculate performance for each model across time bins
        degradation_results = {}
        
        for model_name in self.model_names:
            error_col = f'{model_name}_abs_error'
            if error_col in test_sorted.columns:
                model_degradation = test_sorted.groupby('time_bin')[error_col].agg(['mean', 'count']).reset_index()
                model_degradation.columns = ['time_bin', 'mae', 'n_predictions']
                degradation_results[model_name] = model_degradation
        
        # Calculate consensus degradation pattern
        consensus_degradation = []
        
        for i, time_bin in enumerate(labels):
            all_model_maes = []
            total_predictions = 0
            
            for model_name, degradation_df in degradation_results.items():
                if i < len(degradation_df):
                    mae_value = degradation_df.iloc[i]['mae']
                    n_pred = degradation_df.iloc[i]['n_predictions']
                    if not pd.isna(mae_value) and n_pred > 10:  # Minimum sample size
                        all_model_maes.append(mae_value)
                        total_predictions += n_pred
            
            if all_model_maes:
                consensus_mae = np.mean(all_model_maes)
                consensus_std = np.std(all_model_maes)
                consensus_degradation.append({
                    'time_bin': time_bin,
                    'consensus_mae': consensus_mae,
                    'mae_std_across_models': consensus_std,
                    'n_models_agreeing': len(all_model_maes),
                    'total_predictions': total_predictions
                })
        
        consensus_df = pd.DataFrame(consensus_degradation)
        
        print("CONSENSUS DEGRADATION PATTERN:")
        print("-" * 32)
        
        if len(consensus_df) > 0:
            baseline_mae = consensus_df.iloc[0]['consensus_mae']
            
            for _, row in consensus_df.iterrows():
                degradation_pct = ((row['consensus_mae'] - baseline_mae) / baseline_mae) * 100
                model_agreement = row['n_models_agreeing']
                
                agreement_level = "High" if model_agreement >= len(self.model_names) * 0.8 else "Medium" if model_agreement >= len(self.model_names) * 0.6 else "Low"
                
                print(f"{row['time_bin']}: {row['consensus_mae']:.1f}% MAE "
                      f"({degradation_pct:+.1f}% vs baseline) "
                      f"[{model_agreement}/{len(self.model_names)} models, {agreement_level} agreement]")
            
            # Calculate degradation rate
            if len(consensus_df) >= 3:
                early_performance = consensus_df.iloc[0]['consensus_mae']
                month_performance = consensus_df.iloc[2]['consensus_mae']  # 15-30 days
                degradation_rate = ((month_performance - early_performance) / early_performance) * 100
                
                print(f"\nCONSENSUS DEGRADATION RATE: {degradation_rate:+.1f}% over first month")
                
                if degradation_rate > 15:
                    print("  Recommendation: WEEKLY retraining (high degradation)")
                elif degradation_rate > 8:
                    print("  Recommendation: BI-WEEKLY retraining (moderate degradation)")
                elif degradation_rate > 3:
                    print("  Recommendation: MONTHLY retraining (low degradation)")
                else:
                    print("  Recommendation: QUARTERLY retraining (very stable)")
        
        return consensus_df
    
    def analyze_seasonal_pattern_capture(self):
        """Analyze how well models capture seasonal patterns across the board"""
        print("\nSEASONAL PATTERN CAPTURE ANALYSIS")
        print("="*37)
        
        # Add time features
        test_data = self.test_data.copy()
        test_data['month'] = test_data['order_date'].dt.month
        test_data['quarter'] = test_data['order_date'].dt.quarter
        test_data['day_of_week'] = test_data['order_date'].dt.dayofweek
        
        # Analyze actual seasonal patterns in target variable
        actual_seasonality = test_data.groupby('month')[self.target_col].agg(['mean', 'count']).reset_index()
        actual_seasonality.columns = ['month', 'actual_avg_change', 'n_observations']
        actual_seasonality = actual_seasonality[actual_seasonality['n_observations'] >= 10]
        
        print("ACTUAL SEASONAL PATTERNS:")
        print("-" * 27)
        month_names = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 
                      7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
        
        for _, row in actual_seasonality.iterrows():
            month_name = month_names.get(row['month'], str(row['month']))
            print(f"{month_name}: {row['actual_avg_change']:+.1f}% avg change ({row['n_observations']} obs)")
        
        # Analyze how well each model predicts these seasonal patterns
        seasonal_capture_results = {}
        
        for model_name in self.model_names:
            pred_col = f'{model_name}_pred'
            if pred_col in test_data.columns:
                predicted_seasonality = test_data.groupby('month')[pred_col].mean().reset_index()
                predicted_seasonality.columns = ['month', 'predicted_avg_change']
                
                # Merge with actual seasonality
                seasonal_comparison = actual_seasonality.merge(predicted_seasonality, on='month', how='inner')
                
                if len(seasonal_comparison) >= 6:  # Need at least 6 months
                    # Calculate correlation between actual and predicted seasonal patterns
                    seasonal_correlation = seasonal_comparison['actual_avg_change'].corr(
                        seasonal_comparison['predicted_avg_change']
                    )
                    
                    # Calculate MAE of seasonal pattern prediction
                    seasonal_mae = np.mean(np.abs(
                        seasonal_comparison['actual_avg_change'] - seasonal_comparison['predicted_avg_change']
                    ))
                    
                    seasonal_capture_results[model_name] = {
                        'seasonal_correlation': seasonal_correlation,
                        'seasonal_mae': seasonal_mae,
                        'comparison_df': seasonal_comparison
                    }
        
        print(f"\nSEASONAL PATTERN CAPTURE BY MODEL:")
        print("-" * 35)
        
        # Sort models by seasonal capture ability
        seasonal_ranking = sorted(seasonal_capture_results.items(), 
                                key=lambda x: x[1]['seasonal_correlation'], reverse=True)
        
        for rank, (model_name, metrics) in enumerate(seasonal_ranking, 1):
            corr = metrics['seasonal_correlation']
            mae = metrics['seasonal_mae']
            
            capture_quality = "Excellent" if corr > 0.8 else "Good" if corr > 0.6 else "Fair" if corr > 0.4 else "Poor"
            
            print(f"{rank}. {model_name}: {corr:.3f} correlation, {mae:.1f}% seasonal MAE ({capture_quality})")
        
        # Overall seasonal capture consensus
        if seasonal_ranking:
            avg_seasonal_corr = np.mean([metrics['seasonal_correlation'] for _, metrics in seasonal_ranking])
            print(f"\nCONSENSUS SEASONAL CAPTURE: {avg_seasonal_corr:.3f} correlation")
            
            if avg_seasonal_corr > 0.7:
                print("  Assessment: STRONG seasonal pattern capture")
            elif avg_seasonal_corr > 0.5:
                print("  Assessment: MODERATE seasonal pattern capture")
            elif avg_seasonal_corr > 0.3:
                print("  Assessment: WEAK seasonal pattern capture")
            else:
                print("  Assessment: POOR seasonal pattern capture - review seasonal features")
        
        # Analyze Q4 vs non-Q4 performance specifically
        self._analyze_q4_performance()
        
        return seasonal_capture_results
    
    def _analyze_q4_performance(self):
        """Analyze Q4 (holiday season) performance specifically"""
        print(f"\nQ4 (HOLIDAY SEASON) PERFORMANCE:")
        print("-" * 33)
        
        test_data = self.test_data.copy()
        test_data['is_q4'] = test_data['order_date'].dt.quarter == 4
        
        q4_performance = {}
        non_q4_performance = {}
        
        for model_name in self.model_names:
            error_col = f'{model_name}_abs_error'
            if error_col in test_data.columns:
                q4_mae = test_data[test_data['is_q4']][error_col].mean()
                non_q4_mae = test_data[~test_data['is_q4']][error_col].mean()
                
                if not pd.isna(q4_mae) and not pd.isna(non_q4_mae):
                    q4_performance[model_name] = q4_mae
                    non_q4_performance[model_name] = non_q4_mae
        
        if q4_performance and non_q4_performance:
            print("Q4 vs Non-Q4 Performance:")
            
            q4_models = []
            for model_name in q4_performance.keys():
                q4_mae = q4_performance[model_name]
                non_q4_mae = non_q4_performance[model_name]
                q4_difference = ((q4_mae - non_q4_mae) / non_q4_mae) * 100
                
                q4_models.append((model_name, q4_mae, non_q4_mae, q4_difference))
                
            # Sort by Q4 performance
            q4_models.sort(key=lambda x: x[1])  # Sort by Q4 MAE
            
            for model_name, q4_mae, non_q4_mae, q4_diff in q4_models:
                performance_change = "worse" if q4_diff > 0 else "better"
                print(f"  {model_name}: Q4 {q4_mae:.1f}%, Non-Q4 {non_q4_mae:.1f}% "
                      f"({q4_diff:+.1f}% {performance_change} in Q4)")
            
            # Overall Q4 assessment
            avg_q4_impact = np.mean([diff for _, _, _, diff in q4_models])
            
            print(f"\nAVERAGE Q4 IMPACT: {avg_q4_impact:+.1f}% change in accuracy")
            
            if avg_q4_impact > 10:
                print("  Q4 Assessment: SIGNIFICANT accuracy drop during holidays")
            elif avg_q4_impact > 5:
                print("  Q4 Assessment: MODERATE accuracy drop during holidays")
            elif avg_q4_impact > -5:
                print("  Q4 Assessment: STABLE performance during holidays")
            else:
                print("  Q4 Assessment: IMPROVED performance during holidays")
    
    def generate_comprehensive_insights(self):
        """Generate overall insights across all analyses"""
        print("\n" + "="*60)
        print("COMPREHENSIVE CROSS-MODEL INSIGHTS")
        print("="*60)
        
        # Run all analyses
        sku_performance = self.analyze_universal_sku_predictability()
        degradation_analysis = self.analyze_universal_prediction_degradation()
        seasonal_analysis = self.analyze_seasonal_pattern_capture()
        
        print("\nKEY INSIGHTS SUMMARY:")
        print("-" * 22)
        
        # SKU Insights
        best_skus = sku_performance.head(5)
        worst_skus = sku_performance.tail(5)
        
        print(f"1. SKU PREDICTABILITY:")
        print(f"   - Best performing SKUs average: {best_skus['mean_mae'].mean():.1f}% MAE")
        print(f"   - Worst performing SKUs average: {worst_skus['mean_mae'].mean():.1f}% MAE")
        print(f"   - Predictability range: {worst_skus['mean_mae'].mean() - best_skus['mean_mae'].mean():.1f}% points")
        
        # Degradation Insights
        if len(degradation_analysis) >= 3:
            early_performance = degradation_analysis.iloc[0]['consensus_mae']
            month_performance = degradation_analysis.iloc[2]['consensus_mae']
            degradation = ((month_performance - early_performance) / early_performance) * 100
            
            print(f"\n2. PREDICTION DEGRADATION:")
            print(f"   - First week performance: {early_performance:.1f}% MAE")
            print(f"   - One month performance: {month_performance:.1f}% MAE")
            print(f"   - Degradation rate: {degradation:+.1f}% over 30 days")
        
        # Seasonal Insights
        if seasonal_analysis:
            seasonal_correlations = [metrics['seasonal_correlation'] for metrics in seasonal_analysis.values()]
            avg_seasonal_capture = np.mean(seasonal_correlations)
            
            print(f"\n3. SEASONAL PATTERN CAPTURE:")
            print(f"   - Average seasonal correlation: {avg_seasonal_capture:.3f}")
            print(f"   - Best seasonal model: {max(seasonal_analysis.items(), key=lambda x: x[1]['seasonal_correlation'])[0]}")
        
        # Model Consensus Insights
        print(f"\n4. MODEL CONSENSUS:")
        all_model_maes = [results['mae'] for results in self.model_results.values()]
        mae_range = max(all_model_maes) - min(all_model_maes)
        
        print(f"   - MAE range across models: {mae_range:.1f}% points")
        
        if mae_range < 2:
            consensus_level = "HIGH - Models largely agree"
        elif mae_range < 5:
            consensus_level = "MODERATE - Some model disagreement"
        else:
            consensus_level = "LOW - Significant model disagreement"
        
        print(f"   - Model consensus level: {consensus_level}")
        
        return {
            'sku_performance': sku_performance,
            'degradation_analysis': degradation_analysis,
            'seasonal_analysis': seasonal_analysis,
            'summary_insights': {
                'best_sku_mae': best_skus['mean_mae'].mean(),
                'worst_sku_mae': worst_skus['mean_mae'].mean(),
                'degradation_rate': degradation if 'degradation' in locals() else None,
                'seasonal_capture': avg_seasonal_capture if 'avg_seasonal_capture' in locals() else None,
                'model_consensus': mae_range
            }
        }


def run_comprehensive_analysis(test_data, model_results, target_col='demand_pct_change'):
    """
    Run comprehensive analysis across all models to identify universal patterns
    
    This will tell you:
    - Which SKUs are predictable across ALL models
    - When prediction accuracy universally starts degrading
    - Whether seasonal patterns are being captured effectively
    - Model consensus and disagreement patterns
    """
    
    analyzer = ComprehensiveModelAnalysis(test_data, model_results, target_col)
    insights = analyzer.generate_comprehensive_insights()
    
    return analyzer, insights


# Usage with your existing results:

# After running your enhanced demand analysis:
enhanced_model, model_results, test_data, analyses = run_enhanced_demand_analysis(
    enhanced_benchmark_df,  # Your benchmark dataframe
    amazon_order_items=amazon_order_item_metrics,
    tiktok_order_items=tiktok__order_items,
    shopify_order_items=shopify__order_items,
    amazon_daily_sku=amazon_daily_sku_metrics,
    tiktok_daily_sku=tiktok_daily_sku_metrics,
    shopify_daily_sku=shopify_daily_sku_metrics
)
# Run comprehensive cross-model analysis:
comprehensive_analyzer, insights = run_comprehensive_analysis(test_data, model_results)

# Access specific insights:
best_skus = insights['sku_performance'].head(10)  # Universally predictable SKUs
degradation_pattern = insights['degradation_analysis']  # When all models degrade
seasonal_capture = insights['seasonal_analysis']  # How well seasonality is captured
summary = insights['summary_insights']  # Key takeaways
